In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# import seaborn as sns 
#sns.set(style="darkgrid")
plt.rcParams["font.sans-serif"]=["SimHei"]
plt.rcParams["axes.unicode_minus"]=False


from dateutil.relativedelta import relativedelta
from datetime import datetime as dt
import time

import warnings  
warnings.filterwarnings("ignore")

import sys
import pymysql
from sqlalchemy import create_engine
import json
import datetime 
from datetime import timedelta ,time 

pd.set_option('display.max_columns', None)

## 定义数据库连接

In [2]:
def query(sql,
          host="rr-wz9wx0w3yti9d4f6wro.mysql.rds.aliyuncs.com",
          user="ylc",
          password="1O8t5lcJ5aMhwwPEUUjS",
          database = '' ,
          port=3306
         ):   
    conn = pymysql.connect(
        host=host,
        user=user,
        port=port,
        password=password,
        max_allowed_packet=1073741824,
        charset="utf8")
    try:
        df = pd.read_sql(sql, con=conn)
        conn.close()
    except:
        print('error')
        conn.close()
        raise
    return df

## 导入租后数据

In [3]:
# f_path_ck = "E:/myfile/租后数据/租后应收监控excel2024-07-24.xlsx"

# df_zhys = pd.read_excel(f_path_ck,sheet_name='租后应收监控')

# df_zhys.shape

In [4]:
# df_ddfq = pd.read_excel(f_path_ck,sheet_name='订单分期表')
# df_ddfq.shape

In [5]:
# df_zh = df_ddfq.merge(df_zhys,on = '订单号',how = 'left' )
# df_zh.shape

# 获取租后数据

In [6]:
sql = ''' -- 租后应收监控  
SELECT  tprm.* 
,om.`status` as 订单状态值
,case om.`status` when  1 then "待支付" when  2 then "待发货" when  3 then "待收货" when  4 then "租赁中" when  5 then "待归还" 
when  6 then "待商家收货" when  7 then "退押中" when  8 then "已完成" when  10 then "已退款" when  11 then "待退押金" when  12 then "待审核" 
when  13 then "订单取消" when  15 then "检测中" when  9999 then "逾期订单" end as  订单状态
,om.order_finish_date as 订单完成时间,om.order_method
,tmu.true_name ,tmu.id_card_num,tmu.mobile 
from db_digua_business.t_postlease_receivables_monitoring   tprm 
left join db_digua_business.t_order om on tprm.order_id = om.id
left join  db_digua_business.t_member_user tmu on om.user_id = tmu.id
;
'''

df_zhys = query(sql)
df_zhys.shape

(25353, 62)

In [7]:
df_zhys.columns

Index(['id', 'order_id', 'order_number', 'order_time', 'search_time',
       'end_date', 'merchant_id', 'merchant_name', 'channel_id',
       'channel_name', 'model_number', 'specification', 'purchase_amount',
       'platform_package', 'all_deposit', 'all_rental', 'liquidated_damages',
       'buyout_balance_payment', 'advance_periods', 'advance_sum',
       'order_create_time', 'rembursement_status', 'paid_periods',
       'begin_overdue_time', 'overdue_periods', 'max_overdue_days',
       'now_overdue_days', 'total_receivable', 'contract_price',
       'rent_received', 'unearned_rent_revenue', 'paid_buyout_balance_payment',
       'total_received', 'total_unreceived', 'unrecovered_cost',
       'overdue_type', 'gross_profit_receivable', 'overdue_loss',
       'actual_gross_profit', 'overdue_total_unreceived', 'relet_periods',
       'relet_days', 'relet_rent_paid', 'activity_id', 'activity_source',
       'order_label', 'create_time', 'update_time', 'order_finish_date',
       'comm

# 剔除商家数据

In [8]:
# df_zhys["年"] = df_zhys["下单月份"].str[:4]
# pd.crosstab(dfzh1["商家名称"],dfzh1["年"],margins=True).loc[["北京海鸟窝科技有限公司","深圳优优大数据科技有限公司","租着用电脑数码","苏州蚁诺宝"],:]
# 剔除商家数据
def drop_sj(df_zhys,col1,col2):
    df_zhys.drop(df_zhys[df_zhys[col1]=="深圳优优大数据科技有限公司"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="优优2店"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="小豚租（代收）"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="苏州蚁诺宝"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="租着用电脑数码"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="北京海鸟窝科技有限公司"].index,inplace=True)

    df_zhys.drop(df_zhys[df_zhys[col1]=="汇客好租"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="澄心优租"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="CPS渠道合作"].index,inplace=True)

    df_zhys.drop(df_zhys[df_zhys[col1]=="趣智数码"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="格木木二奢名品"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="广州康基贸易有限公司"].index,inplace=True)

    df_zhys.drop(df_zhys[df_zhys[col2].str.contains(pat='探路者',regex=False)==True].index,inplace=True)
    return df_zhys

df_zhys = drop_sj(df_zhys,'merchant_name','model_number')
df_zhys.shape

(23741, 62)

# 历史订单渠道名称补充

In [9]:

qudao_name_df = pd.read_excel('F:/myfile/租后数据/历史订单渠道名称补充.xlsx')
qudao_name_df.drop(columns=['渠道id','下单时间'],inplace = True)

In [10]:
# dfzh1 = df_zh.merge(qudao_name_df,on = '订单号',how = 'left')
dfzh1 =  df_zhys.merge(qudao_name_df,left_on = 'order_number',right_on = '订单号',how = 'left')
# dfzh1.columns

In [11]:
# 渠道名称缺失补充
# dfzh1["渠道名称"] = dfzh1["渠道名称_x"].fillna(dfzh1.渠道名称_y)
dfzh1["channel_name"] = np.where(dfzh1.渠道名称.notnull(),dfzh1.渠道名称,dfzh1["channel_name"])
dfzh1["channel_name"].isnull().sum()

np.int64(0)

# 渠道归属

In [12]:
def qudao_type(a,b,c):
    a = str(a)
    b = str(b)
    if "租物" in b:
        return "芝麻租物"
    elif "芝麻" in b:
        return "芝麻租物"
    elif "抖音" in b:
        return "抖音渠道"
    elif "搜索" in a:
        return "搜索渠道"
    elif "租物" in a:
        return "芝麻租物"
    elif "芝麻" in a:
        return "芝麻租物"
    elif c == 1:
        return "芝麻租物" 
    elif "叮咚直播" in a:
        return "叮咚直播"
    elif "八派信息" in a:
        return "八派信息"
    elif "租瓜直播2号" in a:
        return "租瓜直播2号"
    elif "租瓜直播" in a:
        return "租瓜直播"
    elif "直播" in a:
        return "小舞直播"
    elif "繁星" in a:
        return "繁星"
    elif "生活号" in a:
        return "生活号"
    elif "社群" in a:
        return "支付宝社群"
    elif "付费灯火" in a:
        return "付费灯火"
    else :
        return a


In [13]:
# df.loc[:,"归属渠道"]=df.apply(lambda x:qudao_type(x["来源渠道"],x["activity_name"],x["order_method"]),axis=1)

In [14]:
dfzh1["activity_source"] = dfzh1["activity_source"].fillna('未知活动') 
dfzh1["channel_name"] = dfzh1["channel_name"].fillna('未知渠道') 

dfzh1.loc[:,"归属渠道"]=dfzh1.apply(lambda x:qudao_type(x["channel_name"],x["activity_source"],x["order_method"]),axis=1)

# pd.crosstab(dfzh1["下单月份"],dfzh1["归属渠道"],margins=True)

# 判断颜色 内存

In [15]:
dfzh1["内存"]=dfzh1["specification"].str.split("内存：").str[1].str.split("颜色：").str[0].str.split(" ").str[0]
# dfzh1["内存"].value_counts()[14:]

dfzh1["颜色"]=dfzh1["specification"].str.split("内存：").str[1].str.split("颜色：").str[1].str.split(" ").str[0]
dfzh1["颜色"].value_counts()[1:10]

颜色
原色钛金属         4229
黑色钛金属         2638
暗紫色           1833
深空黑色          1370
蓝色钛金属         1270
银色             877
蓝色钛金属-现货速发     818
黑色             800
蓝色             561
Name: count, dtype: int64

# 判断二手

In [16]:
dfzh1['是否二手'] = np.where(dfzh1['model_number'].str.contains('95新')|dfzh1['model_number'].str.contains('99新')|dfzh1['model_number'].str.contains('准新'),"二手","全新")
dfzh1['是否二手'].value_counts()

是否二手
全新    23662
二手       79
Name: count, dtype: int64

# 日期处理

In [17]:

dfzh1["下单日期"] = dfzh1['order_create_time'].dt.date
dfzh1["下单日期"] = pd.to_datetime(dfzh1["下单日期"],errors="coerce")
dfzh1["下单日期"].head()

dfzh1["月份"]=dfzh1["下单日期"].dt.month
dfzh1["年份"]=dfzh1["下单日期"].dt.year
# df_total["下单月份"].value_counts()

In [18]:
def getjidu(s):
    if s<4:
        return "1季度"
    elif 4<=s<7:
        return "2季度"
    elif 7<=s<10:
        return "3季度"
    else:
        return "4季度"
dfzh1["季度"]= dfzh1.apply(lambda x:getjidu(x["月份"]),axis=1)  
dfzh1["季度"].value_counts()

季度
3季度    9671
2季度    6164
4季度    5121
1季度    2785
Name: count, dtype: int64

# 买断数据获取

In [19]:
sql1 = ''' --   买断信息 
with out_order as ( 
SELECT too.order_id,too.`status` 
,too.real_pay_money ,too.create_time,too.update_time
,too.pay_date ,too.actual_money 
,rank() over(partition by too.order_id order by too.pay_date desc) as rn 
from db_digua_business.t_order_out too
where  too.`status` not in (1)   -- 买断状态：1 未买断 2 已买断  3 部分买断 
-- and too.pay_status= 2    -- '支付状态 1、待支付 2、已支付 3、退款中 4、已退款 6、支付取消 7、已关闭',
-- GROUP BY 1 ,2
-- ORDER BY 5 desc,1
)
SELECT t1.*
,t2.`status`
,case when t2.`status`=2 then '已买断' when  t2.`status`=3 and om.order_finish_date is not null then '已买断'  else '部分买断' end as if_outpay 
,om.order_finish_date
,case when t2.`status`=2 and om.order_finish_date is null then t1.pay_date else om.order_finish_date end as finish_date_new 
from (
SELECT too.order_id  -- ,too.`status` 
-- ,count(DISTINCT too.`status`) nn 
,sum(too.real_pay_money) as outpay_money     -- ,max(too.update_time) as outpay_time
,max(too.pay_date) as pay_date 
from out_order  too
GROUP BY 1 -- ,2
ORDER BY 2 desc ,1) t1 
left join out_order t2 on t1.order_id = t2.order_id and t2.rn=1 
left join db_digua_business.t_order om on t1.order_id = om.id 
;
'''

df_out = query(sql1)
df_out.shape

(2152, 7)

In [20]:
df_out.columns

Index(['order_id', 'outpay_money', 'pay_date', 'status', 'if_outpay',
       'order_finish_date', 'finish_date_new'],
      dtype='object')

# 租后数据 关联 买断数据

In [21]:
dfzh2 = dfzh1.merge(df_out[['order_id', 'outpay_money', 'pay_date', 'status', 'if_outpay']], on = 'order_id', how = 'left')
dfzh2.columns

Index(['id', 'order_id', 'order_number', 'order_time', 'search_time',
       'end_date', 'merchant_id', 'merchant_name', 'channel_id',
       'channel_name', 'model_number', 'specification', 'purchase_amount',
       'platform_package', 'all_deposit', 'all_rental', 'liquidated_damages',
       'buyout_balance_payment', 'advance_periods', 'advance_sum',
       'order_create_time', 'rembursement_status', 'paid_periods',
       'begin_overdue_time', 'overdue_periods', 'max_overdue_days',
       'now_overdue_days', 'total_receivable', 'contract_price',
       'rent_received', 'unearned_rent_revenue', 'paid_buyout_balance_payment',
       'total_received', 'total_unreceived', 'unrecovered_cost',
       'overdue_type', 'gross_profit_receivable', 'overdue_loss',
       'actual_gross_profit', 'overdue_total_unreceived', 'relet_periods',
       'relet_days', 'relet_rent_paid', 'activity_id', 'activity_source',
       'order_label', 'create_time', 'update_time', 'order_finish_date',
       'comm

In [22]:
# dfzh2[['order_finish_date','订单完成时间','pay_date','status']].info()

In [23]:
dfzh2['finish_date_new'] = dfzh2.apply(lambda x:pd.to_datetime(x.pay_date, errors='coerce') if pd.isnull(x.order_finish_date) and x.status==2
                                       else pd.to_datetime(x.订单完成时间, errors='coerce') if pd.isnull(x.order_finish_date)  and pd.notnull(x.订单完成时间)
                                       else pd.to_datetime(x.order_finish_date, errors='coerce'),axis = 1 ) 
# dfzh2['finish_date_new'] = np.where((pd.isnull(dfzh2.order_finish_date)) & (dfzh2.status==2),pd.to_datetime(dfzh2.pay_date, errors='coerce'),pd.to_datetime(dfzh2.order_finish_date, errors='coerce'))
dfzh2['finish_date_new'].notnull().sum()

np.int64(2952)

In [24]:

dfzh2['if_outpay'] = dfzh2.apply(lambda x: '已买断' if x['status'] ==2 
                                 else  '已买断'  if  x['status'] ==3 and pd.notnull(x.finish_date_new) 
                                 else '部分买断' if x['status'] ==3 and pd.isnull(x.finish_date_new) 
                                 else '未知',axis =1)
dfzh2['if_outpay'].value_counts()


if_outpay
未知      21670
已买断      2031
部分买断       47
Name: count, dtype: int64

# 分期 取消数据 获取

In [25]:
sql1 = '''-- 订单分期表  取消的分期
SELECT tos.order_id,count(*) as nn
,count(tos.reality_refund_date) as  reality_refund_date
,sum(tos.real_pay_money) as real_pay_money
,sum(tprm.purchase_amount)/count(tos.order_id) as purchase_amount
,avg(tprm.advance_periods) as  advance_periods
,avg(tprm.advance_sum) as advance_sum
from db_digua_business.t_order_stages tos 
inner join db_digua_business.t_postlease_receivables_monitoring   tprm 
on tos.order_id = tprm.order_id 
where tos.`status`=4 
GROUP BY 1 order by 2 desc 
;
'''

df_cancel = query(sql1)
df_cancel.shape

(1339, 7)

In [26]:
# df_cancel.columns

# 租后数据 关联 取消数据

In [27]:
dfzh3 = dfzh2.merge(df_cancel[['order_id', 'nn', 'reality_refund_date', 'real_pay_money']],on = 'order_id',how = 'left')
dfzh3.columns

Index(['id', 'order_id', 'order_number', 'order_time', 'search_time',
       'end_date', 'merchant_id', 'merchant_name', 'channel_id',
       'channel_name', 'model_number', 'specification', 'purchase_amount',
       'platform_package', 'all_deposit', 'all_rental', 'liquidated_damages',
       'buyout_balance_payment', 'advance_periods', 'advance_sum',
       'order_create_time', 'rembursement_status', 'paid_periods',
       'begin_overdue_time', 'overdue_periods', 'max_overdue_days',
       'now_overdue_days', 'total_receivable', 'contract_price',
       'rent_received', 'unearned_rent_revenue', 'paid_buyout_balance_payment',
       'total_received', 'total_unreceived', 'unrecovered_cost',
       'overdue_type', 'gross_profit_receivable', 'overdue_loss',
       'actual_gross_profit', 'overdue_total_unreceived', 'relet_periods',
       'relet_days', 'relet_rent_paid', 'activity_id', 'activity_source',
       'order_label', 'create_time', 'update_time', 'order_finish_date',
       'comm

金额逾期率去除续租订单

In [28]:
# dfzh3 = dfzh3[dfzh3['relet_periods']< 1] 

In [29]:
dfzh3.shape

(23748, 80)

In [30]:
# 不同还款状态下  完成时间的情况
# dfzh3.groupby(['rembursement_status']).agg({'order_id':'count','order_finish_date':'count','订单完成时间':'count','finish_date_new':'count','pay_date':'count','nn':'count','outpay_money':'sum'})

In [31]:
# 不同订单状态下  完成时间的情况
# dfzh3.groupby(['订单状态']).agg({'order_id':'count','order_finish_date':'count','订单完成时间':'count','finish_date_new':'count','pay_date':'count','nn':'count','outpay_money':'sum'})

In [32]:
# 有取消记录的用户分布情况
# dfzh3[dfzh3.nn>=1].groupby(['rembursement_status']).agg({'order_id':'count','order_finish_date':'count','订单完成时间':'count'
                                                        #  ,'finish_date_new':'count','pay_date':'count','nn':'count','real_pay_money':'sum'})


In [33]:
# 有取消记录的用户分布情况
# dfzh3[dfzh3.nn>=1].groupby(['订单状态']).agg({'order_id':'count','order_finish_date':'count','订单完成时间':'count'
                                                        #  ,'finish_date_new':'count','pay_date':'count','nn':'count','real_pay_money':'sum'})


# 按风控策略调整时间节点划分时间段

In [34]:
#按风控策略调整时间节点划分时间段
dfzh1.loc[:"2023-01-09","时间段"]=" -2023.1.9前"
dfzh1.loc["2023-01-10":"2023-03-03","时间段"]="2023.1.10-2023.3.3"
dfzh1.loc["2023-03-04":"2023-03-24","时间段"]="2023.3.4-2023.3.24"
dfzh1.loc["2023-03-25":"2023-04-28","时间段"]="2023.3.25-2023.4.28"
dfzh1.loc["2023-04-29":"2023-05-17","时间段"]="2023.4.29-2023.5.17"
dfzh1.loc["2023-05-18":"2023-05-31","时间段"]="2023.5.18-2023.5.31"
dfzh1.loc["2023-06-01":"2023-06-30","时间段"]="2023.6.1-2023.6.30"
dfzh1.loc["2023-07-01":"2023-08-11","时间段"]="2023.7.1-2023.8.11"
dfzh1.loc["2023-08-12":"2023-09-15","时间段"]="2023.8.12-2023.9.15"
dfzh1.loc["2023-09-16":"2023-09-22","时间段"]="2023.9.16-2023.9.22"
dfzh1.loc["2023-09-23":"2023-11-16","时间段"]="2023.9.23-2023.11.16"
dfzh1.loc["2023-11-17":"2023-11-28","时间段"]="2023.11.17-2023.11.28"
dfzh1.loc["2023-11-29":"2023-12-05","时间段"]="2023.11.29-2023.12.5"
dfzh1.loc["2023-12-06":"2024-01-05","时间段"]="2023.12.06-2024.1.5"
dfzh1.loc["2024-01-06":"2024-01-23","时间段"]="2024.1.6-2024.1.23"
dfzh1.loc["2024-01-24":"2024-2-21","时间段"]="2024.1.24-2024.2.21"
dfzh1.loc["2024-02-22":"2024-02-29","时间段"]="2024.2.22-2024.2.29"
dfzh1.loc["2024-03-01":"2024-03-12","时间段"]="2024.3.1-2024.3.12"
dfzh1.loc["2024-03-13":"2024-04-03","时间段"]="2024.3.13-2024.4.3"
dfzh1.loc["2024-04-04":"2024-04-10","时间段"]="2024.4.4-2024.4.10"
dfzh1.loc["2024-04-11":"2024-04-18","时间段"]="2024.4.11-2024.4.18"
dfzh1.loc["2024-04-19":"2024-04-24","时间段"]="2024.4.19-2024.4.24"
dfzh1.loc["2024-04-25":"2024-04-27","时间段"]="2024.4.25-2024.4.27"
dfzh1.loc["2024-04-28":,"时间段"]="2024.4.28- "

# 账期数据提取   含续租

## 获取续租分期数据

In [35]:
sql_xz = ''' 
-- 状态,1:未支付  2:扣款失败 3:已支付 4：已取消 5、申请扣款、6已退款',
-- '是否续租账期,0非续租账期，1预授权账期，2自定义续租账期',
SELECT ymos.*
from db_rent.ya_merchant_order_stages ymos 
inner join db_digua_business.t_postlease_receivables_monitoring tprm 
on ymos.order_id = tprm.order_id 
;
'''

df_xzfq = query(sql_xz)
df_xzfq.shape             #  ya_merchant_order_stages

(310741, 18)

## 重命名

In [36]:
df_xzfq = df_xzfq.rename(columns={'money':'当前应付租金','part_payment':'实付金额','refund_date':'应付日期'
,'reality_refund_date':'实付日期','sort':'当前期数'})

In [37]:
# df_ddfq = df_ddfq.rename(columns={'排序还款期数':'当前期数'})
# df_ddfq.columns

In [38]:
# df_xzfq.columns

## 实还金额核查

In [39]:
# total_check = df_ddfq.merge(df_xzfq,on=['订单ID','当前期数'],how='inner')
# total_check.info()

In [40]:
# total_check1= total_check[(total_check.实付金额_y < total_check.实付金额_x)][['订单ID','订单号','应付日期_x','实付日期_x','实付日期_y','实付金额_x','实付金额_y','状态','status']]    #   #(total_check.应付日期_x.notnull()) & (total_check.应付日期_y.isnull())
# total_check1= total_check[(total_check.实付金额 < total_check.实际支付金额)][['订单ID','订单号','还款日期','实际还款日期','实付日期','实际支付金额','实付金额','状态','status']]
# total_check1

In [41]:
# total_check2= total_check[(total_check.实付金额_y > total_check.实付金额_x)][['订单ID','订单号','应付日期_x','实付日期_x','实付日期_y','实付金额_x','实付金额_y','状态','status']]    #   #(total_check.应付日期_x.notnull()) & (total_check.应付日期_y.isnull())

# total_check2= total_check[(total_check.实付金额 > total_check.实际支付金额)][['订单ID','订单号','还款日期','实际还款日期','实付日期','实际支付金额','实付金额','状态','status']]
# total_check2

In [42]:
# total_check2.to_excel('E:/myfile/业务相关数据/total_check_0709-1.xlsx')

# 数据合并

In [43]:
df_xz_concat = df_xzfq.merge(dfzh3,how='right',on ='order_id')
# df_xz_concat = df_fq.merge(dfzh1,how='right',right_on ='订单id',left_on='订单ID')

df_xz_concat['当前期数'].notnull().sum()

np.int64(310638)

In [44]:
df_xz_concat['order_id'].nunique()

23741

In [45]:
dfzh3['order_id'].nunique()

23741

In [46]:
df_xz_concat[["order_id","当前期数"]].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 310638 entries, 0 to 310637
Data columns (total 2 columns):
 #   Column    Non-Null Count   Dtype
---  ------    --------------   -----
 0   order_id  310638 non-null  int64
 1   当前期数      310638 non-null  int64
dtypes: int64(2)
memory usage: 4.7 MB


# 去重

In [47]:
check = df_xz_concat[["order_id","当前期数"]]  # ,"逾期天数"
all_duplicates = check[check.duplicated(keep=False)]  
# all_duplicates.to_excel('E:/myfile/业务相关数据/all_duplicates.xlsx')
print(all_duplicates.sort_values(by=['order_id','当前期数']))

       order_id  当前期数
8063     119321     1
8075     119321     1
8064     119321     2
8076     119321     2
8065     119321     3
...         ...   ...
65312    552441    10
65301    552441    11
65313    552441    11
65302    552441    12
65314    552441    12

[168 rows x 2 columns]


In [48]:
#删除重复订单
df_xz_concat.drop_duplicates(subset=["order_id","当前期数"],inplace=True)

In [49]:
# df_xz_concat.to_excel('E:/myfile/业务相关数据/df_xz_concat.xlsx')

## 异常查询

In [50]:
df_xz_concat.columns

Index(['id_x', 'order_id', '当前应付租金', '实付金额', 'interest_money', '应付日期', '实付日期',
       'status_x', '当前期数', 'order_pay_id', 'create_time_x', 'update_time_x',
       'type', 'advance_time', 'merchant_account_id', 'order_relet_id',
       'is_relet', 'pay_type', 'id_y', 'order_number', 'order_time',
       'search_time', 'end_date', 'merchant_id', 'merchant_name', 'channel_id',
       'channel_name', 'model_number', 'specification', 'purchase_amount',
       'platform_package', 'all_deposit', 'all_rental', 'liquidated_damages',
       'buyout_balance_payment', 'advance_periods', 'advance_sum',
       'order_create_time', 'rembursement_status', 'paid_periods',
       'begin_overdue_time', 'overdue_periods', 'max_overdue_days',
       'now_overdue_days', 'total_receivable', 'contract_price',
       'rent_received', 'unearned_rent_revenue', 'paid_buyout_balance_payment',
       'total_received', 'total_unreceived', 'unrecovered_cost',
       'overdue_type', 'gross_profit_receivable', 'overdue

In [51]:
# df_xz_concat[(df_xz_concat.实付金额>0) & (df_xz_concat.status_x==1)][['order_number', '当前应付租金', '实付金额','应付日期', '实付日期'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new']]

实付金额>0  实付日期为空  finish_date_new非空  :实付日期=订单完成时间

In [52]:
df_xz_concat[(df_xz_concat.实付金额>0) & (df_xz_concat.实付日期.isnull()) & (df_xz_concat.finish_date_new.isnull())][['order_number', '当前应付租金', '实付金额','应付日期', '实付日期'
, 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new']]

,order_number,当前应付租金,实付金额,应付日期,实付日期,status_x,当前期数,rembursement_status,订单完成时间,订单状态值,finish_date_new
2776,A202211111913512,650.70,79.00,2023-09-11,NaT,1,11,已逾期,NaT,5,NaT
10584,A2023022108454753,641.86,246.42,2023-09-24,NaT,1,8,还款中,NaT,5,NaT
25146,A2023050408232337,670.85,412.63,2024-12-06,NaT,1,20,续租中,NaT,4,NaT
40685,A20230627115744111,479.73,201.35,2024-03-30,NaT,1,10,还款中,NaT,5,NaT
40981,A2023062811593394,670.85,412.63,2024-11-30,NaT,1,18,续租中,NaT,4,NaT
61030,A2023081019051714,1074.81,700.00,2024-07-13,NaT,1,12,已买断,NaT,8,NaT
61289,A2023081108595369,653.67,473.82,2025-02-13,NaT,1,19,续租中,NaT,4,NaT
107200,A20231105094819133,1085.68,17.18,2024-10-08,NaT,1,12,还款中,NaT,4,NaT
111794,A20231112005735116,732.98,368.38,2025-01-14,NaT,1,15,续租中,NaT,4,NaT
117100,A2023111818174833,656.29,58.96,2024-05-21,NaT,1,7,还款中,NaT,4,NaT


part_payment==0  reality_refund_date.notnull()  还款状态== '续租中' part_payment=money   还款状态== '已逾期' reality_refund_date=pd.na

In [53]:
dd_test = df_xz_concat[(df_xz_concat.实付金额==0) & (df_xz_concat.实付日期.notnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
, 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn']]

dd_test

,order_number,order_id,当前应付租金,实付金额,应付日期,实付日期,status_x,当前期数,rembursement_status,订单完成时间,订单状态值,finish_date_new,nn
102,A202207181530165,9777,306.60,0.0,2023-01-18,2023-03-18 19:54:59,2,7,已逾期,NaT,5,NaT,NaN
103,A202207181530165,9777,306.60,0.0,2023-02-18,2023-03-18 19:55:00,2,8,已逾期,NaT,5,NaT,NaN
104,A202207181530165,9777,306.60,0.0,2023-03-18,2023-03-18 19:55:01,2,9,已逾期,NaT,5,NaT,NaN
137,A202207200249265,10056,736.56,0.0,2022-12-20,2023-03-18 19:55:02,2,6,已逾期,NaT,4,NaT,NaN
138,A202207200249265,10056,736.56,0.0,2023-01-20,2023-03-18 19:55:03,2,7,已逾期,NaT,4,NaT,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9648,A2023021504342232,133713,396.66,0.0,2023-03-18,2023-03-18 20:05:58,2,2,已逾期,NaT,5,NaT,NaN
10662,A2023022114312437,145022,556.04,0.0,2023-03-24,2023-03-20 09:02:06,2,2,已逾期,NaT,5,NaT,NaN
12279,A2023030700325746,166163,288.77,0.0,2024-01-10,2023-04-17 12:08:57,1,11,已逾期,NaT,5,NaT,NaN
14488,A2023032216313863,187926,1.00,0.0,2023-03-25,2023-03-22 17:23:18,4,1,已完成,NaT,8,NaT,12.0


In [54]:
dd_test = df_xz_concat[(df_xz_concat.实付金额==0) & (df_xz_concat.实付日期.notnull())& (df_xz_concat.finish_date_new.isnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
, 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn']]
dd_test

,order_number,order_id,当前应付租金,实付金额,应付日期,实付日期,status_x,当前期数,rembursement_status,订单完成时间,订单状态值,finish_date_new,nn
102,A202207181530165,9777,306.60,0.0,2023-01-18,2023-03-18 19:54:59,2,7,已逾期,NaT,5,NaT,NaN
103,A202207181530165,9777,306.60,0.0,2023-02-18,2023-03-18 19:55:00,2,8,已逾期,NaT,5,NaT,NaN
104,A202207181530165,9777,306.60,0.0,2023-03-18,2023-03-18 19:55:01,2,9,已逾期,NaT,5,NaT,NaN
137,A202207200249265,10056,736.56,0.0,2022-12-20,2023-03-18 19:55:02,2,6,已逾期,NaT,4,NaT,NaN
138,A202207200249265,10056,736.56,0.0,2023-01-20,2023-03-18 19:55:03,2,7,已逾期,NaT,4,NaT,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9648,A2023021504342232,133713,396.66,0.0,2023-03-18,2023-03-18 20:05:58,2,2,已逾期,NaT,5,NaT,NaN
10662,A2023022114312437,145022,556.04,0.0,2023-03-24,2023-03-20 09:02:06,2,2,已逾期,NaT,5,NaT,NaN
12279,A2023030700325746,166163,288.77,0.0,2024-01-10,2023-04-17 12:08:57,1,11,已逾期,NaT,5,NaT,NaN
14488,A2023032216313863,187926,1.00,0.0,2023-03-25,2023-03-22 17:23:18,4,1,已完成,NaT,8,NaT,12.0


实付为0，实付日期非空，完成时间为空，无期数取消：实付日期清空

In [55]:
dd_test = df_xz_concat[(df_xz_concat.实付金额==0) & (df_xz_concat.实付日期.notnull())& (df_xz_concat.finish_date_new.isnull())& (df_xz_concat.nn.isnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
, 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn']]
dd_test.订单状态值.value_counts()

订单状态值
5    174
4     60
Name: count, dtype: int64

实付日期为空，完成时间为空，取消期数>=1 ：状态=已取消

In [56]:
dd_test = df_xz_concat[ (df_xz_concat.实付日期.isnull())& (df_xz_concat.finish_date_new.isnull())& (df_xz_concat.nn>=1)][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
, 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn']]
dd_test.订单状态值.value_counts()

订单状态值
10    2604
8      204
4       10
5        6
Name: count, dtype: int64

部分还款，实付日期非空，完成时间为空 用户：  实付日期清空  

In [57]:
dd_test1 = df_xz_concat[(df_xz_concat.实付金额>0)&(df_xz_concat.实付金额<df_xz_concat.当前应付租金) & (df_xz_concat.实付日期.notnull()) & (df_xz_concat.finish_date_new.isnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
, 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn']]

dd_test1

,order_number,order_id,当前应付租金,实付金额,应付日期,实付日期,status_x,当前期数,rembursement_status,订单完成时间,订单状态值,finish_date_new,nn
1980,A202210092154098,27461,591.60,408.40,2023-01-09,2023-03-18 20:04:40,2,4,已逾期,NaT,5,NaT,NaN
49251,A2023071909113511,477555,583.26,200.00,2024-02-22,2024-02-28 20:07:18,1,8,已逾期,NaT,5,NaT,NaN
121949,A2023112606505333,814549,541.32,22.76,2024-08-29,2024-08-12 17:47:02,1,10,已逾期,NaT,4,NaT,NaN


 部分还款，实付日期为空，完成时间非空 用户：  实付日期=完成时间

In [58]:
dd_test1 = df_xz_concat[(df_xz_concat.实付金额>0)&(df_xz_concat.实付金额<df_xz_concat.当前应付租金) & (df_xz_concat.实付日期.isnull()) & (df_xz_concat.finish_date_new.notnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
, 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn']]

dd_test1.订单状态值.value_counts()

订单状态值
8    140
Name: count, dtype: int64

已买断 但无实付日期及金额：实付日回写

In [59]:
dd_test1 = df_xz_concat[(df_xz_concat.实付金额==0) & (df_xz_concat.实付日期.isnull()) & (df_xz_concat.finish_date_new.notnull())& (df_xz_concat.outpay_money>0)][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
, 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','outpay_money']]

# dd_test1.订单状态值.value_counts()
dd_test1

,order_number,order_id,当前应付租金,实付金额,应付日期,实付日期,status_x,当前期数,rembursement_status,订单完成时间,订单状态值,finish_date_new,nn,outpay_money
22,A202207050851373,8924,354.00,0.0,2024-05-01,NaT,1,23,已买断,2024-04-17 01:37:33,8,2024-04-17 01:37:33,NaN,329.72
23,A202207050851373,8924,47.20,0.0,2024-05-31,NaT,1,24,已买断,2024-04-17 01:37:33,8,2024-04-17 01:37:33,NaN,329.72
419,A202208032116483,12219,483.00,0.0,2023-12-31,NaT,1,18,已买断,2023-11-06 10:12:57,8,2023-11-06 10:12:57,NaN,2709.20
420,A202208032116483,12219,483.00,0.0,2024-01-30,NaT,1,19,已买断,2023-11-06 10:12:57,8,2023-11-06 10:12:57,NaN,2709.20
421,A202208032116483,12219,483.00,0.0,2024-02-29,NaT,1,20,已买断,2023-11-06 10:12:57,8,2023-11-06 10:12:57,NaN,2709.20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167329,A2024041316125534,995895,1061.16,0.0,2024-11-16,NaT,1,8,已买断,2024-05-17 15:02:17,8,2024-05-17 15:02:17,10.0,10014.38
167330,A2024041316125534,995895,1061.16,0.0,2024-12-16,NaT,1,9,已买断,2024-05-17 15:02:17,8,2024-05-17 15:02:17,10.0,10014.38
167331,A2024041316125534,995895,1061.16,0.0,2025-01-16,NaT,1,10,已买断,2024-05-17 15:02:17,8,2024-05-17 15:02:17,10.0,10014.38
167332,A2024041316125534,995895,1061.16,0.0,2025-02-16,NaT,1,11,已买断,2024-05-17 15:02:17,8,2024-05-17 15:02:17,10.0,10014.38


##  实还日期 修正

In [60]:
df_xz_concat['实付日期'] = pd.to_datetime(df_xz_concat['实付日期']).dt.date
df_xz_concat['finish_date_new'] = pd.to_datetime(df_xz_concat['finish_date_new']).dt.date

In [61]:
# 部分还款 用户 实付日期非空 且非买断 非完成 ： 实付日期清空  
df_xz_concat['实付日期1'] = np.where((df_xz_concat.实付金额>0)&(df_xz_concat.实付金额<df_xz_concat.当前应付租金) & (df_xz_concat.实付日期.notnull()) & (df_xz_concat.finish_date_new.isnull()),pd.NaT,df_xz_concat.实付日期)
df_xz_concat[(df_xz_concat.实付金额>0)&(df_xz_concat.实付金额<df_xz_concat.当前应付租金) & (df_xz_concat.实付日期.notnull()) & (df_xz_concat.finish_date_new.isnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
, 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','实付日期1']]                            

,order_number,order_id,当前应付租金,实付金额,应付日期,实付日期,status_x,当前期数,rembursement_status,订单完成时间,订单状态值,finish_date_new,nn,实付日期1
1980,A202210092154098,27461,591.60,408.40,2023-01-09,2023-03-18,2,4,已逾期,NaT,5,NaT,NaN,NaT
49251,A2023071909113511,477555,583.26,200.00,2024-02-22,2024-02-28,1,8,已逾期,NaT,5,NaT,NaN,NaT
121949,A2023112606505333,814549,541.32,22.76,2024-08-29,2024-08-12,1,10,已逾期,NaT,4,NaT,NaN,NaT


In [62]:
# 已完成 还款 用户  实付日期 补充 订单完成时间       实付金额>0  实付日期为空  finish_date_new非空  :实付日期=订单完成时间finish_date_new
df_xz_concat['实付日期1'] = np.where((df_xz_concat.实付金额>0) & (df_xz_concat.实付日期1.isnull()) & (df_xz_concat.finish_date_new.notnull()),df_xz_concat.finish_date_new,df_xz_concat.实付日期1)
df_xz_concat[(df_xz_concat.实付金额>0) & (df_xz_concat.实付日期1.isnull()) & (df_xz_concat.finish_date_new.isnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
, 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','实付日期1']]    

,order_number,order_id,当前应付租金,实付金额,应付日期,实付日期,status_x,当前期数,rembursement_status,订单完成时间,订单状态值,finish_date_new,nn,实付日期1
1980,A202210092154098,27461,591.60,408.40,2023-01-09,2023-03-18,2,4,已逾期,NaT,5,NaT,NaN,NaT
2776,A202211111913512,35537,650.70,79.00,2023-09-11,NaT,1,11,已逾期,NaT,5,NaT,NaN,NaT
10584,A2023022108454753,144517,641.86,246.42,2023-09-24,NaT,1,8,还款中,NaT,5,NaT,9.0,NaT
25146,A2023050408232337,291542,670.85,412.63,2024-12-06,NaT,1,20,续租中,NaT,4,NaT,NaN,NaT
40685,A20230627115744111,422689,479.73,201.35,2024-03-30,NaT,1,10,还款中,NaT,5,NaT,7.0,NaT
40981,A2023062811593394,424866,670.85,412.63,2024-11-30,NaT,1,18,续租中,NaT,4,NaT,NaN,NaT
49251,A2023071909113511,477555,583.26,200.00,2024-02-22,2024-02-28,1,8,已逾期,NaT,5,NaT,NaN,NaT
61030,A2023081019051714,533644,1074.81,700.00,2024-07-13,NaT,1,12,已买断,NaT,8,NaT,NaN,NaT
61289,A2023081108595369,535010,653.67,473.82,2025-02-13,NaT,1,19,续租中,NaT,4,NaT,NaN,NaT
107200,A20231105094819133,751673,1085.68,17.18,2024-10-08,NaT,1,12,还款中,NaT,4,NaT,2.0,NaT


In [63]:
# 租完即送用户 A2023081019051714 补还款时间 
df_xz_concat['实付日期1'] = np.where((df_xz_concat.order_number=='A2023081019051714') & (df_xz_concat.当前期数==12),'2024-07-22',df_xz_concat['实付日期1'])
df_xz_concat[(df_xz_concat.order_number=='A2023081019051714') & (df_xz_concat.当前期数==12)][['实付日期1']]

,实付日期1
61030,2024-07-22


In [64]:
# 未还款 且 未完成 未取消用户 ： 实付日期清空
df_xz_concat['实付日期1'] = np.where((df_xz_concat.实付金额==0) & (df_xz_concat.实付日期.notnull())& (df_xz_concat.finish_date_new.isnull())& (df_xz_concat.nn.isnull()),pd.NaT,df_xz_concat.实付日期1)
df_xz_concat[(df_xz_concat.实付金额==0) & (df_xz_concat.实付日期.notnull())& (df_xz_concat.finish_date_new.isnull())& (df_xz_concat.nn.isnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
, 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','实付日期1']] 


,order_number,order_id,当前应付租金,实付金额,应付日期,实付日期,status_x,当前期数,rembursement_status,订单完成时间,订单状态值,finish_date_new,nn,实付日期1
102,A202207181530165,9777,306.60,0.0,2023-01-18,2023-03-18,2,7,已逾期,NaT,5,NaT,NaN,NaT
103,A202207181530165,9777,306.60,0.0,2023-02-18,2023-03-18,2,8,已逾期,NaT,5,NaT,NaN,NaT
104,A202207181530165,9777,306.60,0.0,2023-03-18,2023-03-18,2,9,已逾期,NaT,5,NaT,NaN,NaT
137,A202207200249265,10056,736.56,0.0,2022-12-20,2023-03-18,2,6,已逾期,NaT,4,NaT,NaN,NaT
138,A202207200249265,10056,736.56,0.0,2023-01-20,2023-03-18,2,7,已逾期,NaT,4,NaT,NaN,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8874,A2023021015233353,125911,633.21,0.0,2023-03-13,2023-03-18,2,2,已逾期,NaT,5,NaT,NaN,NaT
9570,A2023021414250354,132517,334.26,0.0,2023-03-17,2023-03-18,2,2,已逾期,NaT,5,NaT,NaN,NaT
9648,A2023021504342232,133713,396.66,0.0,2023-03-18,2023-03-18,2,2,已逾期,NaT,5,NaT,NaN,NaT
10662,A2023022114312437,145022,556.04,0.0,2023-03-24,2023-03-20,2,2,已逾期,NaT,5,NaT,NaN,NaT


In [65]:
# 实付为0，实付日期为空，但 完成时间非空，买断金额>0： 实付日期回写

df_xz_concat['实付日期1'] = np.where((df_xz_concat.实付金额==0) & (df_xz_concat.实付日期1.isnull()) & (df_xz_concat.finish_date_new.notnull())& (df_xz_concat.outpay_money>0),df_xz_concat.finish_date_new,df_xz_concat.实付日期1)
df_xz_concat[(df_xz_concat.实付金额==0) & (df_xz_concat.实付日期1.notnull()) & (df_xz_concat.finish_date_new.notnull())& (df_xz_concat.outpay_money>0)][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
, 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','实付日期1']] 

,order_number,order_id,当前应付租金,实付金额,应付日期,实付日期,status_x,当前期数,rembursement_status,订单完成时间,订单状态值,finish_date_new,nn,实付日期1
22,A202207050851373,8924,354.00,0.0,2024-05-01,NaT,1,23,已买断,2024-04-17 01:37:33,8,2024-04-17,NaN,2024-04-17
23,A202207050851373,8924,47.20,0.0,2024-05-31,NaT,1,24,已买断,2024-04-17 01:37:33,8,2024-04-17,NaN,2024-04-17
419,A202208032116483,12219,483.00,0.0,2023-12-31,NaT,1,18,已买断,2023-11-06 10:12:57,8,2023-11-06,NaN,2023-11-06
420,A202208032116483,12219,483.00,0.0,2024-01-30,NaT,1,19,已买断,2023-11-06 10:12:57,8,2023-11-06,NaN,2023-11-06
421,A202208032116483,12219,483.00,0.0,2024-02-29,NaT,1,20,已买断,2023-11-06 10:12:57,8,2023-11-06,NaN,2023-11-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167329,A2024041316125534,995895,1061.16,0.0,2024-11-16,NaT,1,8,已买断,2024-05-17 15:02:17,8,2024-05-17,10.0,2024-05-17
167330,A2024041316125534,995895,1061.16,0.0,2024-12-16,NaT,1,9,已买断,2024-05-17 15:02:17,8,2024-05-17,10.0,2024-05-17
167331,A2024041316125534,995895,1061.16,0.0,2025-01-16,NaT,1,10,已买断,2024-05-17 15:02:17,8,2024-05-17,10.0,2024-05-17
167332,A2024041316125534,995895,1061.16,0.0,2025-02-16,NaT,1,11,已买断,2024-05-17 15:02:17,8,2024-05-17,10.0,2024-05-17


In [66]:
# 部分还款 
# df_xz_concat[(df_xz_concat.实付金额>0) & (df_xz_concat.实付日期1.isnull()) & (df_xz_concat.finish_date_new.isnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期1'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','outpay_money']]

In [67]:
# 未完成 但含有分期取消的 订单
# check = df_xz_concat[(df_xz_concat.实付日期1.isnull()) & (df_xz_concat.finish_date_new.isnull()) & df_xz_concat.nn>=1][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期1'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','outpay_money']]
# check.订单状态值.value_counts()

In [68]:
# 实付0，实付日期为空，但 完成时间非空：为取消订单，实付日期=完成时间
df_xz_concat['实付日期new'] = np.where( (df_xz_concat.实付日期1.isnull()) & (df_xz_concat.finish_date_new.notnull()),df_xz_concat.finish_date_new,df_xz_concat.实付日期1)
df_xz_concat[ (df_xz_concat.实付日期new.isnull()) & (df_xz_concat.finish_date_new.notnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期1'
, 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','outpay_money']]

,order_number,order_id,当前应付租金,实付金额,应付日期,实付日期1,status_x,当前期数,rembursement_status,订单完成时间,订单状态值,finish_date_new,nn,outpay_money


In [69]:
# 属于部分还款 :无实付日期
# dd_test2 = df_xz_concat[(df_xz_concat.实付金额 >0) & (df_xz_concat.实付日期new.isnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期new'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','outpay_money']]
# dd_test2

In [70]:
# df_xz_concat[df_xz_concat.应付日期.isnull()][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期new'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','outpay_money']]

In [71]:
# A202207050851373 应付日期修复
df_xz_concat['应付日期'] = np.where((df_xz_concat.应付日期.isnull()) & (df_xz_concat.order_number=='A202207050851373'),pd.to_datetime('2023-07-05') ,df_xz_concat.应付日期 )
df_xz_concat[df_xz_concat.应付日期.isnull()][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期new'
, 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','outpay_money']]

,order_number,order_id,当前应付租金,实付金额,应付日期,实付日期new,status_x,当前期数,rembursement_status,订单完成时间,订单状态值,finish_date_new,nn,outpay_money


In [72]:
# 实付金额>0  实付日期.isnull()  还款状态==已买断  :  逾期天数=0  实付日期=订单完成时间
# 实付金额>0  实付日期.isnull()  还款状态==已逾期  :实付金额 =0  
# 实付金额==0  实付日期.notnull()  还款状态== '续租中' 实付金额=当前应付租金   还款状态== '已逾期' 实付日期=pd.na
# 0<实付金额<当前应付租金  实付日期.notnull()  还款状态== '已逾期'  :  实付金额=0  & 实付日期=pd.na

# df_xz_concat['实付金额new'] = df_xz_concat.apply(lambda x:
#     0 if x.实付金额>0 and pd.isna(x.实付日期) and x.还款状态=='已逾期'
#     else 0 if x.实付金额>0 and x.实付金额<x.当前应付租金 and pd.notna(x.实付日期) and x.还款状态=='已逾期'
#     else x.当前应付租金 if x.实付金额==0 and pd.notna(x.实付日期) and x.还款状态=='续租中'
#     else x.实付金额
#     ,axis = 1)



In [73]:
# df_xz_concat['实付日期new'] = df_xz_concat.apply(lambda x:
#     x.订单完成时间 if x.实付金额>0 and pd.isna(x.实付日期) and x.还款状态=='已买断'
#     else np.nan if x.实付金额>=0 and x.实付金额 < x.当前应付租金 and pd.notna(x.实付日期) and x.还款状态=='已逾期'
#     else x.实付日期
#     ,axis = 1)

## 获取逾期截止时间

In [74]:
df_xz_concat['now_date'] = pd.Timestamp(dt.now().date() )
df_xz_concat['now_date']

0        2024-10-25
1        2024-10-25
2        2024-10-25
3        2024-10-25
4        2024-10-25
            ...    
310633   2024-10-25
310634   2024-10-25
310635   2024-10-25
310636   2024-10-25
310637   2024-10-25
Name: now_date, Length: 310554, dtype: datetime64[s]

## 重新定义状态

In [75]:
# df_xz_concat.columns

In [76]:
df_xz_concat[(df_xz_concat.实付日期new.isnull()) & (~df_xz_concat.订单状态值.isin([8,10]))][['rembursement_status']].value_counts()

rembursement_status
还款中                    69326
已逾期                    30126
未到首期还款日                14670
续租中                    14431
检测中                       13
Name: count, dtype: int64

In [77]:
df_xz_concat[(df_xz_concat.实付日期new.isnull())& (pd.isna(df_xz_concat.finish_date_new))][['订单状态']].value_counts()

订单状态
租赁中     122668
待归还       4033
已退款       2604
待收货        930
待发货        922
已完成        214
检测中         13
Name: count, dtype: int64

In [78]:
conditions = [  
    ((pd.isna(df_xz_concat['实付日期new'])) & (pd.isna(df_xz_concat['finish_date_new'])) & (df_xz_concat['订单状态值'].isin([8, 10]))),  
    ((pd.to_datetime(df_xz_concat['now_date']) <= pd.to_datetime(df_xz_concat['应付日期'])) ),  
    ((pd.notna(df_xz_concat['实付日期new'])) & (pd.to_datetime(df_xz_concat['实付日期new']) <= pd.to_datetime(df_xz_concat['应付日期']))),  
    ((pd.notna(df_xz_concat['实付日期new'])) & (pd.to_datetime(df_xz_concat['实付日期new']) > pd.to_datetime(df_xz_concat['应付日期']))) ,
    ((pd.isna(df_xz_concat['实付日期new'])) & (pd.to_datetime(df_xz_concat['now_date']) > pd.to_datetime(df_xz_concat['应付日期'])) )  

]  
  
choices = ['已取消', '未到还款日', '正常还款', '已逾期支付','已逾期']  
  
# 使用numpy.where来应用条件  
df_xz_concat['状态'] = np.select(conditions, choices, default='其他')

df_xz_concat['状态'].value_counts()

状态
正常还款     136268
未到还款日    130588
已逾期支付     21898
已逾期       18982
已取消        2818
Name: count, dtype: int64

In [79]:
df_xz_concat.head()

,id_x,order_id,当前应付租金,实付金额,interest_money,应付日期,实付日期,status_x,当前期数,order_pay_id,create_time_x,update_time_x,type,advance_time,merchant_account_id,order_relet_id,is_relet,pay_type,id_y,order_number,order_time,search_time,end_date,merchant_id,merchant_name,channel_id,channel_name,model_number,specification,purchase_amount,platform_package,all_deposit,all_rental,liquidated_damages,buyout_balance_payment,advance_periods,advance_sum,order_create_time,rembursement_status,paid_periods,begin_overdue_time,overdue_periods,max_overdue_days,now_overdue_days,total_receivable,contract_price,rent_received,unearned_rent_revenue,paid_buyout_balance_payment,total_received,total_unreceived,unrecovered_cost,overdue_type,gross_profit_receivable,overdue_loss,actual_gross_profit,overdue_total_unreceived,relet_periods,relet_days,relet_rent_paid,activity_id,activity_source,order_label,create_time_y,update_time_y,order_finish_date,commodity_premium,overdue_fine,pause_reimbursement,create_by_id,update_by_id,del_flag,订单状态值,订单状态,订单完成时间,order_method,true_name,id_card_num,mobile,订单号,渠道名称,归属渠道,内存,颜色,是否二手,下单日期,月份,年份,季度,outpay_money,pay_date,status_y,if_outpay,finish_date_new,nn,reality_refund_date,real_pay_money,实付日期1,实付日期new,now_date,状态
0,6506,8924,357.95,357.95,0.0,1656979200000000000,2022-07-05,3,1,9576,2022-07-05 08:51:37,2022-07-05 10:41:10,0,NaT,6270.0,NaN,0,0,1,A202207050851373,2022年07月,2022-07,2024-10-02,14,迪瓜优选,0,未知渠道,99新 iPhone 12 Pro 国行 官方质检【租物特惠】,内存：128G 颜色：金色,5150.0,租完归还/可买断,5303.0,4295.4,0.0,3181.8,1,357.95,2022-07-05 08:51:37,已买断,12,NaT,0,4,0,7477.2,7530.0,7847.2,-3881.52,329.72,8176.92,-699.72,-3026.92,,2327.2,0.0,3026.92,0.0,11,301,3551.8,NaN,未知活动,,2024-07-16 11:27:01,2024-07-18 15:50:56,2024-04-17 01:37:33,0.0,0.0,0,None,None,0,8,已完成,2024-04-17 01:37:33,0,胡水飘,362202199805282550,13027948681,A202207050851373,未知渠道,未知渠道,128G,金色,二手,2022-07-05,7,2022,3季度,329.72,2024-04-17 01:37:33,2.0,已买断,2024-04-17,NaN,NaN,NaN,2022-07-05,2022-07-05,2024-10-25,正常还款
1,6507,8924,357.95,357.95,0.0,1659657600000000000,2022-08-05,3,2,9576,2022-07-05 08:51:37,2022-08-05 02:00:02,0,NaT,6277.0,NaN,0,0,1,A202207050851373,2022年07月,2022-07,2024-10-02,14,迪瓜优选,0,未知渠道,99新 iPhone 12 Pro 国行 官方质检【租物特惠】,内存：128G 颜色：金色,5150.0,租完归还/可买断,5303.0,4295.4,0.0,3181.8,1,357.95,2022-07-05 08:51:37,已买断,12,NaT,0,4,0,7477.2,7530.0,7847.2,-3881.52,329.72,8176.92,-699.72,-3026.92,,2327.2,0.0,3026.92,0.0,11,301,3551.8,NaN,未知活动,,2024-07-16 11:27:01,2024-07-18 15:50:56,2024-04-17 01:37:33,0.0,0.0,0,None,None,0,8,已完成,2024-04-17 01:37:33,0,胡水飘,362202199805282550,13027948681,A202207050851373,未知渠道,未知渠道,128G,金色,二手,2022-07-05,7,2022,3季度,329.72,2024-04-17 01:37:33,2.0,已买断,2024-04-17,NaN,NaN,NaN,2022-08-05,2022-08-05,2024-10-25,正常还款
2,6508,8924,357.95,357.95,0.0,1662336000000000000,2022-09-05,3,3,9576,2022-07-05 08:51:37,2022-09-05 02:00:02,0,NaT,6284.0,NaN,0,0,1,A202207050851373,2022年07月,2022-07,2024-10-02,14,迪瓜优选,0,未知渠道,99新 iPhone 12 Pro 国行 官方质检【租物特惠】,内存：128G 颜色：金色,5150.0,租完归还/可买断,5303.0,4295.4,0.0,3181.8,1,357.95,2022-07-05 08:51:37,已买断,12,NaT,0,4,0,7477.2,7530.0,7847.2,-3881.52,329.72,8176.92,-699.72,-3026.92,,2327.2,0.0,3026.92,0.0,11,301,3551.8,NaN,未知活动,,2024-07-16 11:27:01,2024-07-18 15:50:56,2024-04-17 01:37:33,0.0,0.0,0,None,None,0,8,已完成,2024-04-17 01:37:33,0,胡水飘,362202199805282550,13027948681,A202207050851373,未知渠道,未知渠道,128G,金色,二手,2022-07-05,7,2022,3季度,329.72,2024-04-17 01:37:33,2.0,已买断,2024-04-17,NaN,NaN,NaN,2022-09-05,2022-09-05,2024-10-25,正常还款
3,6509,8924,357.95,357.95,0.0,1664928000000000000,2022-10-09,3,4,9576,2022-07-05 08:51:37,2022-10-09 02:00:02,0,NaT,6292.0,NaN,0,0,1,A202207050851373,2022年07月,2022-07,2024-10-02,14,迪瓜优选,0,未知渠道,99新 iPhone 12 Pro 国行 官方质检【租物特惠】,内存：128G 颜色：金色,5150.0,租完归还/可买断,5303.0,4295.4,0.0,3181.8,1,357.95,2022-07-05 08:51:37,已买断,12,NaT,0,4,0,7477.2,7530.0,7847.2,-3881.52,329.72,8176.92,-699.72,-3026.92,,2327.2,0.0,3026.92,0.0,11,301,3551.8,NaN,未知活动,,2024-07-16 11:27:01,2024-07-18 15:50:56,2024-04-17 01:37:33,0.0,0.0,0,None,None,0,8,已完成,2024-04-17 01:37:33,0,胡水飘,362202199805282

In [80]:
df_xz_concat.shape

(310554, 101)

In [81]:
df_xz_concat.实付日期new.value_counts()

实付日期new
2024-10-15    889
2024-10-10    768
2024-09-19    759
2024-08-15    751
2024-10-18    750
             ... 
2022-10-30      1
2022-09-07      1
2022-07-21      1
2022-07-13      1
2022-09-27      1
Name: count, Length: 834, dtype: int64

In [82]:
df_xz_concat.info()

<class 'pandas.core.frame.DataFrame'>
Index: 310554 entries, 0 to 310637
Columns: 101 entries, id_x to 状态
dtypes: datetime64[ns](12), datetime64[s](1), float64(32), int32(2), int64(21), object(33)
memory usage: 239.3+ MB


In [83]:
# df_xz_concat['实付日期new'] = pd.to_datetime(df_xz_concat['实付日期new'], errors='coerce')
# df_xz_concat['应付日期'] = pd.to_datetime(df_xz_concat['实付日期new'], errors='coerce')

In [84]:
# # E:/myfile/业务相关数据/风控相关数据输出_{Today}.xlsx
# df_xz_concat.to_excel('E:/myfile/业务相关数据/报错表0902.xlsx')

## 重新定义逾期天数

In [85]:
df_xz_concat['实付日期new'].isnull().sum()

np.int64(131384)

In [86]:
empty_rows = df_xz_concat[df_xz_concat['应付日期'].isna()]
empty_rows.order_number  

Series([], Name: order_number, dtype: object)

In [87]:
empty_rows.order_number 

Series([], Name: order_number, dtype: object)

In [88]:

# 实付日期为空且完成时间为空  订单状态为已退款或已完成  :  逾期天数=0 

df_xz_concat['逾期天数'] = df_xz_concat.apply(lambda x:
    0 if x['状态'] in ['未到还款日','正常还款','已取消']  
    else (pd.to_datetime(x['实付日期new']) - pd.to_datetime(x['应付日期'])).days  if pd.to_datetime(x['实付日期new']) >= pd.to_datetime(x['应付日期'])
    else  (pd.to_datetime(x['now_date']) - pd.to_datetime(x['应付日期'])).days   if   pd.to_datetime(x['now_date']) > pd.to_datetime(x['应付日期']) and pd.isna(x['实付日期new'])
    else 0 
    ,  axis=1)

df_xz_concat['逾期天数'].value_counts()

逾期天数
0      269687
1        4875
2        2693
3        2173
4        1869
        ...  
701         1
640         1
738         1
732         1
707         1
Name: count, Length: 699, dtype: int64

In [89]:
df_xz_concat[df_xz_concat['逾期天数']>0]['rembursement_status'].value_counts()

rembursement_status
已逾期        28147
还款中         5141
已买断         3450
续租中         3273
已完成          711
检测中          143
未到首期还款日        2
Name: count, dtype: int64

# 逾期天数处理

In [90]:
def over_due_days(df):  
    # 转换日期为pandas的日期类型
    df['实付日期'] = pd.to_datetime(df['实付日期'])  
    df['应付日期'] = pd.to_datetime(df['应付日期'])  
      
    # 创建一个新的列来存储结果  
    df['逾期天数new'] = np.nan  # 初始化为NaN  
      
    # 应用逻辑判断  
    mask1 = df['状态'].isin(['已逾期', '未到还款日', '已取消','已提前支付'])  
    mask2 = df['实付日期'].isnull()  
    mask3 = (df['实付日期'] - df['应付日期']).dt.days <= 0  
      
    # 更新计算天数列  
    df.loc[mask1, '逾期天数new'] = df.loc[mask1, '逾期天数']  
    df.loc[~mask1 & mask2, '逾期天数new'] = 0   
    df.loc[~mask1 & ~mask2 & mask3, '逾期天数new'] = 0  
    df.loc[~mask1 & ~mask2 & ~mask3, '逾期天数new'] = (df['实付日期'] - df['应付日期']).dt.days  
      
    # 实付日期为空时设置计算天数为0：  
    # df.loc[df['实付日期'].isnull(), '逾期天数new'] = 0  
      
    return df 

    # df_ddfq = over_due_days(df_ddfq)

##  核查 

In [91]:
# df_ddfq.columns

In [92]:
# df_xz_concat.shape

In [93]:
# check_fq_df = df_xz_concat.merge(df_ddfq,on=['订单ID','当前期数'],how='left')
# check_fq_df.shape

In [94]:
# check_fq_df.columns

In [95]:
# check_fq_df[['订单ID', '当前应付租金_x', '实付金额','应付日期_x', '实付日期_x','status', '当前期数', 'is_relet','订单号_x','下单月份', '下单日期','还款状态', '实付金额new', '实付日期new', '逾期天数_x','状态_x'
            #  ,'当前应付租金_y', '实际支付金额', '应付日期_y', '实付日期_y', '状态_y', '逾期天数_y','逾期天数new']].to_excel('E:/myfile/业务相关数据/check_fq_df.xlsx')

# 确认是否到表现期

In [96]:
# import os
# file_name = os.path.basename(f_path_ck)  

# df_xz_concat['date_str'] = file_name.split('excel')[1].split('.')[0]

# df_xz_concat['date_str'].head(1)

In [97]:
# df_xz_concat['now_date'] = pd.Timestamp(dt.now().date() )
# df_xz_concat['now_date']

In [98]:
# dfzh1 = over_due_days(dfzh1)

df_xz_concat['now_date'] = pd.Timestamp(dt.now().date() )
df_xz_concat['应付日期'] = pd.to_datetime(df_xz_concat['应付日期']).dt.date
# df_xz_concat['agr_days'] = (pd.to_datetime(df_xz_concat['now_date']) - df_xz_concat['应付日期']).dt.days 


In [99]:

from datetime import timedelta
df_xz_concat['agr_1d'] = pd.to_datetime(df_xz_concat['应付日期']) + timedelta(days=1)
df_xz_concat['agr_4d'] = pd.to_datetime(df_xz_concat['应付日期']) + timedelta(days=4)
df_xz_concat['agr_7d'] = pd.to_datetime(df_xz_concat['应付日期']) + timedelta(days=7)
df_xz_concat['agr_15d'] = pd.to_datetime(df_xz_concat['应付日期']) + timedelta(days=15)
df_xz_concat['agr_30d'] = pd.to_datetime(df_xz_concat['应付日期']) + timedelta(days=30)

df_xz_concat[['agr_30d','应付日期']].head(10)

,agr_30d,应付日期
0,2022-08-04,2022-07-05
1,2022-09-04,2022-08-05
2,2022-10-05,2022-09-05
3,2022-11-04,2022-10-05
4,2022-12-05,2022-11-05
5,2023-01-04,2022-12-05
6,2023-02-04,2023-01-05
7,2023-03-07,2023-02-05
8,2023-04-04,2023-03-05
9,2023-05-05,2023-04-05


In [100]:
import os

# file_name = os.path.basename(f_path_ck)  
df_xz_concat['date_str'] = df_xz_concat['now_date']
df_xz_concat['agr_1d_cum'] = np.where(pd.to_datetime(df_xz_concat.agr_1d) <= pd.to_datetime(df_xz_concat.date_str),1,0)

df_xz_concat['agr_4d_cum'] = np.where(pd.to_datetime(df_xz_concat.agr_4d) <= pd.to_datetime(df_xz_concat.date_str),1,0)
df_xz_concat['agr_7d_cum'] = np.where(pd.to_datetime(df_xz_concat.agr_7d) <= pd.to_datetime(df_xz_concat.date_str),1,0)
df_xz_concat['agr_15d_cum'] = np.where(pd.to_datetime(df_xz_concat.agr_15d) <= pd.to_datetime(df_xz_concat.date_str),1,0)
df_xz_concat['agr_30d_cum'] = np.where(pd.to_datetime(df_xz_concat.agr_30d) <= pd.to_datetime(df_xz_concat.date_str),1,0)


df_xz_concat[['agr_30d','应付日期','date_str','agr_30d_cum']].head(10)

,agr_30d,应付日期,date_str,agr_30d_cum
0,2022-08-04,2022-07-05,2024-10-25,1
1,2022-09-04,2022-08-05,2024-10-25,1
2,2022-10-05,2022-09-05,2024-10-25,1
3,2022-11-04,2022-10-05,2024-10-25,1
4,2022-12-05,2022-11-05,2024-10-25,1
5,2023-01-04,2022-12-05,2024-10-25,1
6,2023-02-04,2023-01-05,2024-10-25,1
7,2023-03-07,2023-02-05,2024-10-25,1
8,2023-04-04,2023-03-05,2024-10-25,1
9,2023-05-05,2023-04-05,2024-10-25,1


# 观察日定义  每月月底时间

In [101]:
from pandas.tseries.offsets import MonthEnd
def mob_date(df):
    df['下单日期'] = pd.to_datetime(df['下单日期'])
    df['实付日期new'] = pd.to_datetime(df['实付日期new'])
    for n in range(0,13):  
        df[f'mob_date_{n}'] = df['下单日期'] + MonthEnd(n+1) 
        df[f'mob_date_{n}'] = pd.to_datetime(df[f'mob_date_{n}'])
        df[f'paid_date_mob{n}'] = np.where(df['实付日期new']  > df[f'mob_date_{n}'],pd.NaT,df['实付日期new'])
        df[f'paid_money_mob{n}'] = np.where(df['实付日期new']  > df[f'mob_date_{n}'],np.nan,df['实付金额'])
        df[f'paid_date_mob{n}'] = pd.to_datetime(df[f'paid_date_mob{n}'])
    return df

df_xz_concat = mob_date(df_xz_concat)
df_xz_concat[['下单日期','mob_date_0','mob_date_1','mob_date_2','mob_date_3','实付金额','paid_money_mob1','paid_date_mob12','paid_money_mob12']]

,下单日期,mob_date_0,mob_date_1,mob_date_2,mob_date_3,实付金额,paid_money_mob1,paid_date_mob12,paid_money_mob12
0,2022-07-05,2022-07-31,2022-08-31,2022-09-30,2022-10-31,357.95,357.95,2022-07-05,357.95
1,2022-07-05,2022-07-31,2022-08-31,2022-09-30,2022-10-31,357.95,357.95,2022-08-05,357.95
2,2022-07-05,2022-07-31,2022-08-31,2022-09-30,2022-10-31,357.95,NaN,2022-09-05,357.95
3,2022-07-05,2022-07-31,2022-08-31,2022-09-30,2022-10-31,357.95,NaN,2022-10-09,357.95
4,2022-07-05,2022-07-31,2022-08-31,2022-09-30,2022-10-31,357.95,NaN,2022-11-08,357.95
...,...,...,...,...,...,...,...,...,...
310633,2024-10-24,2024-10-31,2024-11-30,2024-12-31,2025-01-31,0.00,0.00,NaT,0.00
310634,2024-10-24,2024-10-31,2024-11-30,2024-12-31,2025-01-31,0.00,0.00,NaT,0.00
310635,2024-10-24,2024-10-31,2024-11-30,2024-12-31,2025-01-31,0.00,0.00,NaT,0.00
310636,2024-10-24,2024-10-31,2024-11-30,2024-12-31,2025-01-31,0.00,0.00,NaT,0.00


In [102]:
df_xz_concat['date_str']  = pd.to_datetime( df_xz_concat['date_str'] )
df_xz_concat['应付日期'] = pd.to_datetime(df_xz_concat['应付日期']) 

In [103]:
# df_xz_concat.columns

In [104]:
# df_xz_concat[df_xz_concat.order_id== 12514][['下单日期','mob_date_10','mob_date_11','实付金额','paid_money_mob10','paid_money_mob11','paid_date_mob12','paid_money_mob12']]


# 续租用户

In [105]:
df_xz_concat.is_relet.value_counts()

is_relet
0    284883
1     25610
2        61
Name: count, dtype: int64

In [106]:
df_relet = df_xz_concat[df_xz_concat.is_relet>=1]
df_relet.shape

(25671, 152)

## 每个mob对应的逾期天数

In [107]:
from pandas.tseries.offsets import MonthEnd
def mob_date(df):
    df['下单日期'] = pd.to_datetime(df['下单日期'])
    df['实付日期new'] = pd.to_datetime(df['实付日期new'])
    for n in range(13,24):  
        df[f'mob_date_{n}'] = df['下单日期'] + MonthEnd(n+1) 
        df[f'mob_date_{n}'] = pd.to_datetime(df[f'mob_date_{n}'])
        df[f'paid_date_mob{n}'] = np.where(df['实付日期new']  > df[f'mob_date_{n}'],pd.NaT,df['实付日期new'])
        df[f'paid_money_mob{n}'] = np.where(df['实付日期new']  > df[f'mob_date_{n}'],np.nan,df['实付金额'])
        df[f'paid_date_mob{n}'] = pd.to_datetime(df[f'paid_date_mob{n}'])
    return df

df_relet = mob_date(df_relet)
df_relet[['下单日期','mob_date_15','mob_date_21','mob_date_20','mob_date_13','实付金额','paid_money_mob13','paid_date_mob15','paid_money_mob18']]

,下单日期,mob_date_15,mob_date_21,mob_date_20,mob_date_13,实付金额,paid_money_mob13,paid_date_mob15,paid_money_mob18
12,2022-07-05,2023-10-31,2024-04-30,2024-03-31,2023-08-31,354.0,354.0,2023-07-07,354.0
13,2022-07-05,2023-10-31,2024-04-30,2024-03-31,2023-08-31,354.0,354.0,2023-08-07,354.0
14,2022-07-05,2023-10-31,2024-04-30,2024-03-31,2023-08-31,354.0,NaN,2023-09-06,354.0
15,2022-07-05,2023-10-31,2024-04-30,2024-03-31,2023-08-31,365.8,NaN,2023-09-29,365.8
16,2022-07-05,2023-10-31,2024-04-30,2024-03-31,2023-08-31,354.0,NaN,NaT,354.0
...,...,...,...,...,...,...,...,...,...
160429,2024-04-02,2025-07-31,2026-01-31,2025-12-31,2025-05-31,0.0,0.0,NaT,0.0
160430,2024-04-02,2025-07-31,2026-01-31,2025-12-31,2025-05-31,0.0,0.0,NaT,0.0
160431,2024-04-02,2025-07-31,2026-01-31,2025-12-31,2025-05-31,0.0,0.0,NaT,0.0
160432,2024-04-02,2025-07-31,2026-01-31,2025-12-31,2025-05-31,0.0,0.0,NaT,0.0


In [108]:
# 每个mob对应的逾期天数  mob24无逾期数据
df_mob_list = []
for i in range(11,24):  #mob24无逾期数据
    df_mob = df_relet[((df_relet[f'mob_date_{i}'] - df_relet['应付日期']).dt.days > 0) 
                      & ((df_relet['date_str'] - df_relet['应付日期']).dt.days > 0) 
                      & ((df_relet['date_str'] - df_relet[f'mob_date_{i}']).dt.days > 0)]  
   
    conditions = [ ( ( df_mob['逾期天数']==0) | (pd.notnull(df_mob[f'paid_date_mob{i}']))),  
                  (((df_mob[f'mob_date_{i}'] - df_mob['date_str']).dt.days > 0)  & (pd.isnull(df_mob[f'paid_date_mob{i}'])) ),  
                  (((df_mob[f'mob_date_{i}'] - df_mob['date_str']).dt.days <= 0) & ( pd.isnull(df_mob[f'paid_date_mob{i}'])))]  
  
    choices = [0,(df_mob['date_str'] - df_mob['应付日期']).dt.days,(df_mob[f'mob_date_{i}'] - df_mob['应付日期']).dt.days]  
  
    # 使用numpy.where来应用条件  
    df_mob['overdue_days_mob'] = np.select(conditions, choices, default = df_mob['逾期天数'])

    df_mob['paid_money_mob'] = df_mob[f'paid_money_mob{i}']

    df_mob_group = df_mob.groupby(by=['order_number','下单日期']).agg({'当前应付租金':'sum','paid_money_mob':'sum','overdue_days_mob':'max'}).reset_index()
    df_mob_group['MOB'] = f'mob_{i}'

    df_mob_list.append(df_mob_group)

df_mob_relet = pd.concat(df_mob_list, ignore_index=True)
df_mob_relet

,order_number,下单日期,当前应付租金,paid_money_mob,overdue_days_mob,MOB
0,A202207050851373,2022-07-05,354.00,354.00,0,mob_12
1,A2022072220482114,2022-07-22,495.00,0.00,9,mob_12
2,A202207250005332,2022-07-25,293.40,293.40,0,mob_12
3,A2022072701455912,2022-07-27,297.00,0.00,3,mob_12
4,A202207282047042,2022-07-28,571.50,0.00,2,mob_12
...,...,...,...,...,...,...
6807,A202210260958545,2022-10-26,857.48,857.48,0,mob_23
6808,A202210262316508,2022-10-26,4465.50,4465.50,0,mob_23
6809,A202210272136577,2022-10-27,4803.22,2103.60,200,mob_23
6810,A2022102818505912,2022-10-28,5084.00,930.00,273,mob_23


In [109]:
df_mob_relet['overdue_days_1'] = np.where(df_mob_relet['overdue_days_mob']>= 1,1,0)
df_mob_relet['overdue_days_4'] = np.where(df_mob_relet['overdue_days_mob']>= 4,1,0)
df_mob_relet['overdue_days_7'] = np.where(df_mob_relet['overdue_days_mob']>= 7,1,0)
df_mob_relet['overdue_days_15'] = np.where(df_mob_relet['overdue_days_mob']>= 15,1,0)
df_mob_relet['overdue_days_31'] = np.where(df_mob_relet['overdue_days_mob']>= 31,1,0)
df_mob_relet['overdue_days_30'] = np.where(df_mob_relet['overdue_days_mob']>= 30,1,0)
df_mob_relet['overdue_days_90'] = np.where(df_mob_relet['overdue_days_mob']>= 90,1,0)
df_mob_relet['下单月份'] = df_mob_relet['下单日期'].dt.strftime('%Y-%m')  

In [110]:
# df_15_all = group_data(df_15, '型号')
# df_15_all = group_data(df_15, '下单日期')
# df_15_all = df_15_all[["去重订单数","前置拦截","拦截率","是否进件","预授权通过率","机审强拒","强拒比例","机审通过件", '人审拒绝', "风控通过件","风控通过率","进件前取消", "出库前风控强拒","出库前强拒比例","是否出库",'取消率','人审拒绝率',"进件出库率","订单出库率"]]


# df_15_all = df_15_all[["去重订单数","前置拦截","拦截率","是否进件","预授权通过率","机审强拒","强拒比例","机审通过件", "风控通过件","风控通过率", '客户取消', "出库前风控强拒","出库前强拒比例","是否出库","进件出库率","订单出库率"]]
# df_15_all

In [111]:
# df_mob_total['采购金额'] 
df_mob_relet = df_mob_relet.merge(dfzh1[['order_number','purchase_amount','归属渠道']],on = 'order_number',how='left')
df_mob_relet.head()

,order_number,下单日期,当前应付租金,paid_money_mob,overdue_days_mob,MOB,overdue_days_1,overdue_days_4,overdue_days_7,overdue_days_15,overdue_days_31,overdue_days_30,overdue_days_90,下单月份,purchase_amount,归属渠道
0,A202207050851373,2022-07-05,354.0,354.0,0,mob_12,0,0,0,0,0,0,0,2022-07,5150.0,未知渠道
1,A2022072220482114,2022-07-22,495.0,0.0,9,mob_12,1,1,1,0,0,0,0,2022-07,5890.0,其他渠道场景渠道。
2,A202207250005332,2022-07-25,293.4,293.4,0,mob_12,0,0,0,0,0,0,0,2022-07,3867.0,搜索渠道
3,A2022072701455912,2022-07-27,297.0,0.0,3,mob_12,1,0,0,0,0,0,0,2022-07,5280.0,其他渠道场景渠道。
4,A202207282047042,2022-07-28,571.5,0.0,2,mob_12,1,0,0,0,0,0,0,2022-07,9000.0,搜索渠道


In [112]:
df_xz_concat.columns

Index(['id_x', 'order_id', '当前应付租金', '实付金额', 'interest_money', '应付日期', '实付日期',
       'status_x', '当前期数', 'order_pay_id',
       ...
       'paid_money_mob9', 'mob_date_10', 'paid_date_mob10', 'paid_money_mob10',
       'mob_date_11', 'paid_date_mob11', 'paid_money_mob11', 'mob_date_12',
       'paid_date_mob12', 'paid_money_mob12'],
      dtype='object', length=152)

In [113]:
df_relet_mob10 = df_xz_concat[((df_xz_concat[f'mob_date_10'] - df_xz_concat['应付日期']).dt.days > 0) 
& ((df_xz_concat['date_str'] - df_xz_concat['应付日期']).dt.days > 0) 
& ((df_xz_concat['date_str'] - df_xz_concat[f'mob_date_10']).dt.days > 0)].groupby(by=['order_number','下单日期']).agg({'当前应付租金':'sum','paid_money_mob10':'sum'}).reset_index()

df_relet_mob10

,order_number,下单日期,当前应付租金,paid_money_mob10
0,A202207050851373,2022-07-05,3937.45,3937.45
1,A202207131230485,2022-07-13,6287.71,571.61
2,A202207151707153,2022-07-15,4399.23,4399.23
3,A202207162356303,2022-07-16,6287.60,6287.60
4,A202207171820512,2022-07-17,1292.61,1175.10
...,...,...,...,...
8216,A2023112917465175,2023-11-29,4326.60,4326.60
8217,A2023112918040714,2023-11-29,6662.90,6662.90
8218,A2023112918393654,2023-11-29,5985.50,5985.50
8219,A2023112920270028,2023-11-29,4663.60,4663.60


In [114]:
df_mob_relet = df_mob_relet.merge(df_relet_mob10[['order_number','paid_money_mob10']],on = 'order_number',how='left')
df_mob_relet.head()

,order_number,下单日期,当前应付租金,paid_money_mob,overdue_days_mob,MOB,overdue_days_1,overdue_days_4,overdue_days_7,overdue_days_15,overdue_days_31,overdue_days_30,overdue_days_90,下单月份,purchase_amount,归属渠道,paid_money_mob10
0,A202207050851373,2022-07-05,354.0,354.0,0,mob_12,0,0,0,0,0,0,0,2022-07,5150.0,未知渠道,3937.45
1,A2022072220482114,2022-07-22,495.0,0.0,9,mob_12,1,1,1,0,0,0,0,2022-07,5890.0,其他渠道场景渠道。,3963.30
2,A202207250005332,2022-07-25,293.4,293.4,0,mob_12,0,0,0,0,0,0,0,2022-07,3867.0,搜索渠道,3225.97
3,A2022072701455912,2022-07-27,297.0,0.0,3,mob_12,1,0,0,0,0,0,0,2022-07,5280.0,其他渠道场景渠道。,3253.50
4,A202207282047042,2022-07-28,571.5,0.0,2,mob_12,1,0,0,0,0,0,0,2022-07,9000.0,搜索渠道,6261.60


In [115]:
# 八派信息
# df_mob_relet.to_excel('C:/Users/Administrator/Desktop/风控/八派880902.xlsx')

## 续租订单 每月对应的续租用户及采购金额

In [116]:
df_relet.columns

Index(['id_x', 'order_id', '当前应付租金', '实付金额', 'interest_money', '应付日期', '实付日期',
       'status_x', '当前期数', 'order_pay_id',
       ...
       'paid_money_mob20', 'mob_date_21', 'paid_date_mob21',
       'paid_money_mob21', 'mob_date_22', 'paid_date_mob22',
       'paid_money_mob22', 'mob_date_23', 'paid_date_mob23',
       'paid_money_mob23'],
      dtype='object', length=185)

In [117]:
df_relet1 = df_relet[df_relet.当前期数==13].merge(df_relet_mob10[['order_number','paid_money_mob10']],on = 'order_number',how='left')
# df_relet1
df_relet_mon = df_relet1.groupby(by=['search_time']).agg({'order_number': 'count', '实付金额': 'sum','purchase_amount':'sum','paid_money_mob10_y':'sum'})
df_relet_mon['relet_principal'] = df_relet_mon['purchase_amount'] - df_relet_mon['paid_money_mob10_y']
df_relet_mon

,order_number,实付金额,purchase_amount,paid_money_mob10_y,relet_principal
search_time,,,,,
2022-07,5,2010.90,29187.00,20641.82,8545.18
2022-08,15,5776.54,98008.00,65333.41,32674.59
2022-09,14,4931.40,92730.85,68930.70,23800.15
2022-10,15,5420.93,87371.00,64734.60,22636.40
2022-11,17,8727.60,132745.00,83282.80,49462.20
2022-12,21,10731.90,177135.00,116275.20,60859.80
2023-01,61,49187.69,391326.00,273352.00,117974.00
2023-02,72,67235.73,457244.00,321404.44,135839.56
2023-03,69,78635.43,472351.00,301927.04,170423.96


## 续租订单 vintage 4+ 

In [118]:
vintage_relet_d4 = pd.pivot_table(df_mob_relet[df_mob_relet.overdue_days_4==1],index='下单月份',columns='MOB',values='overdue_days_4',aggfunc='sum')  #.reset_index()  #,margins=True

vintage_relet_d4 = vintage_relet_d4[['mob_12','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22']]  #'下单月份','mob_0',,'mob_23'
# vintage_df_d4
vintage_divide_relet_d4 = vintage_relet_d4.divide(df_relet_mon["order_number"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_relet_d4['订单量'] = df_relet_mon["order_number"]
vintage_divide_relet_d4[['订单量','mob_12','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22']]  #  ,'mob_23'

MOB,订单量,mob_12,mob_13,mob_14,mob_15,mob_16,mob_17,mob_18,mob_19,mob_20,mob_21,mob_22
2022-07,5,20.00%,40.00%,40.00%,20.00%,20.00%,20.00%,40.00%,40.00%,40.00%,20.00%,nan%
2022-08,15,13.33%,13.33%,20.00%,26.67%,26.67%,20.00%,33.33%,26.67%,26.67%,26.67%,26.67%
2022-09,14,35.71%,14.29%,21.43%,28.57%,35.71%,42.86%,35.71%,35.71%,35.71%,35.71%,35.71%
2022-10,15,nan%,nan%,13.33%,20.00%,26.67%,33.33%,26.67%,33.33%,33.33%,26.67%,20.00%
2022-11,17,nan%,5.88%,11.76%,17.65%,23.53%,23.53%,23.53%,23.53%,29.41%,23.53%,23.53%
2022-12,21,9.52%,9.52%,19.05%,19.05%,14.29%,14.29%,19.05%,19.05%,19.05%,19.05%,nan%
2023-01,61,4.92%,14.75%,22.95%,24.59%,27.87%,29.51%,27.87%,21.31%,19.67%,nan%,nan%
2023-02,72,9.72%,18.06%,22.22%,27.78%,30.56%,36.11%,27.78%,23.61%,nan%,nan%,nan%
2023-03,69,8.70%,13.04%,18.84%,15.94%,24.64%,28.99%,23.19%,nan%,nan%,nan%,nan%
2023-04,137,10.95%,16.79%,21.17%,26.28%,29.93%,33.58%,nan%,nan%,nan%,nan%,nan%


## 续租 金额 vintage 4+ 

In [119]:
df_mob_relet['unpaid_money_mob'] = df_mob_relet['purchase_amount'] - df_mob_relet['paid_money_mob'] - df_mob_relet['paid_money_mob10']


In [120]:
# df_mob_relet.to_excel(f'E:/myfile/业务相关数据/df_mob_relet-{date_str}.xlsx')

In [121]:
money_relet_d4 = pd.pivot_table(df_mob_relet[df_mob_relet.overdue_days_4==1],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum')  #.reset_index()  #,margins=True

money_relet_d4 = money_relet_d4[['mob_12','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22']]  #'下单月份','mob_0', ,'mob_23'
# vintage_df_d4
money_divide_relet_d4 = money_relet_d4.divide(df_relet_mon["relet_principal"],axis=0).applymap(lambda x:format(x,".2%"))
money_divide_relet_d4['relet_principal'] = df_relet_mon["relet_principal"]
money_divide_relet_d4[['relet_principal','mob_12','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22']]

MOB,relet_principal,mob_12,mob_13,mob_14,mob_15,mob_16,mob_17,mob_18,mob_19,mob_20,mob_21,mob_22
2022-07,8545.18,22.55%,42.79%,42.79%,22.55%,22.55%,22.55%,16.56%,16.56%,-4.70%,-28.15%,nan%
2022-08,32674.59,7.94%,2.77%,14.46%,16.85%,15.73%,14.46%,14.69%,13.08%,13.08%,11.51%,11.51%
2022-09,23800.15,36.34%,16.46%,18.09%,16.58%,17.15%,17.45%,15.49%,11.18%,3.97%,3.08%,3.08%
2022-10,22636.40,nan%,nan%,4.91%,9.85%,8.82%,9.67%,6.41%,-3.23%,-2.33%,-0.07%,6.38%
2022-11,49462.20,nan%,9.76%,9.78%,13.42%,12.84%,9.55%,9.55%,9.55%,6.31%,9.55%,9.55%
2022-12,60859.80,13.13%,13.13%,19.95%,18.87%,14.36%,14.36%,13.31%,13.45%,12.13%,12.13%,nan%
2023-01,117974.00,4.94%,8.75%,11.16%,9.09%,2.38%,-4.48%,-5.01%,-1.85%,-2.17%,nan%,nan%
2023-02,135839.56,9.28%,11.38%,9.87%,3.47%,-0.72%,-12.07%,-7.38%,-5.33%,nan%,nan%,nan%
2023-03,170423.96,9.25%,10.63%,10.33%,3.76%,-4.20%,-13.90%,-6.08%,nan%,nan%,nan%,nan%
2023-04,344818.84,11.75%,13.26%,12.00%,5.31%,-3.58%,-6.24%,nan%,nan%,nan%,nan%,nan%


## 续租订单 vintage 7+ 

In [122]:
vintage_relet_d7 = pd.pivot_table(df_mob_relet[df_mob_relet.overdue_days_7==1],index='下单月份',columns='MOB',values='overdue_days_7',aggfunc='sum')  #.reset_index()  #,margins=True

vintage_relet_d7 = vintage_relet_d7[['mob_12','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22']]  #'下单月份','mob_0',,'mob_23'
# vintage_df_d4
vintage_divide_relet_d7 = vintage_relet_d7.divide(df_relet_mon["order_number"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_relet_d7['订单量'] = df_relet_mon["order_number"]
vintage_divide_relet_d7[['订单量','mob_12','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22']]  #  ,'mob_23'

MOB,订单量,mob_12,mob_13,mob_14,mob_15,mob_16,mob_17,mob_18,mob_19,mob_20,mob_21,mob_22
2022-07,5,20.00%,20.00%,40.00%,20.00%,20.00%,20.00%,40.00%,40.00%,40.00%,20.00%,nan%
2022-08,15,13.33%,6.67%,20.00%,26.67%,26.67%,20.00%,33.33%,26.67%,26.67%,26.67%,26.67%
2022-09,14,7.14%,14.29%,14.29%,14.29%,28.57%,35.71%,35.71%,35.71%,35.71%,35.71%,35.71%
2022-10,15,nan%,nan%,13.33%,20.00%,26.67%,33.33%,26.67%,33.33%,33.33%,26.67%,20.00%
2022-11,17,nan%,nan%,11.76%,17.65%,17.65%,23.53%,23.53%,23.53%,29.41%,23.53%,23.53%
2022-12,21,nan%,9.52%,14.29%,19.05%,14.29%,14.29%,14.29%,19.05%,19.05%,19.05%,nan%
2023-01,61,4.92%,14.75%,21.31%,24.59%,27.87%,27.87%,27.87%,21.31%,19.67%,nan%,nan%
2023-02,72,8.33%,16.67%,19.44%,25.00%,26.39%,33.33%,27.78%,23.61%,nan%,nan%,nan%
2023-03,69,7.25%,11.59%,17.39%,14.49%,23.19%,28.99%,23.19%,nan%,nan%,nan%,nan%
2023-04,137,10.22%,16.79%,20.44%,26.28%,29.93%,32.85%,nan%,nan%,nan%,nan%,nan%


## 续租 金额 vintage 7+ 

In [123]:
money_relet_d7 = pd.pivot_table(df_mob_relet[df_mob_relet.overdue_days_7==1],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum')  #.reset_index()  #,margins=True

money_relet_d7 = money_relet_d7[['mob_12','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22']]  #'下单月份','mob_0', ,'mob_23'
# vintage_df_d4
money_divide_relet_d7 = money_relet_d7.divide(df_relet_mon["relet_principal"],axis=0).applymap(lambda x:format(x,".2%"))
money_divide_relet_d7['relet_principal'] = df_relet_mon["relet_principal"]
money_divide_relet_d7[['relet_principal','mob_12','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22']]

MOB,relet_principal,mob_12,mob_13,mob_14,mob_15,mob_16,mob_17,mob_18,mob_19,mob_20,mob_21,mob_22
2022-07,8545.18,22.55%,22.55%,42.79%,22.55%,22.55%,22.55%,16.56%,16.56%,-4.70%,-28.15%,nan%
2022-08,32674.59,7.94%,0.14%,14.46%,16.85%,15.73%,14.46%,14.69%,13.08%,13.08%,11.51%,11.51%
2022-09,23800.15,7.24%,16.46%,16.46%,16.46%,16.58%,17.15%,15.49%,11.18%,3.97%,3.08%,3.08%
2022-10,22636.40,nan%,nan%,4.91%,9.85%,8.82%,9.67%,6.41%,-3.23%,-2.33%,-0.07%,6.38%
2022-11,49462.20,nan%,nan%,9.78%,13.42%,12.19%,9.55%,9.55%,9.55%,6.31%,9.55%,9.55%
2022-12,60859.80,nan%,13.13%,16.99%,18.87%,14.36%,14.36%,14.36%,13.45%,12.13%,12.13%,nan%
2023-01,117974.00,4.94%,8.75%,10.70%,9.09%,2.38%,-3.90%,-5.01%,-1.85%,-2.17%,nan%,nan%
2023-02,135839.56,7.90%,10.61%,10.04%,5.40%,5.03%,-7.61%,-7.38%,-5.33%,nan%,nan%,nan%
2023-03,170423.96,7.08%,9.07%,10.33%,4.38%,-2.44%,-13.90%,-6.08%,nan%,nan%,nan%,nan%
2023-04,344818.84,10.97%,13.26%,11.96%,5.31%,-3.58%,-5.18%,nan%,nan%,nan%,nan%,nan%


## 续租订单 vintage 15+ 

In [124]:
vintage_relet_d15 = pd.pivot_table(df_mob_relet[df_mob_relet.overdue_days_15==1],index='下单月份',columns='MOB',values='overdue_days_15',aggfunc='sum')  #.reset_index()  #,margins=True

vintage_relet_d15 = vintage_relet_d15[['mob_12','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22']]  #'下单月份','mob_0',,'mob_23'
# vintage_df_d4
vintage_divide_relet_d15 = vintage_relet_d15.divide(df_relet_mon["order_number"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_relet_d15['订单量'] = df_relet_mon["order_number"]
vintage_divide_relet_d15[['订单量','mob_12','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22']]  #  ,'mob_23'

MOB,订单量,mob_12,mob_13,mob_14,mob_15,mob_16,mob_17,mob_18,mob_19,mob_20,mob_21,mob_22
2022-07,5,nan%,20.00%,40.00%,20.00%,20.00%,20.00%,20.00%,40.00%,20.00%,nan%,nan%
2022-08,15,6.67%,6.67%,20.00%,26.67%,26.67%,20.00%,26.67%,20.00%,26.67%,26.67%,26.67%
2022-09,14,nan%,7.14%,14.29%,14.29%,28.57%,35.71%,28.57%,28.57%,35.71%,35.71%,35.71%
2022-10,15,nan%,nan%,nan%,20.00%,13.33%,33.33%,26.67%,33.33%,26.67%,20.00%,20.00%
2022-11,17,nan%,nan%,11.76%,11.76%,17.65%,17.65%,23.53%,23.53%,23.53%,23.53%,23.53%
2022-12,21,nan%,9.52%,9.52%,19.05%,14.29%,14.29%,14.29%,19.05%,19.05%,19.05%,nan%
2023-01,61,1.64%,11.48%,18.03%,24.59%,26.23%,24.59%,27.87%,21.31%,19.67%,nan%,nan%
2023-02,72,5.56%,12.50%,19.44%,22.22%,25.00%,31.94%,27.78%,23.61%,nan%,nan%,nan%
2023-03,69,4.35%,7.25%,13.04%,11.59%,18.84%,26.09%,23.19%,nan%,nan%,nan%,nan%
2023-04,137,5.84%,14.60%,18.25%,18.98%,25.55%,29.20%,nan%,nan%,nan%,nan%,nan%


## 续租 金额 vintage 15+ 

In [125]:
money_relet_d15 = pd.pivot_table(df_mob_relet[df_mob_relet.overdue_days_15==1],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum')  #.reset_index()  #,margins=True

money_relet_d15 = money_relet_d15[['mob_12','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22']]  #'下单月份','mob_0', ,'mob_23'
# vintage_df_d4
money_divide_relet_d15 = money_relet_d15.divide(df_relet_mon["relet_principal"],axis=0).applymap(lambda x:format(x,".2%"))
money_divide_relet_d15['relet_principal'] = df_relet_mon["relet_principal"]
money_divide_relet_d15[['relet_principal','mob_12','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22']]

MOB,relet_principal,mob_12,mob_13,mob_14,mob_15,mob_16,mob_17,mob_18,mob_19,mob_20,mob_21,mob_22
2022-07,8545.18,nan%,22.55%,42.79%,22.55%,22.55%,22.55%,16.75%,16.56%,16.75%,nan%,nan%
2022-08,32674.59,7.80%,0.14%,14.46%,16.85%,15.73%,14.46%,14.53%,14.46%,13.08%,11.51%,11.51%
2022-09,23800.15,nan%,7.02%,16.46%,16.46%,16.58%,17.15%,16.58%,13.66%,3.97%,3.08%,3.08%
2022-10,22636.40,nan%,nan%,nan%,9.85%,10.16%,9.67%,6.41%,-3.23%,3.03%,6.38%,6.38%
2022-11,49462.20,nan%,nan%,9.78%,9.78%,12.19%,7.45%,9.55%,9.55%,9.55%,9.55%,9.55%
2022-12,60859.80,nan%,13.13%,13.13%,18.87%,14.36%,14.36%,14.36%,13.45%,12.13%,12.13%,nan%
2023-01,117974.00,2.02%,6.97%,10.45%,9.09%,3.56%,0.72%,-5.01%,-1.85%,-2.17%,nan%,nan%
2023-02,135839.56,5.09%,9.11%,10.04%,7.70%,5.72%,-4.70%,-7.38%,-5.33%,nan%,nan%,nan%
2023-03,170423.96,3.32%,6.24%,9.14%,5.70%,1.22%,-8.82%,-6.08%,nan%,nan%,nan%,nan%
2023-04,344818.84,7.09%,11.79%,11.91%,8.72%,1.20%,-2.22%,nan%,nan%,nan%,nan%,nan%


## 续租订单 vintage 31+ 

In [126]:
vintage_relet_d31 = pd.pivot_table(df_mob_relet[df_mob_relet.overdue_days_31==1],index='下单月份',columns='MOB',values='overdue_days_31',aggfunc='sum')  #.reset_index()  #,margins=True

vintage_relet_d31 = vintage_relet_d31[['mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22']]  #'下单月份','mob_0',,'mob_23''mob_11',
# vintage_df_d4
vintage_divide_relet_d31 = vintage_relet_d31.divide(df_relet_mon["order_number"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_relet_d31['订单量'] = df_relet_mon["order_number"]
vintage_divide_relet_d31[['订单量','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22']]  #  ,'mob_23'

MOB,订单量,mob_13,mob_14,mob_15,mob_16,mob_17,mob_18,mob_19,mob_20,mob_21,mob_22
2022-07,5,20.00%,40.00%,20.00%,20.00%,20.00%,20.00%,40.00%,20.00%,nan%,nan%
2022-08,15,6.67%,13.33%,20.00%,20.00%,20.00%,20.00%,20.00%,26.67%,26.67%,26.67%
2022-09,14,nan%,14.29%,14.29%,28.57%,35.71%,28.57%,28.57%,35.71%,35.71%,35.71%
2022-10,15,nan%,nan%,6.67%,13.33%,26.67%,26.67%,20.00%,20.00%,20.00%,20.00%
2022-11,17,nan%,5.88%,11.76%,11.76%,17.65%,23.53%,23.53%,23.53%,23.53%,23.53%
2022-12,21,9.52%,9.52%,14.29%,9.52%,14.29%,14.29%,14.29%,14.29%,19.05%,nan%
2023-01,61,1.64%,13.11%,19.67%,19.67%,21.31%,26.23%,21.31%,19.67%,nan%,nan%
2023-02,72,8.33%,13.89%,19.44%,22.22%,30.56%,27.78%,23.61%,nan%,nan%,nan%
2023-03,69,4.35%,11.59%,10.14%,14.49%,24.64%,23.19%,nan%,nan%,nan%,nan%
2023-04,137,8.03%,13.14%,16.06%,21.90%,26.28%,nan%,nan%,nan%,nan%,nan%


## 续租 金额 vintage 31+ 

In [127]:
money_relet_d31 = pd.pivot_table(df_mob_relet[df_mob_relet.overdue_days_31==1],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum')  #.reset_index()  #,margins=True

money_relet_d31 = money_relet_d31[['mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22']]  #'下单月份','mob_0', ,'mob_23''mob_11',
# vintage_df_d4
money_divide_relet_d31 = money_relet_d31.divide(df_relet_mon["relet_principal"],axis=0).applymap(lambda x:format(x,".2%"))
money_divide_relet_d31['relet_principal'] = df_relet_mon["relet_principal"]
money_divide_relet_d31[['relet_principal','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22']]

MOB,relet_principal,mob_13,mob_14,mob_15,mob_16,mob_17,mob_18,mob_19,mob_20,mob_21,mob_22
2022-07,8545.18,22.55%,42.79%,22.55%,22.55%,22.55%,16.75%,16.56%,16.75%,nan%,nan%
2022-08,32674.59,0.14%,9.35%,14.46%,14.46%,14.46%,14.46%,14.46%,13.08%,11.51%,11.51%
2022-09,23800.15,nan%,16.46%,16.46%,16.58%,17.15%,16.58%,13.66%,3.97%,3.08%,3.08%
2022-10,22636.40,nan%,nan%,-0.31%,10.16%,8.82%,6.41%,5.92%,4.40%,6.38%,6.38%
2022-11,49462.20,nan%,9.76%,9.78%,9.78%,7.45%,9.55%,9.55%,9.55%,9.55%,9.55%
2022-12,60859.80,13.13%,13.13%,16.09%,13.13%,14.36%,14.36%,14.36%,14.36%,12.13%,nan%
2023-01,117974.00,2.02%,8.33%,9.38%,7.96%,3.84%,-4.56%,-1.85%,-2.17%,nan%,nan%
2023-02,135839.56,8.31%,9.35%,8.41%,5.55%,-1.56%,-7.38%,-5.33%,nan%,nan%,nan%
2023-03,170423.96,3.20%,9.07%,6.27%,4.87%,-6.25%,-6.08%,nan%,nan%,nan%,nan%
2023-04,344818.84,8.16%,11.50%,10.26%,5.34%,1.19%,nan%,nan%,nan%,nan%,nan%


# 非续租租期  每个mob 观察日 对应的逾期天数

实付日期逻辑：如实付日期大于观察日，实付日期清空 


达到表现期逻辑：应付日期小于观察日，且应付日期小于当前日期 

逾期天数判断逻辑:如状态为已取消状态则0，如实付日期不为空则 0，如实付日期为空且观察日期大于当前日期则 当前日期-应付日期，否则 观察日期-应付日期

In [128]:
# 非续租租期数据
df_fq_concat = df_xz_concat[df_xz_concat.is_relet==0]

In [129]:
# df_fq_concat.columns

In [130]:
df_mob_list = []
for i in range(0,13):
    df_mob = df_fq_concat[((df_fq_concat[f'mob_date_{i}'] - df_fq_concat['应付日期']).dt.days > 0) 
                          & ((df_fq_concat['date_str'] - df_fq_concat['应付日期']).dt.days > 0) 
                          & ((df_fq_concat['date_str'] - df_fq_concat[f'mob_date_{i}']).dt.days > 0)]  # &((dfzh1['date_str'] - dfzh1[f'mob_date_{i}']).dt.days > 0)
# df_mob2 = dfzh1[((dfzh1['mob_date_2'] - dfzh1['应付日期']).dt.days > 0) & (dfzh1.状态 != '已取消') & ((dfzh1['date_str'] - dfzh1['mob_date_2']).dt.days > 0)]

    # df_mob['overdue_days_mob'] = df_mob.apply(lambda x:
    #                                           0 if x['逾期天数']==0  
    #                                           else 0  if pd.notnull(x[f'paid_date_mob{i}']) 
    #                                           else (x['date_str'] - x['应付日期']).days  if  ((x[f'mob_date_{i}'] - x['date_str']).days > 0)  and pd.isnull(x[f'paid_date_mob{i}'])
    #                                           else (x[f'mob_date_{i}'] - x['应付日期']).days if ((x[f'mob_date_{i}'] - x['date_str']).days <= 0)  and pd.isnull(x[f'paid_date_mob{i}'])
    #                                           else x['逾期天数'] ,axis=1 )
    # # np.where(df_mob[f'paid_date_mob{i}'].isnull(),(np.minimum(df_mob['date_str'],df_mob[f'mob_date_{i}']) - df_mob['应付日期']).dt.days  ,(df_mob[f'paid_date_mob{i}'] - df_mob['应付日期']).dt.days)
    # df_mob['paid_money_mob'] = df_mob[f'paid_money_mob{i}']

    # df_mob_group = df_mob.groupby(by=['order_number','下单日期']).agg({'当前应付租金':'sum','paid_money_mob':'sum','overdue_days_mob':'max'}).reset_index()
    # df_mob_group['MOB'] = f'mob_{i}'

    # df_mob_list.append(df_mob_group)

    conditions = [ ( ( df_mob['逾期天数']==0) | (pd.notnull(df_mob[f'paid_date_mob{i}']))),  
                  (((df_mob[f'mob_date_{i}'] - df_mob['date_str']).dt.days > 0)  & (pd.isnull(df_mob[f'paid_date_mob{i}'])) ),  
                  (((df_mob[f'mob_date_{i}'] - df_mob['date_str']).dt.days <= 0) & ( pd.isnull(df_mob[f'paid_date_mob{i}'])))]  
  
    choices = [0,(df_mob['date_str'] - df_mob['应付日期']).dt.days,(df_mob[f'mob_date_{i}'] - df_mob['应付日期']).dt.days]  
  
    # 使用numpy.where来应用条件  
    df_mob['overdue_days_mob'] = np.select(conditions, choices, default = df_mob['逾期天数'])

    df_mob['paid_money_mob'] = df_mob[f'paid_money_mob{i}']

    df_mob_group = df_mob.groupby(by=['order_number','下单日期']).agg({'当前应付租金':'sum','paid_money_mob':'sum','overdue_days_mob':'max'}).reset_index()
    df_mob_group['MOB'] = f'mob_{i}'

    df_mob_list.append(df_mob_group)

df_mob_total = pd.concat(df_mob_list, ignore_index=True)
df_mob_total.tail(10)

,order_number,下单日期,当前应付租金,paid_money_mob,overdue_days_mob,MOB
160895,A20230929125649162,2023-09-29,5940.20,5940.20,0,mob_12
160896,A2023092913285786,2023-09-29,3059.15,3059.15,0,mob_12
160897,A2023092915245034,2023-09-29,6534.21,6534.21,0,mob_12
160898,A2023092916270939,2023-09-29,6534.21,6534.21,0,mob_12
160899,A2023092917273729,2023-09-29,5330.00,5330.00,0,mob_12
160900,A202309291903233,2023-09-29,6534.21,6534.21,0,mob_12
160901,A2023092919075110,2023-09-29,9097.30,9097.30,0,mob_12
160902,A2023092920393638,2023-09-29,10397.10,6929.92,120,mob_12
160903,A2023092920574147,2023-09-29,4391.68,4391.68,0,mob_12
160904,A2023092922274841,2023-09-29,5529.21,5529.21,0,mob_12


In [131]:
# df_mob_total[df_mob_total.订单号=='A202208061505082']

In [132]:
df_mob_total['overdue_days_1'] = np.where(df_mob_total['overdue_days_mob']>= 1,1,0)
df_mob_total['overdue_days_4'] = np.where(df_mob_total['overdue_days_mob']>= 4,1,0)
df_mob_total['overdue_days_7'] = np.where(df_mob_total['overdue_days_mob']>= 7,1,0)
df_mob_total['overdue_days_15'] = np.where(df_mob_total['overdue_days_mob']>= 15,1,0)
df_mob_total['overdue_days_31'] = np.where(df_mob_total['overdue_days_mob']>= 31,1,0)
df_mob_total['overdue_days_30'] = np.where(df_mob_total['overdue_days_mob']>= 30,1,0)
df_mob_total['overdue_days_90'] = np.where(df_mob_total['overdue_days_mob']>= 90,1,0)
df_mob_total['下单月份'] = df_mob_total['下单日期'].dt.strftime('%Y-%m')  

In [133]:
# dfzh3.columns

In [134]:
# df_mob_total['采购金额'] 
df_mob_total = df_mob_total.merge(dfzh3[['order_number','purchase_amount','归属渠道']],on = 'order_number',how='left')
df_mob_total.shape

(160996, 16)

In [135]:
# df_mob_total = df_mob_total[df_mob_total['归属渠道']!='抖音渠道']

In [136]:
from datetime import datetime as dt    
today_date = dt.now().date()  
date_str = today_date.strftime('%Y-%m-%d')  

# df_mob_total.to_excel(f'E:/myfile/业务相关数据/df_mob_total-{date_str}.xlsx')

## 每月对应的 用户数及应还金额

### 总体

In [137]:
df_mon = df_fq_concat[df_fq_concat.当前期数==1].groupby(by=['search_time']).agg({'order_number': 'count', 'contract_price': 'sum','total_receivable':'sum','purchase_amount':'sum'})
# df_mon.columns = df_mon.columns.droplevel(1) 
df_mon

,order_number,contract_price,total_receivable,purchase_amount
search_time,,,,
2022-07,22,195693.00,179868.68,125405.00
2022-08,56,527606.04,506786.10,358451.00
2022-09,44,402654.00,399642.06,277982.75
2022-10,41,337968.00,341574.52,249543.09
2022-11,43,440057.00,441724.49,344258.81
2022-12,66,740353.20,755276.75,566276.59
2023-01,221,2173455.52,2132088.54,1533459.00
2023-02,332,3218625.44,3135967.82,2181552.82
2023-03,316,3063400.23,2878390.23,2083021.00


### 租物渠道

In [138]:
df_mon_zm = df_fq_concat[(df_fq_concat.当前期数==1) & (df_fq_concat.归属渠道=='芝麻租物')].groupby(by=['search_time']).agg({'order_number': 'count', 'contract_price': 'sum','total_receivable':'sum','purchase_amount':'sum'})
# df_mon.columns = df_mon.columns.droplevel(1) 
df_mon_zm

,order_number,contract_price,total_receivable,purchase_amount
search_time,,,,
2022-09,5,34573.00,32602.60,24742.68
2022-10,10,65644.00,61320.59,48835.00
2022-11,10,89430.00,83635.09,68730.22
2022-12,2,10106.20,9214.70,7657.00
2023-01,18,158715.70,149223.48,110070.00
2023-02,38,306451.62,249389.58,206619.00
2023-03,61,550399.00,465525.32,374912.00
2023-04,240,2442118.00,2099147.04,1628122.00
2023-05,145,1413669.00,1220124.44,905764.00


### 搜索渠道

In [139]:
df_mon_ss = df_fq_concat[(df_fq_concat.当前期数==1) & (df_fq_concat.归属渠道=='搜索渠道')].groupby(by=['search_time']).agg({'order_number': 'count', 'contract_price': 'sum','total_receivable':'sum','purchase_amount':'sum'})
# df_mon.columns = df_mon.columns.droplevel(1) 
df_mon_ss

,order_number,contract_price,total_receivable,purchase_amount
search_time,,,,
2022-07,7,67151.00,66981.29,46581.00
2022-08,22,210198.00,204766.81,145114.00
2022-09,8,96962.00,97022.13,67126.34
2022-10,6,59404.00,63328.32,44442.63
2022-11,10,113610.00,115602.98,88153.59
2022-12,37,424727.00,437501.83,324164.59
2023-01,134,1337111.40,1312608.65,944943.00
2023-02,200,2015258.42,2005418.10,1369414.82
2023-03,150,1468754.23,1411490.92,1002057.00


### 抖音渠道

In [140]:
df_mon_dy = df_fq_concat[(df_fq_concat.当前期数==1) & (df_fq_concat.归属渠道=='抖音渠道')].groupby(by=['search_time']).agg({'order_number': 'count', 'contract_price': 'sum','total_receivable':'sum','purchase_amount':'sum'})
# df_mon.columns = df_mon.columns.droplevel(1) 
df_mon_dy

,order_number,contract_price,total_receivable,purchase_amount
search_time,,,,
2024-04,4,26875.0,21921.15,15930.0
2024-05,3,17517.0,17512.80,12690.0
2024-06,49,306191.0,303996.54,220205.0
2024-07,109,623891.0,620161.98,443358.0
2024-08,57,428533.0,444305.14,271022.0
2024-09,8,41644.0,41019.35,30173.0


八派渠道

In [141]:
df_mon_8p = df_fq_concat[(df_fq_concat.当前期数==1) & (df_fq_concat.归属渠道=='八派信息')].groupby(by=['search_time']).agg({'order_number': 'count', 'contract_price': 'sum','total_receivable':'sum','purchase_amount':'sum'})
# df_mon.columns = df_mon.columns.droplevel(1) 
df_mon_8p

,order_number,contract_price,total_receivable,purchase_amount
search_time,,,,
2024-06,14,187153.0,160720.85,100247.0
2024-08,59,747698.2,735582.83,459823.0
2024-09,71,903008.7,893622.21,559383.0
2024-10,35,452904.9,436979.19,263129.0


## vintage 1+ 订单口径

In [142]:
vintage_df_d1 = pd.pivot_table(df_mob_total[df_mob_total.overdue_days_1==1],index='下单月份',columns='MOB',values='overdue_days_1',aggfunc='sum')  #.reset_index()  #,margins=True
# vintage_df_d4 = vintage_df_d4[vintage_df_d4['归属渠道']!='抖音渠道']
vintage_df_d1 = vintage_df_d1[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0',
# vintage_df_d4
vintage_df_d1 = vintage_df_d1.divide(df_mon["order_number"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_df_d1['订单量'] = df_mon["order_number"]
vintage_df_d1[['订单量','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]

MOB,订单量,mob_1,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2022-07,22,9.09%,27.27%,27.27%,36.36%,36.36%,36.36%,36.36%,40.91%,40.91%,40.91%,36.36%,36.36%
2022-08,56,16.07%,23.21%,28.57%,30.36%,33.93%,35.71%,37.50%,41.07%,41.07%,41.07%,41.07%,39.29%
2022-09,44,9.09%,13.64%,20.45%,25.00%,29.55%,29.55%,29.55%,27.27%,29.55%,22.73%,22.73%,22.73%
2022-10,41,7.32%,7.32%,9.76%,14.63%,24.39%,21.95%,19.51%,21.95%,29.27%,21.95%,24.39%,19.51%
2022-11,43,9.30%,11.63%,9.30%,13.95%,16.28%,23.26%,25.58%,18.60%,20.93%,20.93%,18.60%,18.60%
2022-12,66,10.61%,9.09%,18.18%,19.70%,19.70%,21.21%,24.24%,27.27%,27.27%,25.76%,25.76%,24.24%
2023-01,221,5.88%,13.57%,15.84%,16.74%,22.62%,25.34%,27.15%,28.96%,30.77%,31.22%,30.32%,29.41%
2023-02,332,8.43%,12.35%,16.27%,18.37%,20.48%,22.59%,23.80%,25.00%,25.00%,24.40%,23.49%,23.19%
2023-03,316,7.91%,13.61%,18.35%,22.47%,25.63%,28.16%,28.16%,28.80%,30.06%,31.33%,28.80%,28.16%
2023-04,634,4.10%,8.99%,15.14%,17.67%,19.72%,22.87%,23.50%,24.76%,25.87%,26.66%,25.87%,25.24%


## vintage 4+ 订单 口径 

### 总体

In [143]:

vintage_df_d4 = pd.pivot_table(df_mob_total[df_mob_total.overdue_days_4==1],index='下单月份',columns='MOB',values='overdue_days_4',aggfunc='sum')  #.reset_index()  #,margins=True
# vintage_df_d4 = vintage_df_d4[vintage_df_d4['归属渠道']!='抖音渠道']
vintage_df_d4 = vintage_df_d4[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0',
# vintage_df_d4
vintage_divide_d4 = vintage_df_d4.divide(df_mon["order_number"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_d4['订单量'] = df_mon["order_number"]
vintage_divide_d4[['订单量','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]

MOB,订单量,mob_1,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2022-07,22,9.09%,22.73%,27.27%,31.82%,36.36%,36.36%,36.36%,36.36%,40.91%,40.91%,36.36%,36.36%
2022-08,56,14.29%,21.43%,26.79%,28.57%,30.36%,35.71%,37.50%,41.07%,41.07%,41.07%,41.07%,39.29%
2022-09,44,6.82%,11.36%,18.18%,25.00%,29.55%,27.27%,25.00%,27.27%,27.27%,22.73%,22.73%,22.73%
2022-10,41,4.88%,4.88%,9.76%,14.63%,21.95%,19.51%,17.07%,21.95%,24.39%,19.51%,21.95%,19.51%
2022-11,43,6.98%,11.63%,9.30%,13.95%,13.95%,23.26%,23.26%,18.60%,20.93%,20.93%,18.60%,18.60%
2022-12,66,3.03%,7.58%,15.15%,19.70%,19.70%,18.18%,24.24%,25.76%,27.27%,25.76%,25.76%,24.24%
2023-01,221,5.43%,11.31%,14.48%,16.29%,21.27%,25.34%,26.24%,28.51%,29.86%,31.22%,30.32%,29.41%
2023-02,332,8.13%,11.75%,15.96%,17.17%,20.48%,22.29%,23.49%,25.00%,24.40%,24.40%,23.49%,23.19%
2023-03,316,6.65%,12.34%,16.14%,19.94%,23.10%,26.27%,25.95%,27.53%,28.80%,30.70%,28.80%,28.16%
2023-04,634,3.63%,8.20%,13.09%,15.77%,18.30%,21.29%,22.24%,23.82%,25.08%,25.55%,25.87%,25.24%


### 芝麻租物

In [144]:
# 芝麻租物渠道
vintage_df_d4_zm = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_4==1) & (df_mob_total.归属渠道=='芝麻租物')],index='下单月份',columns='MOB',values='overdue_days_4',aggfunc='sum')  #.reset_index()  #,margins=True

vintage_df_d4_zm = vintage_df_d4_zm[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0',

vintage_divide_d4_zm = vintage_df_d4_zm.divide(df_mon_zm["order_number"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_d4_zm['订单量'] = df_mon_zm["order_number"]
vintage_divide_d4_zm[['订单量','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]


MOB,订单量,mob_1,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2022-09,5,nan%,20.00%,20.00%,40.00%,40.00%,40.00%,40.00%,40.00%,40.00%,40.00%,40.00%,40.00%
2022-10,10,10.00%,nan%,10.00%,10.00%,10.00%,20.00%,10.00%,10.00%,10.00%,10.00%,10.00%,10.00%
2022-11,10,nan%,10.00%,10.00%,10.00%,10.00%,10.00%,30.00%,20.00%,20.00%,20.00%,20.00%,20.00%
2022-12,2,nan%,nan%,nan%,nan%,nan%,nan%,50.00%,nan%,nan%,nan%,nan%,nan%
2023-01,18,nan%,5.56%,11.11%,11.11%,11.11%,22.22%,27.78%,22.22%,27.78%,27.78%,27.78%,27.78%
2023-02,38,2.63%,5.26%,7.89%,5.26%,5.26%,10.53%,7.89%,7.89%,5.26%,5.26%,5.26%,5.26%
2023-03,61,1.64%,1.64%,4.92%,6.56%,11.48%,13.11%,14.75%,16.39%,18.03%,19.67%,18.03%,18.03%
2023-04,240,2.92%,6.25%,9.58%,10.83%,11.67%,16.67%,17.08%,18.33%,18.75%,19.58%,19.58%,19.58%
2023-05,145,nan%,4.14%,8.97%,8.97%,10.34%,8.97%,15.17%,17.93%,19.31%,17.93%,20.00%,18.62%
2023-06,263,2.66%,6.84%,11.03%,14.45%,15.97%,18.25%,21.29%,24.33%,24.33%,26.62%,25.86%,25.48%


### 搜索渠道

In [145]:
# 搜索渠道
vintage_df_d4_ss = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_4==1) & (df_mob_total.归属渠道=='搜索渠道')],index='下单月份',columns='MOB',values='overdue_days_4',aggfunc='sum')  #.reset_index()  #,margins=True

vintage_df_d4_ss = vintage_df_d4_ss[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0',

vintage_divide_d4_ss = vintage_df_d4_ss.divide(df_mon_ss["order_number"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_d4_ss['订单量'] = df_mon_ss["order_number"]
vintage_divide_d4_ss[['订单量','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]


MOB,订单量,mob_1,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2022-07,7,14.29%,42.86%,42.86%,42.86%,42.86%,42.86%,42.86%,42.86%,57.14%,42.86%,42.86%,42.86%
2022-08,22,13.64%,18.18%,22.73%,22.73%,22.73%,31.82%,36.36%,45.45%,36.36%,36.36%,36.36%,36.36%
2022-09,8,37.50%,50.00%,62.50%,62.50%,62.50%,62.50%,50.00%,62.50%,62.50%,62.50%,62.50%,62.50%
2022-10,6,nan%,16.67%,16.67%,33.33%,33.33%,16.67%,16.67%,16.67%,16.67%,16.67%,16.67%,16.67%
2022-11,10,30.00%,40.00%,30.00%,40.00%,40.00%,50.00%,40.00%,50.00%,50.00%,50.00%,50.00%,50.00%
2022-12,37,2.70%,10.81%,18.92%,21.62%,24.32%,24.32%,27.03%,32.43%,29.73%,29.73%,29.73%,27.03%
2023-01,134,5.97%,12.69%,16.42%,15.67%,20.15%,23.88%,25.37%,27.61%,27.61%,29.85%,28.36%,26.87%
2023-02,200,10.50%,15.00%,18.50%,20.50%,24.00%,27.00%,29.00%,30.00%,29.50%,29.50%,28.50%,28.50%
2023-03,150,10.67%,18.67%,24.67%,27.33%,30.00%,34.00%,32.67%,34.00%,34.67%,36.67%,34.67%,34.67%
2023-04,191,4.19%,10.99%,16.75%,18.85%,21.47%,21.47%,25.13%,26.70%,28.80%,28.80%,30.89%,29.32%


### 抖音渠道

In [146]:
# 抖音渠道
vintage_df_d4_dy = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_4==1) & (df_mob_total.归属渠道=='抖音渠道')],index='下单月份',columns='MOB',values='overdue_days_4',aggfunc='sum')  #.reset_index()  #,margins=True

# vintage_df_d4_dy = vintage_df_d4_dy[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0',
# vintage_df_d4_dy
vintage_df_d4_dy.divide(df_mon_dy["order_number"],axis=0).applymap(lambda x:format(x,".2%"))


MOB,mob_1,mob_2,mob_3,mob_4,mob_5
2024-04,nan%,nan%,25.00%,25.00%,25.00%
2024-05,nan%,nan%,nan%,nan%,nan%
2024-06,20.41%,26.53%,26.53%,nan%,nan%
2024-07,15.60%,24.77%,nan%,nan%,nan%
2024-08,19.30%,nan%,nan%,nan%,nan%
2024-09,nan%,nan%,nan%,nan%,nan%


八派信息

In [147]:
# 八派信息
vintage_df_d4_8p = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_4==1) & (df_mob_total.归属渠道=='八派信息')],index='下单月份',columns='MOB',values='overdue_days_4',aggfunc='sum')  #.reset_index()  #,margins=True
vintage_df_d4_8p.divide(df_mon_8p["order_number"],axis=0).applymap(lambda x:format(x,".2%"))




MOB,mob_1,mob_2,mob_3
2024-06,7.14%,7.14%,14.29%
2024-08,1.69%,nan%,nan%
2024-09,nan%,nan%,nan%
2024-10,nan%,nan%,nan%


## vintage 7+ 订单 口径 

### 总体

In [148]:
vintage_df_d7 = pd.pivot_table(df_mob_total[df_mob_total.overdue_days_7==1],index='下单月份',columns='MOB',values='overdue_days_7',aggfunc='sum')  #.reset_index()  #,margins=True

vintage_df_d7 = vintage_df_d7[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0',
# vintage_df_d7
vintage_divide_d7 = vintage_df_d7.divide(df_mon["order_number"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_d7['订单量'] = df_mon["order_number"]
vintage_divide_d7[['订单量','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]

MOB,订单量,mob_1,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2022-07,22,4.55%,22.73%,27.27%,31.82%,36.36%,36.36%,36.36%,36.36%,40.91%,40.91%,36.36%,36.36%
2022-08,56,10.71%,21.43%,26.79%,28.57%,30.36%,33.93%,37.50%,41.07%,41.07%,41.07%,41.07%,39.29%
2022-09,44,6.82%,11.36%,18.18%,20.45%,29.55%,27.27%,25.00%,25.00%,27.27%,22.73%,22.73%,22.73%
2022-10,41,4.88%,4.88%,7.32%,14.63%,21.95%,19.51%,17.07%,19.51%,24.39%,19.51%,21.95%,19.51%
2022-11,43,6.98%,11.63%,9.30%,11.63%,11.63%,20.93%,23.26%,18.60%,18.60%,18.60%,18.60%,18.60%
2022-12,66,1.52%,7.58%,13.64%,19.70%,18.18%,18.18%,24.24%,25.76%,27.27%,25.76%,25.76%,24.24%
2023-01,221,4.98%,10.86%,14.48%,16.29%,19.46%,24.89%,24.43%,27.60%,29.86%,30.77%,30.32%,29.41%
2023-02,332,7.53%,10.84%,15.06%,16.87%,19.88%,21.39%,23.19%,24.70%,24.10%,24.10%,23.49%,23.19%
2023-03,316,4.75%,11.08%,14.56%,18.99%,22.15%,24.37%,24.68%,25.63%,28.80%,30.70%,28.80%,28.16%
2023-04,634,3.31%,7.41%,11.99%,15.14%,17.67%,20.82%,21.61%,23.34%,24.61%,25.24%,25.87%,25.24%


### 租物渠道

In [149]:
# 搜索渠道
vintage_df_d7_zm = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_7==1) & (df_mob_total.归属渠道=='芝麻租物')],index='下单月份',columns='MOB',values='overdue_days_7',aggfunc='sum')  #.reset_index()  #,margins=True

vintage_df_d7_zm = vintage_df_d7_zm[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0',

vintage_divide_d7_zm = vintage_df_d7_zm.divide(df_mon_zm["order_number"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_d7_zm['订单量'] = df_mon_zm["order_number"]
vintage_divide_d7_zm[['订单量','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]


MOB,订单量,mob_1,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2022-09,5,nan%,20.00%,20.00%,40.00%,40.00%,40.00%,40.00%,40.00%,40.00%,40.00%,40.00%,40.00%
2022-10,10,10.00%,nan%,10.00%,10.00%,10.00%,20.00%,10.00%,10.00%,10.00%,10.00%,10.00%,10.00%
2022-11,10,nan%,10.00%,10.00%,10.00%,10.00%,10.00%,30.00%,20.00%,20.00%,20.00%,20.00%,20.00%
2022-12,2,nan%,nan%,nan%,nan%,nan%,nan%,50.00%,nan%,nan%,nan%,nan%,nan%
2023-01,18,nan%,nan%,11.11%,11.11%,11.11%,22.22%,22.22%,22.22%,27.78%,27.78%,27.78%,27.78%
2023-02,38,nan%,2.63%,2.63%,2.63%,5.26%,7.89%,5.26%,7.89%,5.26%,5.26%,5.26%,5.26%
2023-03,61,1.64%,1.64%,4.92%,6.56%,11.48%,11.48%,13.11%,16.39%,18.03%,19.67%,18.03%,18.03%
2023-04,240,2.92%,5.83%,8.75%,10.83%,11.67%,16.67%,16.67%,18.33%,18.75%,19.58%,19.58%,19.58%
2023-05,145,nan%,4.14%,8.28%,8.97%,8.97%,8.97%,15.17%,17.24%,19.31%,16.55%,19.31%,18.62%
2023-06,263,1.90%,5.70%,10.27%,12.93%,15.21%,17.49%,21.29%,22.81%,24.33%,26.24%,25.48%,25.48%


### 搜索渠道

In [150]:
# 搜索渠道
vintage_df_d7_ss = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_7==1) & (df_mob_total.归属渠道=='搜索渠道')],index='下单月份',columns='MOB',values='overdue_days_7',aggfunc='sum')  #.reset_index()  #,margins=True

vintage_df_d7_ss = vintage_df_d7_ss[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0',

vintage_divide_d7_ss = vintage_df_d7_ss.divide(df_mon_ss["order_number"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_d7_ss['订单量'] = df_mon_ss["order_number"]
vintage_divide_d7_ss[['订单量','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]


MOB,订单量,mob_1,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2022-07,7,14.29%,42.86%,42.86%,42.86%,42.86%,42.86%,42.86%,42.86%,57.14%,42.86%,42.86%,42.86%
2022-08,22,13.64%,18.18%,22.73%,22.73%,22.73%,31.82%,36.36%,45.45%,36.36%,36.36%,36.36%,36.36%
2022-09,8,37.50%,50.00%,62.50%,62.50%,62.50%,62.50%,50.00%,62.50%,62.50%,62.50%,62.50%,62.50%
2022-10,6,nan%,16.67%,16.67%,33.33%,33.33%,16.67%,16.67%,16.67%,16.67%,16.67%,16.67%,16.67%
2022-11,10,30.00%,40.00%,30.00%,40.00%,40.00%,50.00%,40.00%,50.00%,50.00%,50.00%,50.00%,50.00%
2022-12,37,2.70%,10.81%,18.92%,21.62%,24.32%,24.32%,27.03%,32.43%,29.73%,29.73%,29.73%,27.03%
2023-01,134,5.22%,12.69%,16.42%,15.67%,17.16%,23.13%,23.13%,26.12%,27.61%,29.10%,28.36%,26.87%
2023-02,200,10.00%,14.00%,18.00%,20.50%,23.00%,26.00%,29.00%,30.00%,29.50%,29.00%,28.50%,28.50%
2023-03,150,8.00%,17.33%,22.67%,26.00%,29.33%,32.67%,32.00%,32.00%,34.67%,36.67%,34.67%,34.67%
2023-04,191,3.14%,8.90%,15.71%,17.80%,19.90%,20.42%,23.56%,26.18%,28.80%,28.27%,30.89%,29.32%


## vintage 15+ 订单 口径 

### 总体

In [151]:
vintage_df_d15 = pd.pivot_table(df_mob_total[df_mob_total.overdue_days_15==1],index='下单月份',columns='MOB',values='overdue_days_15',aggfunc='sum')  #.reset_index()  #,margins=True

vintage_df_d15 = vintage_df_d15[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0',
# vintage_df_d15
vintage_divide_d15 = vintage_df_d15.divide(df_mon["order_number"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_d15['订单量'] = df_mon["order_number"]
vintage_divide_d15[['订单量','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]

MOB,订单量,mob_1,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2022-07,22,nan%,18.18%,27.27%,27.27%,27.27%,31.82%,36.36%,36.36%,36.36%,36.36%,36.36%,36.36%
2022-08,56,5.36%,14.29%,19.64%,23.21%,28.57%,33.93%,37.50%,39.29%,39.29%,41.07%,41.07%,39.29%
2022-09,44,2.27%,9.09%,18.18%,18.18%,27.27%,27.27%,25.00%,25.00%,25.00%,22.73%,22.73%,22.73%
2022-10,41,4.88%,4.88%,7.32%,14.63%,19.51%,19.51%,17.07%,19.51%,24.39%,19.51%,21.95%,19.51%
2022-11,43,4.65%,11.63%,9.30%,11.63%,11.63%,18.60%,23.26%,18.60%,18.60%,18.60%,18.60%,18.60%
2022-12,66,nan%,7.58%,9.09%,19.70%,16.67%,16.67%,22.73%,21.21%,27.27%,25.76%,25.76%,24.24%
2023-01,221,4.07%,9.95%,11.76%,15.84%,19.00%,24.43%,23.98%,26.70%,29.41%,28.96%,30.32%,29.41%
2023-02,332,4.82%,9.04%,13.86%,14.16%,18.98%,19.88%,21.39%,23.80%,24.10%,23.80%,23.49%,23.19%
2023-03,316,3.16%,8.86%,12.34%,16.46%,18.04%,22.78%,22.78%,23.73%,27.53%,29.75%,28.80%,28.16%
2023-04,634,2.21%,5.84%,10.09%,13.41%,15.62%,19.40%,20.50%,21.77%,23.66%,23.19%,25.39%,25.24%


### 租物渠道

In [152]:

vintage_df_d15_zm = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_15==1) & (df_mob_total.归属渠道=='芝麻租物')],index='下单月份',columns='MOB',values='overdue_days_15',aggfunc='sum')  #.reset_index()  #,margins=True

vintage_df_d15_zm = vintage_df_d15_zm[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0',

vintage_divide_d15_zm = vintage_df_d15_zm.divide(df_mon_zm["order_number"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_d15_zm['订单量'] = df_mon_zm["order_number"]
vintage_divide_d15_zm[['订单量','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]


MOB,订单量,mob_1,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2022-09,5,nan%,nan%,20.00%,40.00%,40.00%,40.00%,40.00%,40.00%,40.00%,40.00%,40.00%,40.00%
2022-10,10,10.00%,nan%,10.00%,10.00%,10.00%,20.00%,10.00%,10.00%,10.00%,10.00%,10.00%,10.00%
2022-11,10,nan%,10.00%,10.00%,10.00%,10.00%,10.00%,30.00%,20.00%,20.00%,20.00%,20.00%,20.00%
2022-12,2,nan%,nan%,nan%,nan%,nan%,nan%,50.00%,nan%,nan%,nan%,nan%,nan%
2023-01,18,nan%,nan%,11.11%,11.11%,11.11%,22.22%,22.22%,22.22%,27.78%,27.78%,27.78%,27.78%
2023-02,38,nan%,2.63%,2.63%,2.63%,5.26%,7.89%,5.26%,7.89%,5.26%,5.26%,5.26%,5.26%
2023-03,61,nan%,1.64%,3.28%,4.92%,6.56%,11.48%,13.11%,14.75%,16.39%,19.67%,18.03%,18.03%
2023-04,240,2.08%,3.75%,7.08%,9.58%,10.83%,15.00%,15.83%,16.67%,18.75%,18.75%,19.58%,19.58%
2023-05,145,nan%,2.76%,8.28%,8.97%,8.97%,8.28%,15.17%,15.86%,17.24%,15.86%,19.31%,18.62%
2023-06,263,1.14%,5.32%,8.75%,12.17%,14.83%,16.73%,20.53%,22.05%,23.57%,25.10%,25.48%,25.48%


### 搜索渠道

In [153]:
# 搜索渠道
vintage_df_d15_ss = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_15==1) & (df_mob_total.归属渠道=='搜索渠道')],index='下单月份',columns='MOB',values='overdue_days_15',aggfunc='sum')  #.reset_index()  #,margins=True

vintage_df_d15_ss = vintage_df_d15_ss[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0',

vintage_divide_d15_ss = vintage_df_d15_ss.divide(df_mon_ss["order_number"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_d15_ss['订单量'] = df_mon_ss["order_number"]
vintage_divide_d15_ss[['订单量','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]


MOB,订单量,mob_1,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2022-07,7,nan%,42.86%,42.86%,42.86%,42.86%,42.86%,42.86%,42.86%,42.86%,42.86%,42.86%,42.86%
2022-08,22,9.09%,9.09%,13.64%,13.64%,22.73%,31.82%,36.36%,40.91%,36.36%,36.36%,36.36%,36.36%
2022-09,8,12.50%,50.00%,62.50%,62.50%,62.50%,62.50%,50.00%,62.50%,62.50%,62.50%,62.50%,62.50%
2022-10,6,nan%,16.67%,16.67%,33.33%,33.33%,16.67%,16.67%,16.67%,16.67%,16.67%,16.67%,16.67%
2022-11,10,20.00%,40.00%,30.00%,40.00%,40.00%,50.00%,40.00%,50.00%,50.00%,50.00%,50.00%,50.00%
2022-12,37,nan%,10.81%,13.51%,21.62%,21.62%,21.62%,24.32%,24.32%,29.73%,29.73%,29.73%,27.03%
2023-01,134,3.73%,11.19%,11.94%,15.67%,16.42%,22.39%,22.39%,24.63%,26.87%,26.12%,28.36%,26.87%
2023-02,200,5.50%,12.00%,17.00%,17.00%,22.50%,24.00%,27.00%,29.50%,29.50%,28.50%,28.50%,28.50%
2023-03,150,6.00%,13.33%,18.67%,24.00%,24.67%,29.33%,28.67%,29.33%,34.00%,36.00%,34.67%,34.67%
2023-04,191,2.09%,7.85%,12.04%,14.66%,16.23%,19.37%,21.99%,23.56%,26.70%,24.08%,29.32%,29.32%


## vintage 30+ 订单口径

In [154]:
vintage_df_d30 = pd.pivot_table(df_mob_total[df_mob_total.overdue_days_30==1],index='下单月份',columns='MOB',values='overdue_days_30',aggfunc='sum')  #.reset_index()  #,margins=True

vintage_df_d30 = vintage_df_d30[['mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0',
# vintage_df_d31
vintage_df_d30 = vintage_df_d30.divide(df_mon["order_number"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_df_d30['订单量'] = df_mon["order_number"]
vintage_df_d30[['订单量','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'mob_1',

MOB,订单量,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2022-07,22,18.18%,27.27%,27.27%,27.27%,31.82%,36.36%,36.36%,36.36%,36.36%,36.36%,36.36%
2022-08,56,12.50%,17.86%,21.43%,26.79%,30.36%,32.14%,37.50%,35.71%,41.07%,39.29%,39.29%
2022-09,44,6.82%,11.36%,15.91%,25.00%,25.00%,25.00%,22.73%,25.00%,22.73%,22.73%,22.73%
2022-10,41,4.88%,4.88%,9.76%,14.63%,17.07%,12.20%,17.07%,17.07%,19.51%,19.51%,19.51%
2022-11,43,9.30%,9.30%,11.63%,11.63%,13.95%,16.28%,18.60%,18.60%,18.60%,18.60%,18.60%
2022-12,66,4.55%,7.58%,16.67%,16.67%,16.67%,21.21%,21.21%,25.76%,25.76%,25.76%,24.24%
2023-01,221,5.43%,9.50%,13.12%,15.38%,20.81%,20.81%,23.53%,27.15%,27.60%,29.86%,29.41%
2023-02,332,6.63%,9.94%,12.95%,16.57%,17.77%,19.58%,21.99%,22.59%,23.19%,23.19%,23.19%
2023-03,316,6.65%,10.44%,14.87%,17.41%,20.57%,21.52%,23.10%,27.22%,28.48%,27.85%,28.16%
2023-04,634,3.15%,6.62%,10.25%,12.93%,15.77%,18.61%,19.72%,22.40%,22.08%,23.97%,25.24%


## vintage 31+ 订单 口径 

### 总体

In [155]:
vintage_df_d31 = pd.pivot_table(df_mob_total[df_mob_total.overdue_days_31==1],index='下单月份',columns='MOB',values='overdue_days_31',aggfunc='sum')  #.reset_index()  #,margins=True

vintage_df_d31 = vintage_df_d31[['mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0',
# vintage_df_d31
vintage_divide_d31 = vintage_df_d31.divide(df_mon["order_number"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_d31['订单量'] = df_mon["order_number"]
vintage_divide_d31[['订单量','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'mob_1',

MOB,订单量,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2022-07,22,18.18%,27.27%,27.27%,27.27%,31.82%,36.36%,36.36%,36.36%,36.36%,36.36%,36.36%
2022-08,56,12.50%,17.86%,21.43%,26.79%,28.57%,32.14%,37.50%,35.71%,41.07%,39.29%,39.29%
2022-09,44,6.82%,11.36%,15.91%,25.00%,25.00%,25.00%,22.73%,25.00%,22.73%,22.73%,22.73%
2022-10,41,4.88%,4.88%,9.76%,14.63%,17.07%,12.20%,17.07%,17.07%,19.51%,19.51%,19.51%
2022-11,43,9.30%,9.30%,11.63%,11.63%,13.95%,16.28%,18.60%,18.60%,18.60%,18.60%,18.60%
2022-12,66,4.55%,6.06%,16.67%,16.67%,16.67%,21.21%,21.21%,25.76%,25.76%,25.76%,24.24%
2023-01,221,5.43%,9.50%,12.22%,15.38%,20.81%,20.36%,23.53%,27.15%,27.60%,29.86%,29.41%
2023-02,332,6.63%,9.94%,12.95%,16.57%,17.17%,19.58%,21.99%,22.59%,23.19%,23.19%,23.19%
2023-03,316,6.65%,10.44%,14.87%,17.41%,20.57%,21.20%,23.10%,26.58%,28.16%,27.85%,28.16%
2023-04,634,3.15%,6.47%,10.25%,12.93%,15.77%,18.61%,19.72%,22.24%,22.08%,23.97%,25.24%


### 租物渠道

In [156]:

vintage_df_d31_zm = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_31==1) & (df_mob_total.归属渠道=='芝麻租物')],index='下单月份',columns='MOB',values='overdue_days_31',aggfunc='sum')  #.reset_index()  #,margins=True

vintage_df_d31_zm = vintage_df_d31_zm[['mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0',

vintage_divide_d31_zm = vintage_df_d31_zm.divide(df_mon_zm["order_number"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_d31_zm['订单量'] = df_mon_zm["order_number"]
vintage_divide_d31_zm[['订单量','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']] # ,'mob_1'


MOB,订单量,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2022-09,5,nan%,20.00%,20.00%,40.00%,40.00%,40.00%,40.00%,40.00%,40.00%,40.00%,40.00%
2022-10,10,nan%,nan%,10.00%,10.00%,10.00%,nan%,10.00%,nan%,10.00%,10.00%,10.00%
2022-11,10,nan%,10.00%,10.00%,10.00%,10.00%,10.00%,20.00%,20.00%,20.00%,20.00%,20.00%
2022-12,2,nan%,nan%,nan%,nan%,nan%,50.00%,nan%,nan%,nan%,nan%,nan%
2023-01,18,nan%,11.11%,11.11%,11.11%,22.22%,22.22%,22.22%,27.78%,27.78%,27.78%,27.78%
2023-02,38,2.63%,2.63%,2.63%,5.26%,5.26%,5.26%,5.26%,5.26%,5.26%,5.26%,5.26%
2023-03,61,1.64%,1.64%,3.28%,6.56%,9.84%,11.48%,14.75%,14.75%,18.03%,18.03%,18.03%
2023-04,240,2.50%,5.00%,7.92%,9.58%,11.67%,13.75%,15.42%,17.92%,17.50%,18.75%,19.58%
2023-05,145,nan%,3.45%,6.90%,7.59%,8.28%,11.03%,12.41%,15.86%,15.86%,17.24%,18.62%
2023-06,263,2.28%,6.08%,9.89%,12.17%,14.83%,17.49%,19.77%,22.05%,23.57%,25.10%,25.48%


### 搜索渠道

In [157]:
# 搜索渠道
vintage_df_d31_ss = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_31==1) & (df_mob_total.归属渠道=='搜索渠道')],index='下单月份',columns='MOB',values='overdue_days_31',aggfunc='sum')  #.reset_index()  #,margins=True

vintage_df_d31_ss = vintage_df_d31_ss[['mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0','mob_1',

vintage_divide_d31_ss = vintage_df_d31_ss.divide(df_mon_ss["order_number"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_d31_ss['订单量'] = df_mon_ss["order_number"]
vintage_divide_d31_ss[['订单量','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]


MOB,订单量,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2022-07,7,42.86%,42.86%,42.86%,42.86%,42.86%,42.86%,42.86%,42.86%,42.86%,42.86%,42.86%
2022-08,22,9.09%,13.64%,13.64%,22.73%,22.73%,31.82%,36.36%,31.82%,36.36%,36.36%,36.36%
2022-09,8,37.50%,50.00%,62.50%,62.50%,50.00%,50.00%,50.00%,62.50%,62.50%,62.50%,62.50%
2022-10,6,16.67%,16.67%,16.67%,33.33%,16.67%,16.67%,16.67%,16.67%,16.67%,16.67%,16.67%
2022-11,10,40.00%,30.00%,40.00%,40.00%,40.00%,40.00%,50.00%,50.00%,50.00%,50.00%,50.00%
2022-12,37,5.41%,8.11%,18.92%,21.62%,21.62%,24.32%,24.32%,29.73%,29.73%,29.73%,27.03%
2023-01,134,5.22%,8.96%,12.69%,13.43%,18.66%,19.40%,22.39%,24.63%,24.63%,27.61%,26.87%
2023-02,200,8.50%,12.00%,15.50%,20.00%,20.50%,24.50%,27.00%,27.50%,28.00%,28.00%,28.50%
2023-03,150,10.67%,16.67%,23.33%,24.00%,27.33%,26.67%,28.67%,33.33%,34.67%,33.33%,34.67%
2023-04,191,3.66%,7.33%,12.57%,15.18%,16.23%,19.37%,20.94%,25.13%,24.08%,27.23%,29.32%


## vintage 90+ 订单口径

In [158]:
vintage_df_d90 = pd.pivot_table(df_mob_total[df_mob_total.overdue_days_90==1],index='下单月份',columns='MOB',values='overdue_days_90',aggfunc='sum')  #.reset_index()  #,margins=True

vintage_df_d90 = vintage_df_d90[['mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0',
# vintage_df_d31
vintage_df_d90 = vintage_df_d90.divide(df_mon["order_number"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_df_d90['订单量'] = df_mon["order_number"]
vintage_df_d90[['订单量','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'mob_1',

MOB,订单量,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2022-07,22,nan%,18.18%,27.27%,27.27%,27.27%,31.82%,36.36%,36.36%,36.36%,36.36%
2022-08,56,nan%,12.50%,17.86%,21.43%,25.00%,28.57%,32.14%,33.93%,33.93%,39.29%
2022-09,44,nan%,6.82%,11.36%,13.64%,22.73%,22.73%,22.73%,20.45%,22.73%,22.73%
2022-10,41,nan%,4.88%,4.88%,9.76%,7.32%,12.20%,12.20%,14.63%,17.07%,17.07%
2022-11,43,nan%,6.98%,9.30%,11.63%,9.30%,9.30%,11.63%,13.95%,18.60%,18.60%
2022-12,66,nan%,4.55%,6.06%,13.64%,15.15%,16.67%,18.18%,19.70%,24.24%,24.24%
2023-01,221,nan%,3.62%,7.69%,12.22%,13.57%,17.65%,19.46%,22.17%,25.34%,27.60%
2023-02,332,0.30%,5.42%,10.24%,12.95%,15.66%,16.57%,18.07%,19.28%,20.18%,21.39%
2023-03,316,nan%,6.65%,10.76%,13.61%,15.82%,18.35%,19.30%,22.47%,25.00%,26.58%
2023-04,634,nan%,3.00%,5.99%,9.15%,11.04%,14.51%,17.51%,18.77%,20.03%,21.45%


## 金额 vintage 1+  口径 

In [159]:
df_mob_total['unpaid_money_mob'] = df_mob_total['purchase_amount'] - df_mob_total['paid_money_mob']

In [160]:
vintage_money_d1 =  pd.pivot_table(df_mob_total[df_mob_total.overdue_days_1==1],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum')  #.reset_index()  #,margins=True

vintage_money_d1 = vintage_money_d1[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0',

vintage_divide_money_d1 = vintage_money_d1.divide(df_mon["purchase_amount"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_money_d1['采购金额'] = df_mon["purchase_amount"]
vintage_divide_money_d1[['采购金额','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]

KeyError: "['mob_13', 'mob_14', 'mob_15'] not in index"

## 金额 vintage 4+  口径 

In [ ]:
# df_mob_total.columns

In [ ]:
# df_mob_total['unpaid_money_mob'] = df_mob_total['purchase_amount'] - df_mob_total['paid_money_mob']

### 总体

In [ ]:
vintage_money_d4 =  pd.pivot_table(df_mob_total[df_mob_total.overdue_days_4==1],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum')  #.reset_index()  #,margins=True

vintage_money_d4 = vintage_money_d4[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0',
# vintage_money_d4['采购金额'] = df_mon['采购金额']
# vintage_money_d4
vintage_divide_money_d4 = vintage_money_d4.divide(df_mon["purchase_amount"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_money_d4['采购金额'] = df_mon["purchase_amount"]
vintage_divide_money_d4[['采购金额','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]

MOB,采购金额,mob_1,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2022-07,125405.00,9.96%,27.27%,32.59%,34.52%,36.02%,35.77%,35.77%,35.77%,36.70%,33.76%,29.57%,29.57%
2022-08,358451.00,16.55%,25.37%,31.30%,31.87%,33.05%,35.44%,34.51%,35.92%,34.03%,30.79%,27.46%,26.76%
2022-09,277982.75,9.02%,13.86%,17.67%,22.33%,26.23%,24.21%,23.12%,22.98%,22.14%,17.88%,15.98%,15.98%
2022-10,249543.09,4.64%,5.86%,8.64%,13.66%,20.66%,17.70%,15.88%,18.33%,21.14%,16.99%,17.03%,15.04%
2022-11,344258.81,8.23%,11.76%,9.96%,14.05%,12.59%,19.69%,18.74%,15.33%,15.93%,14.96%,12.12%,12.12%
2022-12,566276.59,3.12%,7.69%,12.62%,17.67%,16.18%,15.47%,19.37%,19.45%,19.83%,17.56%,15.50%,13.89%
2023-01,1533459.00,5.21%,10.18%,11.82%,13.05%,16.01%,18.09%,17.41%,17.96%,17.77%,16.52%,14.11%,13.97%
2023-02,2181552.82,8.92%,13.29%,16.46%,17.30%,19.79%,20.32%,20.31%,20.66%,19.11%,18.18%,15.81%,15.55%
2023-03,2083021.00,7.50%,12.46%,15.20%,18.87%,20.33%,22.68%,21.57%,20.67%,20.80%,20.24%,17.81%,17.56%
2023-04,4317651.50,3.76%,8.09%,12.37%,14.29%,15.98%,17.53%,17.94%,18.30%,18.30%,17.77%,16.03%,15.77%


### 租物渠道

In [ ]:

vintage_money_d4_zm = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_4==1) & (df_mob_total.归属渠道=='芝麻租物')],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum')  #.reset_index()  #,margins=True

vintage_money_d4_zm = vintage_money_d4_zm[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0','mob_1',

vintage_divide_money_d4_zm = vintage_money_d4_zm.divide(df_mon_zm["purchase_amount"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_money_d4_zm['采购金额'] = df_mon_zm["purchase_amount"]
vintage_divide_money_d4_zm[['采购金额','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]


MOB,采购金额,mob_1,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2022-09,24742.68,nan%,24.49%,24.49%,29.32%,29.32%,29.32%,29.32%,29.32%,29.32%,29.32%,26.95%,26.95%
2022-10,48835.00,6.57%,nan%,5.58%,5.58%,5.58%,17.43%,11.05%,11.05%,9.46%,9.46%,7.74%,7.74%
2022-11,68730.22,nan%,8.67%,8.67%,8.67%,8.67%,8.67%,18.06%,16.86%,15.91%,15.80%,14.10%,14.10%
2022-12,7657.00,nan%,nan%,nan%,nan%,nan%,nan%,49.54%,nan%,nan%,nan%,nan%,nan%
2023-01,110070.00,nan%,7.46%,3.90%,3.90%,3.90%,11.76%,14.42%,11.76%,12.29%,11.66%,9.53%,9.53%
2023-02,206619.00,1.87%,5.50%,7.13%,5.24%,5.37%,7.67%,6.46%,7.13%,5.37%,5.37%,5.07%,5.07%
2023-03,374912.00,1.39%,1.39%,3.66%,4.42%,9.44%,9.89%,11.21%,12.00%,12.40%,13.37%,12.00%,12.00%
2023-04,1628122.00,3.16%,6.25%,8.90%,9.98%,10.79%,14.34%,14.71%,15.34%,15.52%,15.54%,14.30%,14.24%
2023-05,905764.00,nan%,3.93%,7.87%,7.83%,8.69%,7.89%,12.07%,13.42%,14.05%,12.78%,12.71%,12.15%
2023-06,1710725.00,3.20%,6.89%,10.98%,13.40%,14.62%,16.14%,17.84%,19.57%,19.17%,19.99%,18.13%,17.89%


### 搜索渠道

In [ ]:

vintage_money_d4_ss = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_4==1) & (df_mob_total.归属渠道=='搜索渠道')],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum')  #.reset_index()  #,margins=True

vintage_money_d4_ss = vintage_money_d4_ss[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0','mob_1',

vintage_divide_money_d4_ss = vintage_money_d4_ss.divide(df_mon_ss["purchase_amount"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_money_d4_ss['采购金额'] = df_mon_ss["purchase_amount"]
vintage_divide_money_d4_ss[['采购金额','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]


MOB,采购金额,mob_1,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2022-07,46581.00,9.59%,40.92%,40.92%,40.92%,40.92%,40.92%,40.92%,40.92%,43.42%,38.74%,33.82%,33.82%
2022-08,145114.00,14.92%,20.33%,25.37%,24.74%,24.74%,28.53%,30.37%,34.51%,27.74%,24.82%,21.85%,21.85%
2022-09,67126.34,37.34%,48.35%,54.03%,54.03%,53.64%,52.86%,48.35%,52.07%,51.13%,49.44%,44.70%,44.70%
2022-10,44442.63,nan%,14.09%,14.09%,29.29%,29.29%,14.09%,14.09%,14.09%,14.09%,13.07%,11.88%,11.88%
2022-11,88153.59,32.13%,39.16%,32.13%,38.13%,38.13%,42.44%,36.43%,40.38%,39.00%,35.90%,30.87%,30.87%
2022-12,324164.59,2.70%,10.69%,15.08%,18.90%,19.41%,20.50%,22.14%,23.64%,21.81%,20.40%,18.25%,15.43%
2023-01,944943.00,5.66%,10.83%,13.43%,12.61%,15.16%,16.94%,16.82%,17.23%,16.64%,15.20%,13.11%,12.89%
2023-02,1369414.82,11.35%,16.40%,18.65%,20.33%,22.87%,24.47%,24.87%,24.55%,23.02%,21.58%,18.64%,18.50%
2023-03,1002057.00,12.23%,19.15%,24.12%,27.42%,27.63%,30.59%,28.41%,26.66%,26.05%,25.23%,22.25%,21.99%
2023-04,1275193.00,4.29%,11.11%,16.07%,16.84%,18.39%,17.32%,19.71%,19.20%,19.60%,18.97%,17.11%,16.57%


### 抖音渠道

In [ ]:

# vintage_money_d4_dy = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_4==1) & (df_mob_total.归属渠道=='抖音渠道')],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum')  #.reset_index()  #,margins=True

# vintage_money_d4_dy = vintage_money_d4_dy[['mob_1','mob_3']]  #'下单月份','mob_0','mob_1',,'mob_2',

# vintage_divide_money_d4_dy = vintage_money_d4_dy.divide(df_mon_dy["purchase_amount"],axis=0).applymap(lambda x:format(x,".2%"))
# vintage_divide_money_d4_dy['采购金额'] = df_mon_dy["purchase_amount"]
# vintage_divide_money_d4_dy[['采购金额','mob_1','mob_3']]  #,'mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12'


八派信息

In [ ]:
vintage_money_d4_8p= pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_4==1) & (df_mob_total.归属渠道=='八派信息')],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum')
vintage_divide_money_d4_8p = vintage_money_d4_8p.divide(df_mon_8p["purchase_amount"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_money_d4_8p['采购金额'] = df_mon_8p["purchase_amount"]
vintage_divide_money_d4_8p[['采购金额','mob_1', 'mob_2', 'mob_3']]

MOB,采购金额,mob_1,mob_2,mob_3
2024-06,100247.0,6.76%,6.76%,12.76%
2024-08,459823.0,1.53%,nan%,nan%
2024-09,559383.0,nan%,nan%,nan%
2024-10,263129.0,nan%,nan%,nan%


## 金额vintage 7+ 历史最大逾期天数 口径 

### 总体

In [ ]:
vintage_money_d7 = pd.pivot_table(df_mob_total[df_mob_total.overdue_days_7==1],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum')  #.reset_index()  #,margins=True

vintage_money_d7 = vintage_money_d7[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0',
# vintage_money_d7
vintage_divide_money_d7 = vintage_money_d7.divide(df_mon["purchase_amount"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_money_d7['采购金额'] = df_mon["purchase_amount"]
vintage_divide_money_d7[['采购金额','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]

MOB,采购金额,mob_1,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2022-07,125405.00,3.56%,27.27%,32.59%,34.52%,36.02%,35.77%,35.77%,35.77%,36.70%,33.76%,29.57%,29.57%
2022-08,358451.00,12.55%,25.37%,31.30%,31.87%,33.05%,33.94%,34.51%,35.92%,34.03%,30.79%,27.46%,26.76%
2022-09,277982.75,9.02%,13.86%,17.67%,19.55%,26.23%,24.21%,23.12%,21.79%,22.14%,17.88%,15.98%,15.98%
2022-10,249543.09,4.64%,5.86%,6.79%,13.66%,20.66%,17.70%,15.88%,17.74%,21.14%,16.99%,17.03%,15.04%
2022-11,344258.81,8.23%,11.76%,9.96%,11.50%,11.50%,17.40%,18.74%,15.33%,14.79%,13.97%,12.12%,12.12%
2022-12,566276.59,1.55%,7.69%,11.27%,17.67%,14.94%,15.47%,19.37%,19.45%,19.83%,17.56%,15.50%,13.89%
2023-01,1533459.00,4.73%,9.65%,11.82%,13.05%,14.71%,17.91%,16.52%,17.61%,17.77%,16.59%,14.11%,13.97%
2023-02,2181552.82,8.47%,12.42%,15.86%,17.16%,19.34%,19.75%,20.21%,20.46%,18.94%,18.04%,15.81%,15.55%
2023-03,2083021.00,5.50%,11.58%,13.68%,18.15%,19.75%,21.24%,20.65%,20.22%,20.80%,20.24%,17.81%,17.56%
2023-04,4317651.50,3.39%,7.28%,11.50%,13.76%,15.53%,17.30%,17.51%,18.04%,18.17%,17.64%,16.03%,15.77%


In [ ]:
df_mobv = df_mob_total[df_mob_total.overdue_days_7==1]
date_list = ['2023-07','2023-08']
df_mobv2 = df_mobv[df_mobv['下单月份'].isin(date_list)]
df_mobv2.shape

(4387, 17)

In [ ]:
df_mobv.head()

,order_number,下单日期,当前应付租金,paid_money_mob,overdue_days_mob,MOB,overdue_days_1,overdue_days_4,overdue_days_7,overdue_days_15,overdue_days_31,overdue_days_30,overdue_days_90,下单月份,purchase_amount,归属渠道,unpaid_money_mob
20309,A202207201828125,2022-07-20,1004.38,324.95,11,mob_1,1,1,1,0,0,0,0,2022-07,4794.0,搜索渠道,4469.05
20336,A2022081016253710,2022-08-10,759.28,245.65,20,mob_1,1,1,1,1,0,0,0,2022-08,8040.0,搜索渠道,7794.35
20339,A202208110950253,2022-08-11,883.38,285.80,19,mob_1,1,1,1,1,0,0,0,2022-08,9070.0,其他渠道场景渠道。,8784.20
20345,A202208151308171,2022-08-15,880.90,285.00,15,mob_1,1,1,1,1,0,0,0,2022-08,9050.0,搜索渠道,8765.00
20348,A202208161424009,2022-08-16,1239.14,619.57,14,mob_1,1,1,1,0,0,0,0,2022-08,7370.0,我的小程序入口,6750.43


In [ ]:
# df_mobv2.to_excel('E:/myfile/dfmobv2.xlsx')



In [ ]:
# df_mobv2.to_excel('E:/myfile/dfmobv2.xlsx')

### 租物渠道

In [ ]:

vintage_money_d7_zm = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_7==1) & (df_mob_total.归属渠道=='芝麻租物')],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum')  #.reset_index()  #,margins=True

vintage_money_d7_zm = vintage_money_d7_zm[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0','mob_1',

vintage_divide_money_d7_zm = vintage_money_d7_zm.divide(df_mon_zm["purchase_amount"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_money_d7_zm['采购金额'] = df_mon_zm["purchase_amount"]
vintage_divide_money_d7_zm[['采购金额','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]


MOB,采购金额,mob_1,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2022-09,24742.68,nan%,24.49%,24.49%,29.32%,29.32%,29.32%,29.32%,29.32%,29.32%,29.32%,26.95%,26.95%
2022-10,48835.00,6.57%,nan%,5.58%,5.58%,5.58%,17.43%,11.05%,11.05%,9.46%,9.46%,7.74%,7.74%
2022-11,68730.22,nan%,8.67%,8.67%,8.67%,8.67%,8.67%,18.06%,16.86%,15.91%,15.80%,14.10%,14.10%
2022-12,7657.00,nan%,nan%,nan%,nan%,nan%,nan%,49.54%,nan%,nan%,nan%,nan%,nan%
2023-01,110070.00,nan%,nan%,3.90%,3.90%,3.90%,11.76%,11.76%,11.76%,12.29%,11.66%,9.53%,9.53%
2023-02,206619.00,nan%,3.76%,3.76%,3.76%,5.37%,6.45%,5.37%,7.13%,5.37%,5.37%,5.07%,5.07%
2023-03,374912.00,1.39%,1.39%,3.66%,4.42%,9.44%,8.22%,9.63%,12.00%,12.40%,13.37%,12.00%,12.00%
2023-04,1628122.00,3.16%,5.80%,8.23%,9.98%,10.79%,14.34%,14.37%,15.34%,15.52%,15.54%,14.30%,14.24%
2023-05,905764.00,nan%,3.93%,7.81%,7.83%,7.76%,7.89%,12.07%,12.95%,14.05%,12.44%,12.52%,12.15%
2023-06,1710725.00,2.22%,5.61%,10.19%,12.45%,14.02%,15.53%,17.84%,18.51%,19.17%,19.81%,17.98%,17.89%


### 搜索渠道

In [ ]:

vintage_money_d7_ss = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_7==1) & (df_mob_total.归属渠道=='搜索渠道')],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum')  #.reset_index()  #,margins=True

vintage_money_d7_ss = vintage_money_d7_ss[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0','mob_1',

vintage_divide_money_d7_ss = vintage_money_d7_ss.divide(df_mon_ss["purchase_amount"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_money_d7_ss['采购金额'] = df_mon_ss["purchase_amount"]
vintage_divide_money_d7_ss[['采购金额','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]


MOB,采购金额,mob_1,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2022-07,46581.00,9.59%,40.92%,40.92%,40.92%,40.92%,40.92%,40.92%,40.92%,43.42%,38.74%,33.82%,33.82%
2022-08,145114.00,14.92%,20.33%,25.37%,24.74%,24.74%,28.53%,30.37%,34.51%,27.74%,24.82%,21.85%,21.85%
2022-09,67126.34,37.34%,48.35%,54.03%,54.03%,53.64%,52.86%,48.35%,52.07%,51.13%,49.44%,44.70%,44.70%
2022-10,44442.63,nan%,14.09%,14.09%,29.29%,29.29%,14.09%,14.09%,14.09%,14.09%,13.07%,11.88%,11.88%
2022-11,88153.59,32.13%,39.16%,32.13%,38.13%,38.13%,42.44%,36.43%,40.38%,39.00%,35.90%,30.87%,30.87%
2022-12,324164.59,2.70%,10.69%,15.08%,18.90%,19.41%,20.50%,22.14%,23.64%,21.81%,20.40%,18.25%,15.43%
2023-01,944943.00,4.87%,10.83%,13.43%,12.61%,13.05%,16.65%,15.68%,16.67%,16.64%,15.32%,13.11%,12.89%
2023-02,1369414.82,10.92%,15.28%,18.20%,20.33%,22.15%,23.75%,24.87%,24.55%,23.02%,21.37%,18.64%,18.50%
2023-03,1002057.00,9.22%,18.37%,22.38%,26.06%,27.08%,29.83%,27.94%,26.13%,26.05%,25.23%,22.25%,21.99%
2023-04,1275193.00,3.04%,8.93%,15.09%,15.85%,17.31%,16.74%,18.70%,18.92%,19.60%,18.75%,17.11%,16.57%


In [ ]:
# # 抖音渠道  liudan
# vintage_money_d7_dy = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_7==1) & (df_mob_total.归属渠道=='抖音渠道')],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum')  #.reset_index()  #,margins=True

# vintage_money_d7_dy = vintage_money_d7_dy[['mob_1','mob_3']]  #'下单月份','mob_0','mob_1',,'mob_2',

# vintage_divide_money_d7_dy = vintage_money_d7_dy.divide(df_mon_dy["purchase_amount"],axis=0).applymap(lambda x:format(x,".2%"))
# vintage_divide_money_d7_dy['采购金额'] = df_mon_dy["purchase_amount"]
# vintage_divide_money_d7_dy[['采购金额','mob_1','mob_3']]  #,'mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12'

## 金额vintage 15+ 历史最大逾期天数 口径 

### 总体

In [ ]:
vintage_money_d15 = pd.pivot_table(df_mob_total[df_mob_total.overdue_days_15==1],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum')  #.reset_index()  #,margins=True

vintage_money_d15 = vintage_money_d15[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份',
# vintage_money_d15
vintage_divide_money_d15 = vintage_money_d15.divide(df_mon["purchase_amount"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_money_d15['采购金额'] = df_mon["purchase_amount"]
vintage_divide_money_d15[['采购金额','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]

MOB,采购金额,mob_1,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2022-07,125405.00,nan%,21.60%,32.59%,32.59%,32.59%,34.34%,35.77%,35.77%,35.77%,33.72%,29.57%,29.57%
2022-08,358451.00,7.07%,17.39%,23.88%,26.96%,31.27%,33.94%,34.51%,34.74%,32.95%,30.79%,27.46%,26.76%
2022-09,277982.75,2.99%,11.68%,17.67%,17.29%,24.06%,24.21%,23.12%,21.79%,21.01%,17.88%,15.98%,15.98%
2022-10,249543.09,4.64%,5.86%,6.79%,13.66%,18.55%,17.70%,15.88%,17.74%,21.14%,16.99%,17.03%,15.04%
2022-11,344258.81,5.56%,11.76%,9.96%,11.50%,11.50%,15.51%,18.74%,15.33%,14.79%,13.97%,12.12%,12.12%
2022-12,566276.59,nan%,7.69%,8.54%,17.67%,14.64%,14.56%,18.43%,17.55%,19.83%,17.56%,15.50%,13.89%
2023-01,1533459.00,3.94%,8.89%,9.81%,12.69%,14.36%,17.61%,16.22%,17.33%,17.57%,16.37%,14.11%,13.97%
2023-02,2181552.82,5.43%,10.18%,14.82%,14.83%,18.70%,18.68%,19.22%,19.99%,18.94%,17.92%,15.81%,15.55%
2023-03,2083021.00,3.93%,9.67%,11.94%,15.67%,17.17%,20.09%,19.62%,19.55%,20.06%,20.01%,17.81%,17.56%
2023-04,4317651.50,2.30%,5.86%,9.97%,12.37%,14.09%,16.31%,16.81%,17.15%,17.90%,16.92%,16.05%,15.77%


### 租物渠道

In [ ]:

vintage_money_d15_zm = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_15==1) & (df_mob_total.归属渠道=='芝麻租物')],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum')  #.reset_index()  #,margins=True

vintage_money_d15_zm = vintage_money_d15_zm[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0','mob_1',

vintage_divide_money_d15_zm = vintage_money_d15_zm.divide(df_mon_zm["purchase_amount"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_money_d15_zm['采购金额'] = df_mon_zm["purchase_amount"]
vintage_divide_money_d15_zm[['采购金额','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]


MOB,采购金额,mob_1,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2022-09,24742.68,nan%,nan%,24.49%,29.32%,29.32%,29.32%,29.32%,29.32%,29.32%,29.32%,26.95%,26.95%
2022-10,48835.00,6.57%,nan%,5.58%,5.58%,5.58%,17.43%,11.05%,11.05%,9.46%,9.46%,7.74%,7.74%
2022-11,68730.22,nan%,8.67%,8.67%,8.67%,8.67%,8.67%,18.06%,16.86%,15.91%,15.80%,14.10%,14.10%
2022-12,7657.00,nan%,nan%,nan%,nan%,nan%,nan%,49.54%,nan%,nan%,nan%,nan%,nan%
2023-01,110070.00,nan%,nan%,3.90%,3.90%,3.90%,11.76%,11.76%,11.76%,12.29%,11.66%,9.53%,9.53%
2023-02,206619.00,nan%,3.76%,3.76%,3.76%,5.37%,6.45%,5.37%,7.13%,5.37%,5.37%,5.07%,5.07%
2023-03,374912.00,nan%,1.39%,2.46%,3.34%,4.96%,8.22%,9.63%,11.21%,11.72%,13.37%,12.00%,12.00%
2023-04,1628122.00,2.26%,4.10%,7.14%,8.78%,10.14%,13.03%,13.74%,14.14%,15.52%,15.15%,14.30%,14.24%
2023-05,905764.00,nan%,2.62%,7.81%,7.83%,7.76%,7.64%,12.07%,12.35%,13.16%,12.15%,12.52%,12.15%
2023-06,1710725.00,1.36%,5.25%,8.77%,11.67%,13.68%,15.14%,17.31%,18.04%,18.83%,19.15%,17.98%,17.89%


### 搜索渠道

In [ ]:

vintage_money_d15_ss = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_15==1) & (df_mob_total.归属渠道=='搜索渠道')],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum')  #.reset_index()  #,margins=True

vintage_money_d15_ss = vintage_money_d15_ss[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0','mob_1',

vintage_divide_money_d15_ss = vintage_money_d15_ss.divide(df_mon_ss["purchase_amount"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_money_d15_ss['采购金额'] = df_mon_ss["purchase_amount"]
vintage_divide_money_d15_ss[['采购金额','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]


MOB,采购金额,mob_1,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2022-07,46581.00,nan%,40.92%,40.92%,40.92%,40.92%,40.92%,40.92%,40.92%,40.92%,38.74%,33.82%,33.82%
2022-08,145114.00,11.41%,11.41%,17.03%,17.03%,24.74%,28.53%,30.37%,31.60%,27.74%,24.82%,21.85%,21.85%
2022-09,67126.34,12.40%,48.35%,54.03%,54.03%,53.64%,52.86%,48.35%,52.07%,51.13%,49.44%,44.70%,44.70%
2022-10,44442.63,nan%,14.09%,14.09%,29.29%,29.29%,14.09%,14.09%,14.09%,14.09%,13.07%,11.88%,11.88%
2022-11,88153.59,21.72%,39.16%,32.13%,38.13%,38.13%,42.44%,36.43%,40.38%,39.00%,35.90%,30.87%,30.87%
2022-12,324164.59,nan%,10.69%,12.35%,18.90%,18.90%,18.90%,20.50%,20.34%,21.81%,20.40%,18.25%,15.43%
2023-01,944943.00,3.59%,9.60%,10.18%,12.61%,12.49%,16.16%,15.19%,16.21%,16.31%,14.97%,13.11%,12.89%
2023-02,1369414.82,6.08%,12.93%,17.60%,17.50%,21.98%,22.43%,23.97%,24.39%,23.02%,21.17%,18.64%,18.50%
2023-03,1002057.00,7.36%,15.25%,19.21%,23.98%,24.81%,27.43%,26.27%,25.46%,25.62%,24.89%,22.25%,21.99%
2023-04,1275193.00,2.04%,8.03%,11.87%,13.70%,14.73%,16.09%,17.85%,17.79%,19.09%,17.44%,17.16%,16.57%


## 金额vintage 30+ 历史最大逾期天数 口径 

In [ ]:
vintage_money_d30 = pd.pivot_table(df_mob_total[df_mob_total.overdue_days_30==1],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum')  #.reset_index()  #,margins=True

vintage_money_d30 = vintage_money_d30[['mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_1',
# vintage_money_d31
vintage_divide_money_d30 = vintage_money_d30.divide(df_mon["purchase_amount"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_money_d30['采购金额'] = df_mon["purchase_amount"]
vintage_divide_money_d30[['采购金额','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]

MOB,采购金额,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2022-07,125405.00,21.60%,32.59%,32.59%,32.59%,34.34%,35.77%,35.77%,35.77%,33.72%,29.57%,29.57%
2022-08,358451.00,15.13%,21.76%,25.99%,30.08%,32.04%,32.81%,33.91%,31.77%,30.79%,26.93%,26.76%
2022-09,277982.75,9.02%,13.86%,16.86%,22.23%,23.12%,23.12%,20.90%,21.01%,17.88%,15.98%,15.98%
2022-10,249543.09,5.86%,5.70%,8.64%,13.66%,15.38%,11.91%,15.88%,15.58%,16.99%,15.27%,15.04%
2022-11,344258.81,10.03%,9.96%,11.50%,11.50%,12.59%,15.24%,15.33%,14.79%,13.97%,12.12%,12.12%
2022-12,566276.59,4.61%,7.42%,14.96%,14.64%,14.56%,17.27%,17.55%,19.03%,17.56%,15.50%,13.89%
2023-01,1533459.00,5.06%,7.87%,10.46%,12.06%,15.83%,15.04%,16.16%,17.11%,16.22%,14.04%,13.97%
2023-02,2181552.82,7.52%,11.40%,14.02%,16.89%,17.46%,18.26%,19.20%,18.54%,17.60%,15.73%,15.55%
2023-03,2083021.00,7.57%,10.74%,14.36%,16.75%,18.66%,18.87%,19.45%,19.89%,19.60%,17.47%,17.56%
2023-04,4317651.50,3.23%,6.77%,9.60%,12.14%,14.01%,15.60%,16.27%,17.38%,16.72%,15.90%,15.77%


## 金额vintage 31+ 历史最大逾期天数 口径 

### 总体

In [ ]:
vintage_money_d31 = pd.pivot_table(df_mob_total[df_mob_total.overdue_days_31==1],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum')  #.reset_index()  #,margins=True

vintage_money_d31 = vintage_money_d31[['mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_1',
# vintage_money_d31
vintage_divide_money_d31 = vintage_money_d31.divide(df_mon["purchase_amount"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_money_d31['采购金额'] = df_mon["purchase_amount"]
vintage_divide_money_d31[['采购金额','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]

MOB,采购金额,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2022-07,125405.00,21.60%,32.59%,32.59%,32.59%,34.34%,35.77%,35.77%,35.77%,33.72%,29.57%,29.57%
2022-08,358451.00,15.13%,21.76%,25.99%,30.08%,30.94%,32.81%,33.91%,31.77%,30.79%,26.93%,26.76%
2022-09,277982.75,9.02%,13.86%,16.86%,22.23%,23.12%,23.12%,20.90%,21.01%,17.88%,15.98%,15.98%
2022-10,249543.09,5.86%,5.70%,8.64%,13.66%,15.38%,11.91%,15.88%,15.58%,16.99%,15.27%,15.04%
2022-11,344258.81,10.03%,9.96%,11.50%,11.50%,12.59%,15.24%,15.33%,14.79%,13.97%,12.12%,12.12%
2022-12,566276.59,4.61%,6.04%,14.96%,14.64%,14.56%,17.27%,17.55%,19.03%,17.56%,15.50%,13.89%
2023-01,1533459.00,5.06%,7.87%,9.96%,12.06%,15.83%,14.89%,16.16%,17.11%,16.22%,14.04%,13.97%
2023-02,2181552.82,7.52%,11.40%,14.02%,16.89%,17.24%,18.26%,19.20%,18.54%,17.60%,15.73%,15.55%
2023-03,2083021.00,7.57%,10.74%,14.36%,16.75%,18.66%,18.58%,19.45%,19.62%,19.57%,17.47%,17.56%
2023-04,4317651.50,3.23%,6.59%,9.60%,12.14%,14.01%,15.60%,16.27%,17.30%,16.72%,15.90%,15.77%


### 租物渠道

In [ ]:

vintage_money_d31_zm = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_31==1) & (df_mob_total.归属渠道=='芝麻租物')],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum')  #.reset_index()  #,margins=True

vintage_money_d31_zm = vintage_money_d31_zm[['mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0','mob_1',

vintage_divide_money_d31_zm = vintage_money_d31_zm.divide(df_mon_zm["purchase_amount"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_money_d31_zm['采购金额'] = df_mon_zm["purchase_amount"]
vintage_divide_money_d31_zm[['采购金额','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]


MOB,采购金额,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2022-09,24742.68,nan%,24.49%,24.49%,29.32%,29.32%,29.32%,29.32%,29.32%,29.32%,26.95%,26.95%
2022-10,48835.00,nan%,nan%,5.58%,5.58%,5.58%,nan%,11.05%,nan%,9.46%,7.74%,7.74%
2022-11,68730.22,nan%,8.67%,8.67%,8.67%,8.67%,8.67%,16.86%,15.91%,15.80%,14.10%,14.10%
2022-12,7657.00,nan%,nan%,nan%,nan%,nan%,49.54%,nan%,nan%,nan%,nan%,nan%
2023-01,110070.00,nan%,3.90%,3.90%,3.90%,11.76%,11.76%,11.76%,12.29%,11.66%,9.53%,9.53%
2023-02,206619.00,3.76%,3.76%,3.76%,5.37%,5.37%,5.37%,5.37%,5.37%,5.37%,5.07%,5.07%
2023-03,374912.00,1.39%,1.39%,2.46%,4.96%,7.68%,9.35%,11.21%,10.77%,12.20%,12.00%,12.00%
2023-04,1628122.00,2.73%,5.47%,7.32%,9.24%,10.68%,12.11%,13.27%,14.99%,14.54%,14.00%,14.24%
2023-05,905764.00,nan%,3.15%,6.05%,6.70%,7.64%,9.31%,10.54%,12.47%,12.15%,11.58%,12.15%
2023-06,1710725.00,2.70%,6.06%,9.94%,11.64%,13.59%,15.22%,16.51%,17.98%,18.39%,17.73%,17.89%


### 搜索渠道

In [ ]:

vintage_money_d31_ss = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_31==1) & (df_mob_total.归属渠道=='搜索渠道')],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum')  #.reset_index()  #,margins=True

vintage_money_d31_ss = vintage_money_d31_ss[['mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0','mob_1',

vintage_divide_money_d31_ss = vintage_money_d31_ss.divide(df_mon_ss["purchase_amount"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_money_d31_ss['采购金额'] = df_mon_ss["purchase_amount"]
vintage_divide_money_d31_ss[['采购金额','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]


MOB,采购金额,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2022-07,46581.00,40.92%,40.92%,40.92%,40.92%,40.92%,40.92%,40.92%,40.92%,38.74%,33.82%,33.82%
2022-08,145114.00,11.41%,17.03%,17.03%,24.74%,24.74%,28.12%,29.55%,25.91%,24.82%,21.85%,21.85%
2022-09,67126.34,37.34%,48.35%,54.03%,53.64%,48.35%,48.35%,48.35%,51.13%,49.44%,44.70%,44.70%
2022-10,44442.63,14.09%,14.09%,14.09%,29.29%,14.09%,14.09%,14.09%,14.09%,13.07%,11.88%,11.88%
2022-11,88153.59,39.16%,32.13%,38.13%,38.13%,38.13%,36.43%,40.38%,39.00%,35.90%,30.87%,30.87%
2022-12,324164.59,5.31%,7.99%,16.75%,18.90%,18.90%,20.50%,20.34%,21.81%,20.40%,18.25%,15.43%
2023-01,944943.00,4.84%,7.49%,10.26%,10.69%,14.14%,13.85%,15.22%,15.68%,14.76%,12.99%,12.89%
2023-02,1369414.82,9.23%,13.37%,16.32%,20.14%,20.41%,22.52%,23.40%,22.41%,20.91%,18.50%,18.50%
2023-03,1002057.00,12.37%,17.71%,23.54%,24.46%,26.12%,24.71%,25.43%,25.13%,24.37%,21.75%,21.99%
2023-04,1275193.00,3.75%,7.34%,12.21%,14.32%,14.59%,16.28%,16.93%,18.46%,17.44%,17.09%,16.57%


## 金额vintage 90+ 历史最大逾期天数 口径 

In [ ]:
vintage_money_d90 = pd.pivot_table(df_mob_total[df_mob_total.overdue_days_90==1],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum')  #.reset_index()  #,margins=True

vintage_money_d90 = vintage_money_d90[['mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0',
# vintage_money_d7
vintage_divide_money_d90 = vintage_money_d90.divide(df_mon["purchase_amount"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_money_d90['采购金额'] = df_mon["purchase_amount"]
vintage_divide_money_d90[['采购金额','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]

MOB,采购金额,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2022-07,125405.00,nan%,21.60%,32.59%,32.59%,32.59%,34.34%,35.77%,33.72%,29.57%,29.57%
2022-08,358451.00,nan%,15.13%,21.76%,25.99%,28.95%,30.31%,30.98%,28.87%,25.32%,26.76%
2022-09,277982.75,nan%,9.02%,13.86%,15.49%,20.95%,20.90%,20.11%,16.98%,15.98%,15.98%
2022-10,249543.09,nan%,5.70%,5.70%,8.64%,7.55%,11.91%,11.91%,13.28%,13.76%,13.52%
2022-11,344258.81,nan%,8.23%,9.96%,11.50%,9.96%,9.96%,10.71%,11.67%,12.12%,12.12%
2022-12,566276.59,nan%,4.57%,5.95%,12.33%,13.56%,14.46%,15.04%,14.95%,14.95%,13.89%
2023-01,1533459.00,nan%,3.50%,6.04%,9.79%,10.82%,13.33%,13.80%,14.23%,13.36%,14.08%
2023-02,2181552.82,0.37%,6.25%,11.65%,13.83%,15.69%,16.02%,16.02%,16.22%,14.67%,14.87%
2023-03,2083021.00,nan%,7.43%,10.84%,13.44%,15.05%,16.55%,16.91%,17.81%,16.78%,17.10%
2023-04,4317651.50,nan%,3.12%,6.08%,8.69%,10.54%,12.89%,14.62%,15.20%,14.69%,15.07%


In [ ]:
# liudan 金额逾期率去除续期订单
dfzh3.relet_periods.value_counts()

relet_periods
0     21676
2       387
3       378
4       375
6       340
5       310
1       144
7        69
8        26
11       13
9        11
10        9
12        6
14        2
13        2
Name: count, dtype: int64

In [ ]:
dfzh3.relet_days.value_counts()

relet_days
0      21676
31       359
61       345
92       319
153      216
       ...  
179        1
118        1
148        1
37         1
2          1
Name: count, Length: 69, dtype: int64

In [ ]:
dfzh3.shape

(23748, 80)

In [ ]:
dfzh3.shape

(23748, 80)

In [ ]:
dfzh3.head()

,id,order_id,order_number,order_time,search_time,end_date,merchant_id,merchant_name,channel_id,channel_name,model_number,specification,purchase_amount,platform_package,all_deposit,all_rental,liquidated_damages,buyout_balance_payment,advance_periods,advance_sum,order_create_time,rembursement_status,paid_periods,begin_overdue_time,overdue_periods,max_overdue_days,now_overdue_days,total_receivable,contract_price,rent_received,unearned_rent_revenue,paid_buyout_balance_payment,total_received,total_unreceived,unrecovered_cost,overdue_type,gross_profit_receivable,overdue_loss,actual_gross_profit,overdue_total_unreceived,relet_periods,relet_days,relet_rent_paid,activity_id,activity_source,order_label,create_time,update_time,order_finish_date,commodity_premium,overdue_fine,pause_reimbursement,create_by_id,update_by_id,del_flag,订单状态值,订单状态,订单完成时间,order_method,true_name,id_card_num,mobile,订单号,渠道名称,归属渠道,内存,颜色,是否二手,下单日期,月份,年份,季度,outpay_money,pay_date,status,if_outpay,finish_date_new,nn,reality_refund_date,real_pay_money
0,1,8924,A202207050851373,2022年07月,2022-07,2024-10-02,14,迪瓜优选,0,未知渠道,99新 iPhone 12 Pro 国行 官方质检【租物特惠】,内存：128G 颜色：金色,5150.0,租完归还/可买断,5303.0,4295.40,0.0,3181.8,1,357.95,2022-07-05 08:51:37,已买断,12,NaT,0,4,0,7477.20,7530.0,7847.20,-3881.52,329.72,8176.92,-699.72,-3026.92,,2327.20,0.0,3026.92,0.0,11,301,3551.8,NaN,未知活动,,2024-07-16 11:27:01,2024-07-18 15:50:56,2024-04-17 01:37:33,0.0,0.0,0,None,None,0,8,已完成,2024-04-17 01:37:33,0,胡水飘,362202199805282550,13027948681,A202207050851373,未知渠道,未知渠道,128G,金色,二手,2022-07-05,7,2022,3季度,329.72,2024-04-17 01:37:33,2.0,已买断,2024-04-17 01:37:33,NaN,NaN,NaN
1,2,9305,A202207131230485,2022年07月,2022-07,2023-07-13,14,迪瓜优选,0,未知渠道,全新 苹果 iPhone 13 Pro Max 国行 全网通【租物特惠】,内存：256G 颜色：银色 套餐：租完归还-可随时买断,0.0,租完归还-可随时买断,9799.0,571.61,0.0,0.0,1,571.61,2022-07-13 12:30:48,已完成,1,NaT,0,0,0,571.61,12739.0,571.61,0.00,0.00,571.61,0.00,-571.61,,571.61,0.0,571.61,0.0,0,0,0.0,NaN,未知活动,违约金,2024-07-16 11:27:01,2024-07-18 15:50:56,2022-07-16 10:19:00,0.0,NaN,0,None,None,0,8,已完成,2022-07-16 10:19:00,0,张素芬,341221199902108727,15585053022,A202207131230485,NaN,未知渠道,256G,银色,全新,2022-07-13,7,2022,3季度,NaN,NaT,NaN,未知,2022-07-16 10:19:00,11.0,0.0,0.0
2,3,9514,A202207151707153,2022年07月,2022-07,2023-07-15,14,迪瓜优选,0,我的小程序入口,全新vivo X80 Pro 高通骁龙8蔡司智能5G【租物特惠】,内存：12GB+256G 颜色：假日 属性：租完归还-可随时买断,4794.0,租完归还-可随时买断,5999.0,4799.16,0.0,2399.6,2,799.86,2022-07-15 17:07:15,已买断,12,NaT,0,1,0,7198.76,7798.0,4799.16,-2598.91,2598.91,7398.07,-199.31,-2604.07,,2404.76,0.0,2604.07,0.0,0,0,0.0,NaN,未知活动,,2024-07-16 11:27:01,2024-07-18 15:50:57,2023-06-30 11:54:29,0.0,NaN,0,None,None,0,8,已完成,2023-06-30 11:54:29,0,彭友宁,360782200204020216,15297728428,A202207151707153,我的小程序入口,我的小程序入口,12GB+256G,假日,全新,2022-07-15,7,2022,3季度,2598.91,2023-06-30 11:54:29,2.0,已买断,2023-06-30 11:54:29,NaN,NaN,NaN
3,4,9600,A202207162356303,2022年07月,2022-07,2024-06-16,14,迪瓜优选,0,其他渠道场景渠道。,全新 苹果 iPhone 13 Pro Max 国行 全网通【租物特惠】,内存：256G 颜色：苍岭绿色 套餐：租完归还-可随时买断,9180.0,租完归还-可随时买断,9799.0,6859.20,0.0,5879.4,2,1143.20,2022-07-16 23:56:30,已买断,12,NaT,0,1,0,12738.60,12739.0,6859.20,-6287.60,6287.60,13146.80,-408.20,-3966.80,,3558.60,0.0,3966.80,0.0,0,0,0.0,NaN,未知活动,,2024-07-16 11:27:02,2024-07-18 15:50:57,2022-12-12 13:01:18,0.0,NaN,0,None,None,0,8,已完成,2022-12-13 10:39:58,0,朱兆成,220181199605113514,17643203514,A202207162356303,其他渠道场景渠道。,其他渠道场景渠道。,256G,苍岭绿色,全新,2022-07-16,7,2022,3季度,6287.60,2022-12-12 13:01:18,2.0,已买断,2022-12-12 13:01:18,NaN,NaN,NaN
4,5,9683,A202207171820512,2022年07月,2022-07,2023-07-17,14,迪瓜优选,0,其他渠道场景渠道。,vivo Y55s 5G手机 6000mAh巨能量电池【租物特惠】,内存：8GB+128GB 颜色：樱粉流星 属性：租完归还-可随时买断,1214.0,租完归还-可随时买断,1699.0,1410.12,0.0,679.6,2,235.02,2022-07-17 18:20:51,已买断,12,NaT,0,27,0,2089.72,2208.0,1410.12,-797.88,797.88,2208.00,-118.28,-994.00,,875.72,0.0,994.00,0.0,0,0,0.0,NaN,未知活动,,2024-07-16 11:27:02,2024-07-18 15:50:57,2024-04-20 18:46:54,0.0,0.0,0,None,None,0,8,已完成,2024-04-20 18:46:54,0,俞晓琳,441602199303132641,13750296584,A202207171820512,其他渠道场景渠道。,其他渠道场景渠道。,8GB+128GB,樱粉流星,全新,2022-07-17,7,2022,3季度,797.88,2024-04-20 18

In [ ]:
# dfzh3.info()

# 以下为 租后表分析部分 

## 每月还款状态 sheet 数据输出

## 每个下单月份的金额口径的当前逾期率

In [ ]:
dfzh3["下单月份"] = dfzh3['search_time']
dfzh3["未回收成本"] = dfzh3['unrecovered_cost']
dfzh3["逾期类型"] = dfzh3['overdue_type']
dfzh3["采购金额"] = dfzh3['purchase_amount']

In [ ]:
df_zt = dfzh3[dfzh3["未回收成本"]>0]    #.groupby(["下单月份","逾期类型"]).agg({"未回收成本":np.sum}).reset_index().pivot(index="下单月份",columns="逾期类型",values="未回收成本")

df_zt['M0'] = np.where(df_zt.逾期类型=='M0',df_zt["未回收成本"],0)
df_zt['M1'] = np.where(df_zt.逾期类型=='M1',df_zt["未回收成本"],0)
df_zt['M2'] = np.where(df_zt.逾期类型=='M2',df_zt["未回收成本"],0)
df_zt['M3'] = np.where(df_zt.逾期类型=='M3',df_zt["未回收成本"],0)
df_zt['M4'] = np.where(df_zt.逾期类型=='M4',df_zt["未回收成本"],0)
df_zt['M5'] = np.where(df_zt.逾期类型=='M5',df_zt["未回收成本"],0)
df_zt['M6'] = np.where(df_zt.逾期类型=='M6',df_zt["未回收成本"],0)
dfyq = df_zt.groupby(["下单月份"]).agg({'M0':'sum','M1':'sum','M2':'sum','M3':'sum','M4':'sum','M5':'sum','M6':'sum'})
# dfyq=dfzh3[dfzh3["未回收成本"]>0].groupby(["下单月份","逾期类型"]).agg({"未回收成本":np.sum}).reset_index().pivot(index="下单月份",columns="逾期类型",values="未回收成本")
dfyq=dfyq.fillna(0)
dfyq.loc[:,"合计逾期"] = dfyq.apply(lambda x:x.sum(),axis=1) 
dfyq1=pd.merge(dfyq,dfzh3.groupby(["下单月份"]).agg({"采购金额":np.sum}),left_on="下单月份",right_on="下单月份",how="left")
dfyq1.loc["总计"]=dfyq1.apply(lambda x:x.sum(),axis=0)
dfyq1["金额逾期率"]=dfyq1["合计逾期"]/dfyq1["采购金额"]
dfyq1["金额逾期率"]=dfyq1["金额逾期率"].apply(lambda x:format(x,".2%"))
dfyq1["M1+逾期率"]=(dfyq1["M6"]+dfyq1["M3"]+dfyq1["M2"]+dfyq1["M1"])/dfyq1["采购金额"]
dfyq1["M1+逾期率"]=dfyq1["M1+逾期率"].apply(lambda x:format(x,".2%"))
dfyq1["M3+逾期率"]=(dfyq1["M6"]+dfyq1["M3"])/dfyq1["采购金额"]
dfyq1["M3+逾期率"]=dfyq1["M3+逾期率"].apply(lambda x:format(x,".2%"))
dfyq1["M6+逾期率"]=dfyq1["M6"]/dfyq1["采购金额"]
dfyq1["M6+逾期率"]=dfyq1["M6+逾期率"].apply(lambda x:format(x,".2%"))
dfyq1_new = dfyq1[['M0','M1','M2','M3','M6','合计逾期','采购金额','金额逾期率','M1+逾期率','M3+逾期率','M6+逾期率']]
dfyq1_new_T = dfyq1_new.T
dfyq1_new_T[['2023-01','2023-02','2023-03','2023-04','2023-05','2023-06','2023-07','2023-08','2023-09','2023-10','2023-11','2023-12','2024-01','2024-02'
                ,'2024-03','2024-04','2024-05','2024-06','2024-07','2024-08','2024-09']]

下单月份,2023-01,2023-02,2023-03,2023-04,2023-05,2023-06,2023-07,2023-08,2023-09,2023-10,2023-11,2023-12,2024-01,2024-02,2024-03,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09
M0,0.0,0.0,0.0,0.0,0.0,2760.88,21370.52,33603.28,131612.97,291851.68,60638.68,89197.91,53449.98,51187.72,159154.44,432312.37,320722.02,483605.83,536775.99,620149.6,557230.23
M1,0.0,0.0,0.0,0.0,442.43,5919.79,22006.54,48357.16,189425.39,47878.6,72750.78,73506.86,49321.06,37074.04,111985.98,212719.9,260810.2,233796.4,340758.76,235752.61,0.0
M2,0.0,0.0,0.0,0.0,486.63,6288.21,60551.33,142620.18,3430.0,39352.12,77731.98,93825.56,61591.53,41332.05,146124.44,305967.18,188971.04,295050.26,181700.1,0.0,0.0
M3,1737.0,0.0,321.1,15608.47,60705.31,89950.31,178458.01,81839.67,193245.44,367929.53,419369.58,330381.29,250898.78,125786.8,371267.28,696113.54,229329.4,169180.09,0.0,0.0,0.0
M6,247277.25,363330.54,406023.06,801450.81,465199.32,654040.68,1017185.7,908738.81,733202.26,1179537.95,1232318.89,566245.03,279440.05,31287.93,87050.0,0.0,0.0,0.0,0.0,0.0,0.0
合计逾期,249014.25,363330.54,406344.16,817059.28,526833.69,758959.87,1299572.1,1215159.1,1250916.06,1926549.88,1862809.91,1153156.65,694701.4,286668.54,875582.14,1647112.99,999832.66,1181632.58,1059234.85,855902.21,557230.23
采购金额,1533459.0,2183082.82,2094813.0,4317651.5,3163820.0,3903259.0,6142708.0,6239848.0,6582098.12,9253766.75,12244154.0,7586793.05,5097888.0,2564381.0,5958558.0,11668812.0,8776880.0,10701621.0,13975350.0,15568622.0,18851756.0
金额逾期率,16.24%,16.64%,19.40%,18.92%,16.65%,19.44%,21.16%,19.47%,19.00%,20.82%,15.21%,15.20%,13.63%,11.18%,14.69%,14.12%,11.39%,11.04%,7.58%,5.50%,2.96%
M1+逾期率,16.24%,16.64%,19.40%,18.92%,16.65%,19.37%,20.81%,18.94%,17.01%,17.67%,14.72%,14.02%,12.58%,9.18%,12.02%,10.41%,7.74%,6.52%,3.74%,1.51%,0.00%
M3+逾期率,16.24%,16.64%,19.40%,18.92%,16.62%,19.06%,19.46%,15.88%,14.08%,16.72%,13.49%,11.82%,10.40%,6.13%,7.69%,5.97%,2.61%,1.58%,0.00%,0.00%,0.00%


In [ ]:
df_zt.head()

,id,order_id,order_number,order_time,search_time,end_date,merchant_id,merchant_name,channel_id,channel_name,model_number,specification,purchase_amount,platform_package,all_deposit,all_rental,liquidated_damages,buyout_balance_payment,advance_periods,advance_sum,order_create_time,rembursement_status,paid_periods,begin_overdue_time,overdue_periods,max_overdue_days,now_overdue_days,total_receivable,contract_price,rent_received,unearned_rent_revenue,paid_buyout_balance_payment,total_received,total_unreceived,unrecovered_cost,overdue_type,gross_profit_receivable,overdue_loss,actual_gross_profit,overdue_total_unreceived,relet_periods,relet_days,relet_rent_paid,activity_id,activity_source,order_label,create_time,update_time,order_finish_date,commodity_premium,overdue_fine,pause_reimbursement,create_by_id,update_by_id,del_flag,订单状态值,订单状态,订单完成时间,order_method,true_name,id_card_num,mobile,订单号,渠道名称,归属渠道,内存,颜色,是否二手,下单日期,月份,年份,季度,outpay_money,pay_date,status,if_outpay,finish_date_new,nn,reality_refund_date,real_pay_money,下单月份,未回收成本,逾期类型,采购金额,M0,M1,M2,M3,M4,M5,M6
7,8,9777,A202207181530165,2022年07月,2022-07,2023-07-18,14,迪瓜优选,0,其他渠道场景渠道。,全新vivo X80 天玑9000旗舰芯片 蔡司T光学镜头【租物特惠】,内存：12GB+256G 颜色：至黑 属性：租完归还-可买断,3478.0,租完归还-可买断,4399.0,3519.24,0.0,1759.6,3,759.84,2022-07-18 15:30:16,已逾期,7,2023-01-18,5,646,646,5278.84,5717.0,1986.24,1533.00,0.0,1986.24,3292.60,1491.76,M6,1800.84,1491.76,-1491.76,3292.60,0,0,0.0,NaN,未知活动,,2024-07-16 11:27:03,2024-07-18 15:50:58,NaT,0.0,NaN,0,None,None,0,5,待归还,NaT,0,马德续,13092919980106279X,17507555066,A202207181530165,其他渠道场景渠道。,其他渠道场景渠道。,12GB+256G,至黑,全新,2022-07-18,7,2022,3季度,NaN,NaT,NaN,未知,NaT,NaN,NaN,NaN,2022-07,1491.76,M6,3478.0,0.0,0.0,0.0,0.0,0.0,0.0,1491.76
10,11,10056,A202207200249265,2022年07月,2022-07,2023-07-20,14,迪瓜优选,0,其他渠道场景渠道。,全新 苹果 iPhone 13 国行 全网通 【租物特惠】,内存：256G 颜色：绿色 属性：租完即送,5875.0,租完即送,6799.0,8838.72,0.0,0.0,1,736.56,2022-07-20 02:49:26,已逾期,5,2022-12-20,7,675,675,8838.72,8839.0,3682.80,5155.92,0.0,3682.80,5155.92,2192.20,M6,2963.72,2192.20,-2192.20,5155.92,0,0,0.0,NaN,未知活动,,2024-07-16 11:27:04,2024-07-18 15:50:59,NaT,0.0,NaN,0,None,None,0,4,租赁中,NaT,0,周超龙,410526199804019072,17788144494,A202207200249265,其他渠道场景渠道。,其他渠道场景渠道。,256G,绿色,全新,2022-07-20,7,2022,3季度,NaN,NaT,NaN,未知,NaT,NaN,NaN,NaN,2022-07,2192.20,M6,5875.0,0.0,0.0,0.0,0.0,0.0,0.0,2192.20
11,12,10075,A202207201414474,2022年07月,2022-07,2023-07-20,14,迪瓜优选,0,未知渠道,全新 苹果 iPhone 13 Pro Max 国行 全网通【租物特惠】,内存：256G 颜色：石墨色 套餐：租完即送,9230.0,租完即送,9799.0,12738.72,0.0,0.0,2,2123.12,2022-07-20 14:14:47,已逾期,3,2022-09-20,9,766,766,12738.72,12739.0,3184.68,9554.04,0.0,3184.68,9554.04,6045.32,M6,3508.72,6045.32,-6045.32,9554.04,0,0,0.0,NaN,未知活动,,2024-07-16 11:27:04,2024-07-18 15:50:59,NaT,0.0,NaN,0,None,None,0,4,租赁中,NaT,0,吴迪,522101199005212413,18585215250,A202207201414474,未知渠道,未知渠道,256G,石墨色,全新,2022-07-20,7,2022,3季度,NaN,NaT,NaN,未知,NaT,NaN,NaN,NaN,2022-07,6045.32,M6,9230.0,0.0,0.0,0.0,0.0,0.0,0.0,6045.32
12,13,10099,A202207201828125,2022年07月,2022-07,2023-07-20,14,迪瓜优选,0,顶部搜索框的搜索结果页,全新vivo X80 Pro 高通骁龙8蔡司智能5G【租物特惠】,内存：12GB+256G 颜色：至黑 属性：租完即送,4794.0,租完即送,5499.0,7798.68,0.0,0.0,2,1004.38,2022-07-20 18:28:12,已逾期,2,2022-08-20,10,797,797,7798.68,7799.0,1004.38,6794.30,0.0,1004.38,6794.30,3789.62,M6,3004.68,3789.62,-3789.62,6794.30,0,0,0.0,NaN,未知活动,,2024-07-16 11:27:05,2024-07-18 15:51:00,NaT,0.0,NaN,0,None,None,0,4,租赁中,NaT,0,崔宇文,320882199905254210,18118118211,A202207201828125,顶部搜索框的搜索结果页,搜索渠道,12GB+256G,至黑,全新,2022-07-20,7,2022,3季度,NaN,NaT,NaN,未知,NaT,NaN,NaN,NaN,2022-07,3789.62,M6,4794.0,0.0,0.0,0.0,0.0,0.0,0.0,3789.62
16,17,11564,A202207272231334,2022年07月,2022-07,2023-07-27,14,迪瓜优选,0,其他渠道场景渠道。,全新 苹果 iPhone 13 Pro Max 国行 全网通【租物特惠】,内存：128G 颜色：石墨色 套餐：租完归还-可随时买断,8280.0,租完归还-可随时买断,8999.0,6119.98,0.0,5399.4,3,1321.36,2022-07-27 22:31:33,已逾期,3,2022-08-27,9,790,790,11519.38,11699.0,1321.36,4798.62,0.0,1321.36,10198.02,6958.64,M6,3239.38,6958.64,-6958.64,10198.02,0,0,0.0,NaN,未知活动,,2024-07-16 11:27:06,2024-07-18 15:51:01,NaT,0.0,NaN,0,None,None,0,5,待归还,NaT,0,宋钰航,522426200306046999,18798397677,A202207

In [ ]:
date_list = ['2023-07','2023-08']
df_ztmm = df_zt[df_zt['下单月份'].isin(date_list)]    # t_postlease_receivables_monitoring
df_ztmm.shape

(948, 91)

In [ ]:
# df_ztmm.to_excel('E:/myfile/dfztmm.xlsx')

非续租用户对比

In [ ]:
df_ztmm_7  = df_ztmm[df_ztmm['下单月份']=='2023-07']
df_ztmm_7.shape

(458, 91)

In [ ]:
df_ztmm_7.head()

,id,order_id,order_number,order_time,search_time,end_date,merchant_id,merchant_name,channel_id,channel_name,model_number,specification,purchase_amount,platform_package,all_deposit,all_rental,liquidated_damages,buyout_balance_payment,advance_periods,advance_sum,order_create_time,rembursement_status,paid_periods,begin_overdue_time,overdue_periods,max_overdue_days,now_overdue_days,total_receivable,contract_price,rent_received,unearned_rent_revenue,paid_buyout_balance_payment,total_received,total_unreceived,unrecovered_cost,overdue_type,gross_profit_receivable,overdue_loss,actual_gross_profit,overdue_total_unreceived,relet_periods,relet_days,relet_rent_paid,activity_id,activity_source,order_label,create_time,update_time,order_finish_date,commodity_premium,overdue_fine,pause_reimbursement,create_by_id,update_by_id,del_flag,订单状态值,订单状态,订单完成时间,order_method,true_name,id_card_num,mobile,订单号,渠道名称,归属渠道,内存,颜色,是否二手,下单日期,月份,年份,季度,outpay_money,pay_date,status,if_outpay,finish_date_new,nn,reality_refund_date,real_pay_money,下单月份,未回收成本,逾期类型,采购金额,M0,M1,M2,M3,M4,M5,M6
2857,2938,431052,A202307010727469,2023年07月,2023-07,2024-07-03,14,迪瓜优选,195,芝麻租物,iPhone 14 Pro,内存：128G 颜色：暗紫色,6600.0,租完归还/可买断,7999.0,3527.58,0.0,4799.4,2,322.38,2023-07-01 07:27:47,检测中,12,2024-07-03,0,114,114,8326.98,10399.0,3527.58,0.00,0.0,3527.58,4799.40,3072.42,M3,1726.98,0.00,0.00,0.00,0,0,0.00,32.0,芝麻租物-苹果手机,,2024-07-16 16:35:06,2024-07-18 20:59:59,NaT,0.0,NaN,0,None,None,0,15,检测中,NaT,0,朱地江,341226200007103230,17778092831,NaN,NaN,芝麻租物,128G,暗紫色,全新,2023-07-01,7,2023,3季度,NaN,NaT,NaN,未知,NaT,NaN,NaN,NaN,2023-07,3072.42,M3,6600.0,0.00,0.0,0.0,3072.42,0.0,0.0,0.00
2858,2939,431123,A202307010902211,2023年07月,2023-07,2025-07-29,14,迪瓜优选,3,顶部搜索框的搜索结果页,iPhone14,内存：256G 颜色：紫色,5950.0,租完归还/可买断,6899.0,3732.82,0.0,4139.4,2,341.02,2023-07-01 09:02:21,续租中,12,NaT,0,0,0,7872.22,8969.0,5856.66,-2123.84,0.0,5856.66,2015.56,93.34,,1922.22,0.00,1922.22,0.00,5,123,2123.84,117.0,热租精选feeds,,2024-07-16 16:35:15,2024-07-18 21:00:08,NaT,0.0,NaN,0,None,None,0,4,租赁中,NaT,0,周俊文,510626199807127393,15984940823,NaN,NaN,搜索渠道,256G,紫色,全新,2023-07-01,7,2023,3季度,NaN,NaT,NaN,未知,NaT,NaN,NaN,NaN,2023-07,93.34,,5950.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00
2861,2942,431253,A202307011100232,2023年07月,2023-07,2024-07-03,14,迪瓜优选,4,单人聊天会话中的小程序消息卡片（分享）,iPhone 14 Pro Max,内存：256G 颜色：银色,8220.0,租完归还/可买断,9899.0,6418.44,0.0,5939.4,2,585.84,2023-07-01 11:00:24,已逾期,10,2024-04-04,2,204,204,12357.84,12869.0,5251.92,1166.52,0.0,5251.92,7105.92,2968.08,M6,4137.84,2968.08,-2968.08,7105.92,0,0,0.00,32.0,芝麻租物-苹果手机,,2024-07-16 16:35:30,2024-07-18 21:00:25,NaT,0.0,0.00,0,None,None,0,5,待归还,NaT,0,程佳宝,350581200007101528,13295000540,NaN,NaN,芝麻租物,256G,银色,全新,2023-07-01,7,2023,3季度,NaN,NaT,NaN,未知,NaT,NaN,NaN,NaN,2023-07,2968.08,M6,8220.0,0.00,0.0,0.0,0.00,0.0,0.0,2968.08
2862,2943,431254,A202307011101353,2023年07月,2023-07,2025-07-29,14,迪瓜优选,274,租物灯火ocpc,iPhone14,内存：128GB 颜色：午夜色,5020.0,租完归还/可买断,5999.0,3443.15,0.0,3599.4,2,314.65,2023-07-01 11:01:35,已逾期,12,2024-10-03,1,6,22,7042.55,7799.0,4767.83,-1324.68,0.0,4767.83,2274.72,252.17,M0,2022.55,252.17,-252.17,2274.72,4,92,1324.68,136.0,租物灯火OCPC,,2024-07-16 16:35:30,2024-07-18 21:00:25,NaT,0.0,0.00,0,None,None,0,4,租赁中,NaT,0,张小华,522227198203045219,13250208400,NaN,NaN,芝麻租物,128GB,午夜色,全新,2023-07-01,7,2023,3季度,NaN,NaT,NaN,未知,NaT,NaN,NaN,NaN,2023-07,252.17,M0,5020.0,252.17,0.0,0.0,0.00,0.0,0.0,0.00
2863,2944,431413,A2023070113204620,2023年07月,2023-07,2025-06-28,14,迪瓜优选,195,芝麻租物,iPhone 14 Pro,内存：128G 颜色：深空黑色,6580.0,租完归还/可买断,7999.0,3527.58,0.0,4799.4,2,322.38,2023-07-01 13:20:46,已逾期,12,2024-07-03,4,42,114,8340.44,10399.0,3527.58,-13.46,0.0,3541.04,4799.40,3038.96,M3,1760.44,3038.96,-3038.96,4799.40,0,0,0.00,32.0,芝麻租物-苹果手机,,2024-07-16 16:35:38,2024-07-18 21:00:34,NaT,0.0,13.46,0,None,None,0,4,租赁中,NaT,0,陈广泽,440881200407180614,17802009363,NaN,NaN,芝麻租物,128G,深空黑色,全新,2023-07-01,7,2023,3季度,NaN,NaT,NaN,未知,NaT,NaN,NaN,NaN,2023-07,3038.96,M3,6580.0,0.00,0.0,0.0,3038.96,0.0,0.0,0.00


In [ ]:
df_ztmm_7_list = list(set(df_ztmm_7.order_number.tolist()))
len(df_ztmm_7_list)

458

In [ ]:
df_mobv2.columns

Index(['order_number', '下单日期', '当前应付租金', 'paid_money_mob', 'overdue_days_mob',
       'MOB', 'overdue_days_1', 'overdue_days_4', 'overdue_days_7',
       'overdue_days_15', 'overdue_days_31', 'overdue_days_30',
       'overdue_days_90', '下单月份', 'purchase_amount', '归属渠道',
       'unpaid_money_mob'],
      dtype='object')

In [ ]:
df_mobv2 = df_mobv2 [df_mobv2['下单月份']=='2023-07']
df_mobv2.shape

(2201, 17)

In [ ]:
df_mobv2.head()

,order_number,下单日期,当前应付租金,paid_money_mob,overdue_days_mob,MOB,overdue_days_1,overdue_days_4,overdue_days_7,overdue_days_15,overdue_days_31,overdue_days_30,overdue_days_90,下单月份,purchase_amount,归属渠道,unpaid_money_mob
23187,A2023070315084215,2023-07-03,352.54,1.00,25,mob_1,1,1,1,1,0,0,0,2023-07,7400.0,芝麻租物,7399.00
23207,A20230704133219126,2023-07-04,1542.66,771.33,24,mob_1,1,1,1,1,0,0,0,2023-07,6090.0,搜索渠道,5318.67
23214,A2023070416321099,2023-07-04,294.10,147.05,24,mob_1,1,1,1,1,0,0,0,2023-07,3165.0,芝麻租物,3017.95
23257,A2023070614505672,2023-07-06,2144.36,1072.18,22,mob_1,1,1,1,1,0,0,0,2023-07,8230.0,搜索渠道,7157.82
23266,A20230707005856114,2023-07-07,540.80,1.00,21,mob_1,1,1,1,1,0,0,0,2023-07,7435.0,搜索渠道,7434.00


In [ ]:
df_mobv2.order_number.value_counts()

order_number
A202307191603464      12
A2023072007455044     12
A20230704133219126    12
A2023071519211977     12
A2023072011152536     12
                      ..
A2023071708233943      1
A20230730214954119     1
A2023072506150115      1
A2023072517105944      1
A2023072615120418      1
Name: count, Length: 313, dtype: int64

In [ ]:
df_mobv2_list = list(set(df_mobv2.order_number.tolist()))
len(df_mobv2_list)

313

In [ ]:
df_diff =df_ztmm_7 [~df_ztmm_7['order_number'].isin(df_mobv2_list)]   
df_diff.shape# df_ztmm_7_list

(189, 91)

In [ ]:
df_diff

,id,order_id,order_number,order_time,search_time,end_date,merchant_id,merchant_name,channel_id,channel_name,model_number,specification,purchase_amount,platform_package,all_deposit,all_rental,liquidated_damages,buyout_balance_payment,advance_periods,advance_sum,order_create_time,rembursement_status,paid_periods,begin_overdue_time,overdue_periods,max_overdue_days,now_overdue_days,total_receivable,contract_price,rent_received,unearned_rent_revenue,paid_buyout_balance_payment,total_received,total_unreceived,unrecovered_cost,overdue_type,gross_profit_receivable,overdue_loss,actual_gross_profit,overdue_total_unreceived,relet_periods,relet_days,relet_rent_paid,activity_id,activity_source,order_label,create_time,update_time,order_finish_date,commodity_premium,overdue_fine,pause_reimbursement,create_by_id,update_by_id,del_flag,订单状态值,订单状态,订单完成时间,order_method,true_name,id_card_num,mobile,订单号,渠道名称,归属渠道,内存,颜色,是否二手,下单日期,月份,年份,季度,outpay_money,pay_date,status,if_outpay,finish_date_new,nn,reality_refund_date,real_pay_money,下单月份,未回收成本,逾期类型,采购金额,M0,M1,M2,M3,M4,M5,M6
2857,2938,431052,A202307010727469,2023年07月,2023-07,2024-07-03,14,迪瓜优选,195,芝麻租物,iPhone 14 Pro,内存：128G 颜色：暗紫色,6600.0,租完归还/可买断,7999.0,3527.58,0.0,4799.4,2,322.38,2023-07-01 07:27:47,检测中,12,2024-07-03,0,114,114,8326.98,10399.0,3527.58,0.00,0.0,3527.58,4799.40,3072.42,M3,1726.98,0.00,0.00,0.00,0,0,0.00,32.0,芝麻租物-苹果手机,,2024-07-16 16:35:06,2024-07-18 20:59:59,NaT,0.0,NaN,0,None,None,0,15,检测中,NaT,0,朱地江,341226200007103230,17778092831,NaN,NaN,芝麻租物,128G,暗紫色,全新,2023-07-01,7,2023,3季度,NaN,NaT,NaN,未知,NaT,NaN,NaN,NaN,2023-07,3072.42,M3,6600.0,0.00,0.00,0.00,3072.42,0.0,0.0,0.0
2858,2939,431123,A202307010902211,2023年07月,2023-07,2025-07-29,14,迪瓜优选,3,顶部搜索框的搜索结果页,iPhone14,内存：256G 颜色：紫色,5950.0,租完归还/可买断,6899.0,3732.82,0.0,4139.4,2,341.02,2023-07-01 09:02:21,续租中,12,NaT,0,0,0,7872.22,8969.0,5856.66,-2123.84,0.0,5856.66,2015.56,93.34,,1922.22,0.00,1922.22,0.00,5,123,2123.84,117.0,热租精选feeds,,2024-07-16 16:35:15,2024-07-18 21:00:08,NaT,0.0,NaN,0,None,None,0,4,租赁中,NaT,0,周俊文,510626199807127393,15984940823,NaN,NaN,搜索渠道,256G,紫色,全新,2023-07-01,7,2023,3季度,NaN,NaT,NaN,未知,NaT,NaN,NaN,NaN,2023-07,93.34,,5950.0,0.00,0.00,0.00,0.00,0.0,0.0,0.0
2862,2943,431254,A202307011101353,2023年07月,2023-07,2025-07-29,14,迪瓜优选,274,租物灯火ocpc,iPhone14,内存：128GB 颜色：午夜色,5020.0,租完归还/可买断,5999.0,3443.15,0.0,3599.4,2,314.65,2023-07-01 11:01:35,已逾期,12,2024-10-03,1,6,22,7042.55,7799.0,4767.83,-1324.68,0.0,4767.83,2274.72,252.17,M0,2022.55,252.17,-252.17,2274.72,4,92,1324.68,136.0,租物灯火OCPC,,2024-07-16 16:35:30,2024-07-18 21:00:25,NaT,0.0,0.00,0,None,None,0,4,租赁中,NaT,0,张小华,522227198203045219,13250208400,NaN,NaN,芝麻租物,128GB,午夜色,全新,2023-07-01,7,2023,3季度,NaN,NaT,NaN,未知,NaT,NaN,NaN,NaN,2023-07,252.17,M0,5020.0,252.17,0.00,0.00,0.00,0.0,0.0,0.0
2872,2954,432115,A2023070210124915,2023年07月,2023-07,2025-07-30,14,迪瓜优选,195,芝麻租物,iPhone 14 Pro Max,内存：128G 颜色：银色,7285.0,租完归还/可买断,8999.0,3868.86,0.0,5399.4,2,353.46,2023-07-02 10:12:50,续租中,12,NaT,0,15,0,9277.05,11699.0,7044.90,-3184.83,0.0,7053.69,2223.36,231.31,,1992.05,0.00,1992.05,0.00,5,123,3176.04,32.0,芝麻租物-苹果手机,,2024-07-16 16:36:59,2024-07-18 21:01:52,NaT,0.0,8.79,0,None,None,0,4,租赁中,NaT,0,李嘉翊,45080220020702869X,17396772058,NaN,NaN,芝麻租物,128G,银色,全新,2023-07-02,7,2023,3季度,NaN,NaT,NaN,未知,NaT,NaN,NaN,NaN,2023-07,231.31,,7285.0,0.00,0.00,0.00,0.00,0.0,0.0,0.0
2878,2962,432650,A2023070214532554,2023年07月,2023-07,2025-07-30,14,迪瓜优选,6,小程序商家消息（服务提醒）,iPhone 14 Pro Max,内存：256G 颜色：暗紫色,8230.0,租完归还/可买断,9899.0,6534.21,0.0,5939.4,2,596.31,2023-07-02 14:53:25,已逾期,12,2024-09-04,2,10,51,12493.22,12869.0,7818.73,-1304.13,0.0,7838.34,4654.88,391.66,M1,4263.22,391.66,-391.66,4654.88,3,62,1284.52,99.0,叮咚直播,,2024-07-16 16:37:55,2024-07-18 21:02:47,NaT,0.0,19.61,0,None,None,0,4,租赁中,NaT,0,郑月开,362329199704081114,15280446233,NaN,NaN,小程序商家消息（服务提醒）,256G,暗紫色,全新,2023-07-02,7,2023,3季度,NaN,NaT,NaN,未知,NaT,NaN,NaN,NaN,2023-07,391.66,M1,8230.0,0.00,391.66,0.00,0.00,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

In [ ]:
df_diff.columns

Index(['id', 'order_id', 'order_number', 'order_time', 'search_time',
       'end_date', 'merchant_id', 'merchant_name', 'channel_id',
       'channel_name', 'model_number', 'specification', 'purchase_amount',
       'platform_package', 'all_deposit', 'all_rental', 'liquidated_damages',
       'buyout_balance_payment', 'advance_periods', 'advance_sum',
       'order_create_time', 'rembursement_status', 'paid_periods',
       'begin_overdue_time', 'overdue_periods', 'max_overdue_days',
       'now_overdue_days', 'total_receivable', 'contract_price',
       'rent_received', 'unearned_rent_revenue', 'paid_buyout_balance_payment',
       'total_received', 'total_unreceived', 'unrecovered_cost',
       'overdue_type', 'gross_profit_receivable', 'overdue_loss',
       'actual_gross_profit', 'overdue_total_unreceived', 'relet_periods',
       'relet_days', 'relet_rent_paid', 'activity_id', 'activity_source',
       'order_label', 'create_time', 'update_time', 'order_finish_date',
       'comm

In [ ]:
df_diff.订单状态.value_counts()

订单状态
租赁中    132
待归还     49
检测中      4
已完成      4
Name: count, dtype: int64

In [ ]:
df_diff.逾期类型.value_counts()

逾期类型
      92
M3    38
M2    30
M0    19
M1    10
Name: count, dtype: int64

In [ ]:
df_diff.if_outpay.value_counts()# df_diff.未回收成本.value_counts()

if_outpay
未知      187
部分买断      2
Name: count, dtype: int64

In [ ]:
set_add = set(df_mobv2.order_number.tolist()) & set(df_ztmm_7.order_number.tolist())
len(set_add)

269

In [ ]:
297-257

40

In [ ]:
322 - 257

65

In [ ]:
40/ 297

0.13468013468013468

In [ ]:
sql_tor = '''
SELECT tprm.*
,tor.overdue_day
,tor.amount
,tor.overdue_type
from db_digua_business.t_postlease_receivables_monitoring tprm
left join db_digua_business.t_overdue_record tor on tprm.order_id = tor.order_id
where tprm.search_time = '2023-07'                                
and tor.overdue_type in (1,2)
'''
# tor.amount = 买断价-已付租金

In [ ]:
df_tor = query(sql_tor)
df_tor.shape

(428, 58)

In [ ]:
df_tor = drop_sj(df_tor,'merchant_name','model_number')
df_tor.shape

(403, 58)

In [ ]:
df_tor.head()

,id,order_id,order_number,order_time,search_time,end_date,merchant_id,merchant_name,channel_id,channel_name,model_number,specification,purchase_amount,platform_package,all_deposit,all_rental,liquidated_damages,buyout_balance_payment,advance_periods,advance_sum,order_create_time,rembursement_status,paid_periods,begin_overdue_time,overdue_periods,max_overdue_days,now_overdue_days,total_receivable,contract_price,rent_received,unearned_rent_revenue,paid_buyout_balance_payment,total_received,total_unreceived,unrecovered_cost,overdue_type,gross_profit_receivable,overdue_loss,actual_gross_profit,overdue_total_unreceived,relet_periods,relet_days,relet_rent_paid,activity_id,activity_source,order_label,create_time,update_time,order_finish_date,commodity_premium,overdue_fine,pause_reimbursement,create_by_id,update_by_id,del_flag,overdue_day,amount,overdue_type
0,2942,431253,A202307011100232,2023年07月,2023-07,2024-07-03,14,迪瓜优选,4,单人聊天会话中的小程序消息卡片（分享）,iPhone 14 Pro Max,内存：256G 颜色：银色,8220.0,租完归还/可买断,9899.0,6418.44,0.0,5939.4,2,585.84,2023-07-01 11:00:24,已逾期,10,2024-04-04,2,204,204,12357.84,12869.0,5251.92,1166.52,0.0,5251.92,7105.92,2968.08,M6,4137.84,2968.08,-2968.08,7105.92,0,0,0.0,32.0,芝麻租物-苹果手机,,2024-07-16 16:35:30,2024-07-18 21:00:25,NaT,0.0,0.00,0,None,None,0,204,7617.08,2
1,2946,431535,A2023070115114814,2023年07月,2023-07,2024-07-03,14,迪瓜优选,195,芝麻租物,小米13,内存：8+128GB 颜色：黑色,3400.0,租完归还/可买断,3999.0,2620.84,0.0,1599.6,2,239.74,2023-07-01 15:11:49,已逾期,7,2024-01-04,5,295,295,4220.44,5199.0,1430.29,1190.55,0.0,1430.29,2790.15,1969.71,M6,820.44,1969.71,-1969.71,2790.15,0,0,0.0,40.0,芝麻租物-小米手机,,2024-07-16 16:35:55,2024-07-18 21:00:49,NaT,0.0,NaN,0,None,None,0,295,3768.71,2
2,2947,431563,A2023070115535442,2023年07月,2023-07,2024-07-03,14,迪瓜优选,20,其他渠道场景渠道。,Apple Watch Ultra,版本：GPS+蜂窝款 49 毫米 型号：蓝配灰色野径回环式表带,5635.0,租完归还/可买断,6299.0,5354.00,0.0,2519.6,1,446.07,2023-07-01 15:53:54,已逾期,9,2024-04-04,3,204,204,7878.95,8189.0,4014.63,1334.02,0.0,4019.98,3858.97,1615.02,M6,2243.95,1615.02,-1615.02,3858.97,0,0,0.0,42.0,芝麻租物-智能手表,,2024-07-16 16:36:03,2024-07-18 21:00:57,NaT,0.0,5.35,0,None,None,0,204,4174.37,2
3,2950,432069,A202307020902353,2023年07月,2023-07,2024-07-04,14,迪瓜优选,195,芝麻租物,Redmi Note 12 Turbo,内存：16GB+1TB 颜色：星海蓝,2260.0,租完归还/可买断,2599.0,2079.00,0.0,1039.6,2,346.90,2023-07-02 09:02:36,已买断,12,NaT,0,65,0,3140.83,3379.0,2079.00,-1322.23,1300.0,3401.23,-260.40,-1141.23,,880.83,0.00,1141.23,0.00,0,0,0.0,101.0,芝麻租物--全新 Redmi Note 12 Turbo/Redmi 12C,,2024-07-16 16:36:28,2024-07-18 21:01:21,2024-09-07 10:11:14,0.0,22.23,0,None,None,0,53,1300.00,2
4,2953,432095,A2023070209515129,2023年07月,2023-07,2024-07-04,14,迪瓜优选,195,芝麻租物,iPhone 14 Pro Max,内存：256G 颜色：暗紫色,8195.0,租完归还/可买断,9899.0,6418.44,0.0,5939.4,2,584.26,2023-07-02 09:51:51,已逾期,7,2024-02-05,5,263,263,12357.84,12869.0,3500.56,2917.88,0.0,3500.56,8857.28,4694.44,M6,4162.84,4694.44,-4694.44,8857.28,0,0,0.0,32.0,芝麻租物-苹果手机,,2024-07-16 16:36:51,2024-07-18 21:01:44,NaT,0.0,NaN,0,None,None,0,294,9368.44,2


In [ ]:
len(set(df_tor.order_number))

380

In [ ]:
df_tor_no = df_tor.drop_duplicates(subset='order_number',keep='last')
df_tor_no.shape

(380, 58)

In [ ]:
df_tor_no.columns

Index(['id', 'order_id', 'order_number', 'order_time', 'search_time',
       'end_date', 'merchant_id', 'merchant_name', 'channel_id',
       'channel_name', 'model_number', 'specification', 'purchase_amount',
       'platform_package', 'all_deposit', 'all_rental', 'liquidated_damages',
       'buyout_balance_payment', 'advance_periods', 'advance_sum',
       'order_create_time', 'rembursement_status', 'paid_periods',
       'begin_overdue_time', 'overdue_periods', 'max_overdue_days',
       'now_overdue_days', 'total_receivable', 'contract_price',
       'rent_received', 'unearned_rent_revenue', 'paid_buyout_balance_payment',
       'total_received', 'total_unreceived', 'unrecovered_cost',
       'overdue_type', 'gross_profit_receivable', 'overdue_loss',
       'actual_gross_profit', 'overdue_total_unreceived', 'relet_periods',
       'relet_days', 'relet_rent_paid', 'activity_id', 'activity_source',
       'order_label', 'create_time', 'update_time', 'order_finish_date',
       'comm

In [ ]:
df_tor_no.amount.sum()

np.float64(2899311.63)

In [ ]:
# total_received  purchase_amount  
df_tor_no['upaid_amount_all2'] = df_tor_no['purchase_amount']- df_tor_no['total_received']
df_tor_no['upaid_amount_all2'] 

0      2968.08
1      1969.71
2      1615.02
3     -1141.23
4      4694.44
        ...   
422    3973.44
423    2895.85
424    4005.05
425    4306.93
426    3654.51
Name: upaid_amount_all2, Length: 380, dtype: float64

In [ ]:
df_tor_no[df_tor_no['upaid_amount_all2'] <0 ]['upaid_amount_all2'] 

3     -1141.23
5     -4770.77
41    -1362.64
42    -4139.00
54       -6.14
        ...   
405   -1195.46
406   -4741.93
407   -1283.75
412   -1543.75
420    -115.08
Name: upaid_amount_all2, Length: 72, dtype: float64

In [ ]:
# 标准线
df_tor_no.upaid_amount_all2.sum()

np.float64(1096399.53)

In [ ]:
df_tor_no.upaid_amount_all2.sum()/3971396

np.float64(0.2760740883054724)

In [ ]:
# 金额逾期率
3971396* 0.3137

1245826.9252

In [ ]:
# vintage
overdue_amount_tor_no = 3971396*0.2637
overdue_amount_tor_no

1047257.1252

In [ ]:
df_tor_no.upaid_amount_all2.sum() - overdue_amount_tor_no 

np.float64(49142.40480000002)

In [ ]:
3971396* 0.3137 - df_tor_no.upaid_amount_all2.sum()

np.float64(149427.3951999999)

In [ ]:
# df_ztmm_7 df_mobv2_list  df_ztmm_7_list
# 在逾期记录表的订单
df_tor_ztmm =df_ztmm_7 [df_ztmm_7['order_number'].isin(df_tor_no.order_number.tolist())] 
df_tor_ztmm .shape

(308, 91)

In [ ]:
df_tor_mob2 =df_mobv2[df_mobv2['order_number'].isin(df_tor_no.order_number.tolist())] 
df_tor_mob2.shape

(2079, 17)

In [ ]:
len(set(df_tor_mob2.order_number.tolist()))

269

In [ ]:
df_mobv2.shape

(2201, 17)

In [ ]:
df_ztmm_7.shape

(458, 91)

In [ ]:
# df_tor_no.to_excel('')